In [1]:
import pandas as pd, sqlalchemy as sql, numpy as np, datetime as dt, os, re, time
import operator, requests, string as s, random
import plotly.graph_objects as go
import plotly.express as px

In [2]:
import bs4

In [3]:
from spotify_keys import CLIENT_ID, CLIENT_SECRET, SPOTIFY_BEARER_KEY

# There are 2 Main Sources of Data:
* Data World Data
    * hot100.csv
        * Complete Hot 100 Data from 1999 to 2019. corresponds to 29,154 Songs
    * audioFeatures.csv
        * Spotify Audio Features of Billboard Songs corresponds to 28,377 Songs
* Kaggle Data
    * billboardHot100_1999-2019.csv
        * Incomplete Hot 100 Data from 1999 to 2019
    * songAttributes_1999-2019.csv
        * Spotify Attributes for 150,000+ songs
        * They do not have a direct correspondence with the Hot 100 Songs. But provide a meaningful sample of Songs.
            * If I recreate the distribution of songs by Top level Genre in the Billboard Hot100 I should be able to get a meaningful sample of songs to compare against the more popular Songs

<h1 style="color: #23ADAA" > GOALS:</h1>
<ol>
<li> Locate the Songs with Missing Attributes in audioFeatures.csv</li>
<li> Compare those missing against Songs in the other song Attributes file.</li>
<li> Create a Structured Data Base with Tables corresponding to:
    <ul>
    <li> Time Series of Songs over each week with the wanted Spotify attributes and a column for Top level Genre.</li>
    <li> A Sample of Songs that were released in the same time period (1999 - 2019) and Have the Same Distribution of the Top Level Genre as The Hot 100. </li>
    </ul>
</ol>
<br>
<p>These tables will help generate a Horse Race Chart that will show Chart Progression Over Time and Radar Charts Graphing the Hot 100 Attributes Against The Sample of Other Songs. There will additionally be aggregate statistics about the overall year in music on how music was structured and experienced given the attributes of popular songs and all other songs we have.</p>

In [958]:
# Main Data
df_hot = pd.read_csv("../data/raw/hot100.csv")

In [959]:
df_hot.shape

(325695, 10)

In [960]:
df_hot.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1963-0...,6/1/1963,11,Still,Bill Anderson,StillBill Anderson,1.0,17.0,11.0,8.0
1,http://www.billboard.com/charts/hot-100/1967-0...,1/7/1967,11,Coming Home Soldier,Bobby Vinton,Coming Home SoldierBobby Vinton,1.0,17.0,11.0,8.0
2,http://www.billboard.com/charts/hot-100/1971-0...,7/3/1971,11,She's Not Just Another Woman,The 8th Day,She's Not Just Another WomanThe 8th Day,1.0,17.0,11.0,8.0
3,http://www.billboard.com/charts/hot-100/1975-1...,11/29/1975,11,Saturday Night,Bay City Rollers,Saturday NightBay City Rollers,1.0,17.0,11.0,8.0
4,http://www.billboard.com/charts/hot-100/1979-0...,9/29/1979,11,Pop Muzik,M,Pop MuzikM,1.0,17.0,11.0,8.0


In [961]:
df_hot.WeekID = pd.to_datetime(df_hot.WeekID)

In [962]:
df_hot.sort_values("WeekID")

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
17651,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02,72,Stupid Cupid,Connie Francis,Stupid CupidConnie Francis,1.0,NaN,72.0,1.0
6984,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02,54,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,Nel Blu Dipinto Di Blu (Volaré)Domenico Modugno,1.0,NaN,54.0,1.0
22741,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02,87,Jealousy,Kitty Wells,JealousyKitty Wells,1.0,NaN,87.0,1.0
294973,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02,5,When,Kalin Twins,WhenKalin Twins,1.0,NaN,5.0,1.0
318351,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02,79,The Bird On My Head,David Seville,The Bird On My HeadDavid Seville,1.0,NaN,79.0,1.0
...,...,...,...,...,...,...,...,...,...,...
309370,https://www.billboard.com/charts/hot-100/2020-...,2020-12-26,26,Happy Holiday / The Holiday Season,Andy Williams,Happy Holiday / The Holiday SeasonAndy Williams,NaN,NaN,NaN,NaN
309369,https://www.billboard.com/charts/hot-100/2020-...,2020-12-26,25,For The Night,Pop Smoke Featuring Lil Baby & DaBaby,For The NightPop Smoke Featuring Lil Baby & Da...,NaN,NaN,NaN,NaN
309427,https://www.billboard.com/charts/hot-100/2020-...,2020-12-26,83,Love You Like I Used To,Russell Dickerson,Love You Like I Used ToRussell Dickerson,NaN,NaN,NaN,NaN
309348,https://www.billboard.com/charts/hot-100/2020-...,2020-12-26,4,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,NaN,NaN,NaN,NaN


In [963]:
# We only want data from 1999 - 2020
df_hot = df_hot[(df_hot.WeekID >= dt.datetime(1999, 1, 1)) & (df_hot.WeekID < dt.datetime(2020, 1, 1))].reset_index(drop=True)

In [964]:
df_hot.shape

(109600, 10)

In [965]:
df_hot

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1999-0...,1999-03-27,11,All Night Long,Faith Evans Featuring Puff Daddy,All Night LongFaith Evans Featuring Puff Daddy,1.0,19.0,11.0,8.0
1,https://www.billboard.com/charts/hot-100/2019-...,2019-12-07,11,Dance Monkey,Tones And I,Dance MonkeyTones And I,1.0,19.0,11.0,8.0
2,http://www.billboard.com/charts/hot-100/2005-1...,2005-10-15,11,Run It!,Chris Brown,Run It!Chris Brown,1.0,24.0,11.0,8.0
3,http://www.billboard.com/charts/hot-100/2010-0...,2010-03-27,12,Baby,Justin Bieber Featuring Ludacris,BabyJustin Bieber Featuring Ludacris,1.0,14.0,5.0,8.0
4,https://www.billboard.com/charts/hot-100/2018-...,2018-04-07,12,Pray For Me,The Weeknd & Kendrick Lamar,Pray For MeThe Weeknd & Kendrick Lamar,1.0,10.0,7.0,8.0
...,...,...,...,...,...,...,...,...,...,...
109595,https://www.billboard.com/charts/hot-100/2018-...,2018-03-17,38,You Make It Easy,Jason Aldean,You Make It EasyJason Aldean,1.0,38.0,28.0,6.0
109596,https://www.billboard.com/charts/hot-100/2018-...,2018-03-24,34,You Make It Easy,Jason Aldean,You Make It EasyJason Aldean,1.0,38.0,28.0,7.0
109597,https://www.billboard.com/charts/hot-100/2018-...,2018-03-31,36,You Make It Easy,Jason Aldean,You Make It EasyJason Aldean,1.0,34.0,28.0,8.0
109598,https://www.billboard.com/charts/hot-100/2018-...,2018-04-07,34,You Make It Easy,Jason Aldean,You Make It EasyJason Aldean,1.0,36.0,28.0,9.0


In [966]:
# Data Structure: Song Name, Artist, Position 1-100, Date of Billboard Chart Release/Update, Song's Position at t-1, Cumsum of Weeks, Appearence/Reappearence of Song, Artist-Song Concatentation
df_hot = df_hot.loc[:, ["Song", "Performer", "Week Position", "WeekID","Previous Week Position", "Peak Position", "Weeks on Chart", "Instance", "SongID"]]

In [967]:
df_hot = df_hot.assign(Week_No = lambda x: x.WeekID.apply(dt.datetime.strftime, format="%U"))

In [968]:
df_hot = df_hot.sort_values(["WeekID", "Week Position"]).reset_index(drop=True)

In [969]:
df_hot.Week_No.value_counts()

04    2100
22    2100
29    2100
36    2100
30    2100
33    2100
40    2100
25    2100
15    2100
47    2100
12    2100
27    2100
43    2100
39    2100
32    2100
07    2100
49    2100
17    2100
44    2100
31    2100
42    2100
51    2100
16    2100
48    2100
45    2100
35    2100
34    2100
46    2100
26    2100
01    2100
02    2100
28    2100
03    2100
11    2100
13    2100
20    2100
24    2100
23    2100
09    2100
21    2100
50    2100
14    2100
18    2100
08    2100
06    2100
19    2100
41    2100
10    2100
05    2100
38    2100
37    2100
00    1800
52     700
Name: Week_No, dtype: int64

In [970]:
df_hot.head(10)

,Song,Performer,Week Position,WeekID,Previous Week Position,Peak Position,Weeks on Chart,Instance,SongID,Week_No
0,I'm Your Angel,R. Kelly & Celine Dion,1,1999-01-02,1.0,1.0,5.0,1.0,I'm Your AngelR. Kelly & Celine Dion,00
1,Nobody's Supposed To Be Here,Deborah Cox,2,1999-01-02,2.0,2.0,14.0,1.0,Nobody's Supposed To Be HereDeborah Cox,00
2,Lately,Divine,3,1999-01-02,3.0,1.0,17.0,1.0,LatelyDivine,00
3,...Baby One More Time,Britney Spears,4,1999-01-02,5.0,4.0,7.0,1.0,...Baby One More TimeBritney Spears,00
4,Because Of You,98 Degrees,5,1999-01-02,6.0,3.0,14.0,1.0,Because Of You98 Degrees,00
5,From This Moment On,Shania Twain,6,1999-01-02,4.0,4.0,5.0,1.0,From This Moment OnShania Twain,00
6,Doo Wop (That Thing),Lauryn Hill,7,1999-01-02,7.0,1.0,8.0,1.0,Doo Wop (That Thing)Lauryn Hill,00
7,Trippin',Total Featuring Missy Elliott,8,1999-01-02,12.0,8.0,9.0,1.0,Trippin'Total Featuring Missy Elliott,00
8,Have You Ever?,Brandy,9,1999-01-02,9.0,9.0,5.0,1.0,Have You Ever?Brandy,00
9,Love Like This,Faith Evans,10,1999-01-02,8.0,7.0,8.0,1.0,Love Like ThisFaith Evans,00


In [971]:
# Number of Unique Songs
df_hot.SongID.unique().shape

(8307,)

In [972]:
# We have Equla Numbers for All Billboard Positions and NULLS Only on Song's First Week on Chart
df_hot.groupby("Week Position").count().sort_values("WeekID")

,Song,Performer,WeekID,Previous Week Position,Peak Position,Weeks on Chart,Instance,SongID,Week_No
Week Position,,,,,,,,,
1,1096,1096,1096,1071,1096,1096,1096,1096,1096
73,1096,1096,1096,954,1096,1096,1096,1096,1096
72,1096,1096,1096,956,1096,1096,1096,1096,1096
71,1096,1096,1096,971,1096,1096,1096,1096,1096
70,1096,1096,1096,975,1096,1096,1096,1096,1096
...,...,...,...,...,...,...,...,...,...
29,1096,1096,1096,1063,1096,1096,1096,1096,1096
28,1096,1096,1096,1058,1096,1096,1096,1096,1096
27,1096,1096,1096,1071,1096,1096,1096,1096,1096


## Merging With Attributes

In [973]:
# Spotify Track Data
attr_df = pd.read_csv("../data/raw/audioFeatures.csv")

In [974]:
attr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28492 entries, 0 to 28491
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SongID                     28492 non-null  object 
 1   Performer                  28492 non-null  object 
 2   Song                       28492 non-null  object 
 3   spotify_genre              27260 non-null  object 
 4   spotify_track_id           23743 non-null  object 
 5   spotify_track_preview_url  14164 non-null  object 
 6   spotify_track_album        23737 non-null  object 
 7   spotify_track_explicit     23743 non-null  object 
 8   spotify_track_duration_ms  23743 non-null  float64
 9   spotify_track_popularity   23743 non-null  float64
 10  danceability               23680 non-null  float64
 11  energy                     23680 non-null  float64
 12  key                        23680 non-null  float64
 13  loudness                   23680 non-null  flo

In [975]:
attr_df.columns

Index(['SongID', 'Performer', 'Song', 'spotify_genre', 'spotify_track_id',
       'spotify_track_preview_url', 'spotify_track_album',
       'spotify_track_explicit', 'spotify_track_duration_ms',
       'spotify_track_popularity', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'],
      dtype='object')

In [976]:
attr_df.drop(["spotify_track_preview_url"], axis=1, inplace=True)

In [977]:
for text_col in ["Song", "Performer", "SongID"]:
    df_hot[text_col] = df_hot[text_col].apply(str.lower)

In [978]:
for text_col in ["Song", "Performer", "SongID"]:
    attr_df[text_col] = attr_df[text_col].apply(str.lower)

In [979]:
full_df = pd.merge(df_hot, attr_df, "left", on="SongID")

In [980]:
missing_songs = full_df[full_df.liveness.isnull()].SongID.unique()

In [981]:
missing_songs.shape

(632,)

In [982]:
missing_songs

array(['latelydivine', 'because of you98 degrees',
       "(god must have spent) a little more time on you'n sync",
       'one weekbarenaked ladies', 'hard knock life (ghetto anthem)jay-z',
       'the power of good-byemadonna', 'angel in disguisebrandy',
       'ex-factorlauryn hill',
       'if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez',
       "it ain't my fault 1 & 2silkk the shocker featuring mystikal",
       'girlfriend/boyfriendblackstreet with janet',
       "everybody's free (to wear sunscreen)baz luhrmann",
       'the hardest thing98 degrees', 'one more trydivine',
       "can't get enoughpatty loveless", "i drive myself crazy'n sync",
       'little good-byesshedaisy',
       'she\'s a bitchmissy "misdemeanor" elliott',
       'hello l.o.v.e.john michael montgomery', 'jigga my n****jay-z',
       "let me knowcam'ron", 'what ya wanteve & nokio',
       'all n my grillmissy "misdemeanor" elliott featuring big boi & nicole',
       'i do (cherish

### Additional Data Sources

In [983]:
billboard_add = pd.read_csv("../data/raw/billboardHot100_1999-2019.csv", index_col=0)

In [984]:
billboard_add.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
1,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus
2,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN
3,Billie Eilish,Bad Guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","Billie eilish, Finneas","bad guy \nWhite shirt now red, my bloody nose\...",NaN
4,Khalid,Talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop","Howard lawrence, Guy lawrence, Khalid",Talk \nCan we just talk? Can we just talk?\nTa...,NaN
5,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop","Ed sheeran, Justin bieber, Shellback, Max mart...",I Don't Care \nI'm at a party I don't wanna be...,NaN


In [985]:
billboard_add.Week = pd.to_datetime(billboard_add.Week)

In [986]:
billboard_add.groupby("Week").count().sort_values("Genre")

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Date,Genre,Writing.Credits,Lyrics,Features
Week,,,,,,,,,,
2008-06-05,80,80,80,75,75,71,80,69,80,15
2007-09-05,83,83,83,81,81,70,83,72,83,14
2007-08-29,83,83,83,80,80,69,83,71,83,13
2007-06-06,83,83,83,76,76,73,83,75,83,14
2002-02-12,85,85,85,79,79,69,85,70,85,13
...,...,...,...,...,...,...,...,...,...,...
2019-03-23,100,100,100,90,90,100,100,100,100,16
2017-04-15,100,100,100,96,96,100,100,100,100,29
2017-07-29,100,100,100,84,84,100,100,100,100,31


In [987]:
# Additional Feature Data
features_add = pd.read_csv("../data/raw/songAttributes_1999-2019.csv", index_col=0)

In [988]:
features_add

,acousticness,album,artist,danceability,duration,energy,explicit,instrumentalness,liveness,loudness,mode,name,popularity,speechiness,tempo,timesignature,valence
0,0.000728,Collective Soul (Deluxe Version),Collective Soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,Welcome All Again,35,0.0309,106.022,4,0.365
1,0.018200,Collective Soul (Deluxe Version),Collective Soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,Fuzzy,31,0.0282,120.027,4,0.408
2,0.000473,Collective Soul (Deluxe Version),Collective Soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,Dig,30,0.0559,144.061,4,0.370
3,0.000970,Collective Soul (Deluxe Version),Collective Soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,You,35,0.0254,111.975,4,0.183
4,0.000036,Collective Soul (Deluxe Version),Collective Soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,My Days,21,0.0318,92.721,4,0.666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0.333000,No.6 Collaborations Project,Ed Sheeran,0.747,200733,0.649,False,0.000000,0.1720,-6.218,0,Nothing On You (feat. Paulo Londra & Dave),80,0.2190,88.011,4,0.840
115,0.314000,No.6 Collaborations Project,Ed Sheeran,0.469,204507,0.631,False,0.000000,0.1400,-6.569,0,I Don't Want Your Money (feat. H.E.R.),74,0.0853,91.418,4,0.763
116,0.467000,No.6 Collaborations Project,Ed Sheeran,0.778,212307,0.590,False,0.000000,0.0902,-5.548,1,1000 Nights (feat. Meek Mill & A Boogie Wit da...,75,0.2700,97.995,4,0.515
117,0.110000,No.6 Collaborations Project,Ed Sheeran,0.726,190293,0.731,False,0.000003,0.0928,-6.304,0,Way To Break My Heart (feat. Skrillex),73,0.1590,170.125,4,0.430


In [989]:
for text_col in ["name", "artist"]:
    features_add[text_col] = features_add[text_col].apply(str.lower)

In [990]:
features_add = features_add.assign(SongID = lambda x: x.name + x.artist)

In [991]:
features_add = features_add.drop_duplicates(subset="SongID")

In [992]:
songs_list = list(missing_songs)
songs_list.sort()

In [993]:
songs_list[:20]

["'03 bonnie & clydejay-z featuring beyonce knowles",
 "(god must have spent) a little more time on you'n sync",
 '(hot s**t) country grammarnelly',
 '...ready for it?taylor swift',
 '1-800-273-8255logic featuring alessia cara & khalid',
 '1.521 savage',
 '10,000 hoursdan + shay & justin bieber',
 '15 minutesmarc nelson',
 '24/7kevon edmonds',
 '24/7meek mill featuring ella mai',
 '24k magicbruno mars',
 '3500travi$ scott featuring future & 2 chainz',
 '3ameminem',
 '3britney spears',
 '4 am2 chainz featuring travis scott',
 '4 ammelanie fiona',
 '4:44jay-z',
 '5-1-5-0dierks bentley',
 '7/11beyonce',
 '9 am in dallasdrake']

In [994]:
pattern = r"|".join(songs_list)

In [995]:
#pattern = pattern.replace("*", "")

In [996]:
songs_list[110:115]

['car washchristina aguilera featuring missy elliott',
 'careless whisperwham! featuring george michael',
 'caught their eyesjay-z featuring frank ocean',
 'ch-check it outbeastie boys',
 'cha-cha slidemr. c the slide man']

In [997]:
missing_songs_df = full_df[full_df.liveness.isnull()]

In [998]:
charters_with_avaiable_attributes = missing_songs_df.iloc[:, :10].merge(features_add, on="SongID")

In [999]:
charters_with_avaiable_attributes.SongID.unique().shape

(58,)

In [1000]:
attr_df.columns

Index(['SongID', 'Performer', 'Song', 'spotify_genre', 'spotify_track_id',
       'spotify_track_album', 'spotify_track_explicit',
       'spotify_track_duration_ms', 'spotify_track_popularity', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [1001]:
features_add.columns

Index(['acousticness', 'album', 'artist', 'danceability', 'duration', 'energy',
       'explicit', 'instrumentalness', 'liveness', 'loudness', 'mode', 'name',
       'popularity', 'speechiness', 'tempo', 'timesignature', 'valence',
       'SongID'],
      dtype='object')

In [1002]:
full_df.columns

Index(['Song_x', 'Performer_x', 'Week Position', 'WeekID',
       'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
       'SongID', 'Week_No', 'Performer_y', 'Song_y', 'spotify_genre',
       'spotify_track_id', 'spotify_track_album', 'spotify_track_explicit',
       'spotify_track_duration_ms', 'spotify_track_popularity', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [1003]:
attr_df = attr_df.rename(columns={"spotify_track_album": "album", 'spotify_genre': "genre", 'spotify_track_id': "id",
'spotify_track_preview_url': "url", "spotify_track_explicit": "explicit",
'spotify_track_duration_ms': "duration",'spotify_track_popularity': "popularity",
'time_signature': "timesignature"})

In [1005]:
 full_df.drop(["Song_y", "Performer_y"], axis=1, inplace=True)

In [1006]:
full_df = full_df.rename(columns={"spotify_track_album": "album", 'spotify_genre': "genre", 'spotify_track_id': "id",
'spotify_track_preview_url': "url", "spotify_track_explicit": "explicit",
'spotify_track_duration_ms': "duration",'spotify_track_popularity': "popularity",
'time_signature': "timesignature", "Song_x": "Song","Performer_x": "Performer"})

In [1007]:
full_df.columns

Index(['Song', 'Performer', 'Week Position', 'WeekID',
       'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
       'SongID', 'Week_No', 'genre', 'id', 'album', 'explicit', 'duration',
       'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'timesignature'],
      dtype='object')

In [1008]:
features_add = features_add.rename(columns={"name": "Song"})

In [1009]:
charters_with_avaiable_attributes = missing_songs_df.iloc[:, :10].merge(features_add, on="SongID")

In [1010]:
features_add

,acousticness,album,artist,danceability,duration,energy,explicit,instrumentalness,liveness,loudness,mode,Song,popularity,speechiness,tempo,timesignature,valence,SongID
0,0.000728,Collective Soul (Deluxe Version),collective soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,welcome all again,35,0.0309,106.022,4,0.365,welcome all againcollective soul
1,0.018200,Collective Soul (Deluxe Version),collective soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,fuzzy,31,0.0282,120.027,4,0.408,fuzzycollective soul
2,0.000473,Collective Soul (Deluxe Version),collective soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,dig,30,0.0559,144.061,4,0.370,digcollective soul
3,0.000970,Collective Soul (Deluxe Version),collective soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,you,35,0.0254,111.975,4,0.183,youcollective soul
4,0.000036,Collective Soul (Deluxe Version),collective soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,my days,21,0.0318,92.721,4,0.666,my dayscollective soul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0.333000,No.6 Collaborations Project,ed sheeran,0.747,200733,0.649,False,0.000000,0.1720,-6.218,0,nothing on you (feat. paulo londra & dave),80,0.2190,88.011,4,0.840,nothing on you (feat. paulo londra & dave)ed s...
115,0.314000,No.6 Collaborations Project,ed sheeran,0.469,204507,0.631,False,0.000000,0.1400,-6.569,0,i don't want your money (feat. h.e.r.),74,0.0853,91.418,4,0.763,i don't want your money (feat. h.e.r.)ed sheeran
116,0.467000,No.6 Collaborations Project,ed sheeran,0.778,212307,0.590,False,0.000000,0.0902,-5.548,1,1000 nights (feat. meek mill & a boogie wit da...,75,0.2700,97.995,4,0.515,1000 nights (feat. meek mill & a boogie wit da...
117,0.110000,No.6 Collaborations Project,ed sheeran,0.726,190293,0.731,False,0.000003,0.0928,-6.304,0,way to break my heart (feat. skrillex),73,0.1590,170.125,4,0.430,way to break my heart (feat. skrillex)ed sheeran


In [1011]:
billboard_add.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
1,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus
2,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN
3,Billie Eilish,Bad Guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","Billie eilish, Finneas","bad guy \nWhite shirt now red, my bloody nose\...",NaN
4,Khalid,Talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop","Howard lawrence, Guy lawrence, Khalid",Talk \nCan we just talk? Can we just talk?\nTa...,NaN
5,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop","Ed sheeran, Justin bieber, Shellback, Max mart...",I Don't Care \nI'm at a party I don't wanna be...,NaN


In [1012]:
features_add.head()

,acousticness,album,artist,danceability,duration,energy,explicit,instrumentalness,liveness,loudness,mode,Song,popularity,speechiness,tempo,timesignature,valence,SongID
0,0.000728,Collective Soul (Deluxe Version),collective soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,welcome all again,35,0.0309,106.022,4,0.365,welcome all againcollective soul
1,0.018200,Collective Soul (Deluxe Version),collective soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,fuzzy,31,0.0282,120.027,4,0.408,fuzzycollective soul
2,0.000473,Collective Soul (Deluxe Version),collective soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,dig,30,0.0559,144.061,4,0.370,digcollective soul
3,0.000970,Collective Soul (Deluxe Version),collective soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,you,35,0.0254,111.975,4,0.183,youcollective soul
4,0.000036,Collective Soul (Deluxe Version),collective soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,my days,21,0.0318,92.721,4,0.666,my dayscollective soul


In [1013]:
billboard_add = billboard_add.assign(SongID = lambda x: x.Artists.apply(str.lower).apply(str.replace, args=(",","")) + x.Name.apply(str.lower).apply(str.replace, args=(",","")))

In [1014]:
(set(full_df.columns)) - set(charters_with_avaiable_attributes.columns) 

{'Performer', 'genre', 'id', 'key'}

In [1015]:
full_df[full_df.genre == "[]"].SongID.unique().shape

(240,)

In [1016]:
# I have 632 Songs that Do Not Have Spotify Attributes
# I have 240 Songs that Do Not Have Spotify Genres

# So I Need to Search Spotify's API for these Songs to get 1. Missing Genres and 2. Complete Attribute Information for the Other Missing Songs

In [1017]:
missing_genres_songs = set(full_df[full_df.genre == "[]"].SongID.unique())
missing_attribute_songs = set(full_df[full_df.liveness.isnull()].SongID.unique())
needed_songs = missing_attribute_songs.union(missing_genres_songs)

In [1018]:
pattern = "|".join(needed_songs)

In [1019]:
pattern = pattern.replace("*", "")

In [1020]:
full_df.SongID = full_df.SongID.apply(str.replace, args=("*", ""))

In [1021]:
need_songs_df = full_df[full_df.SongID.str.contains(pattern)]
complete_info_df = full_df[~(full_df.SongID.str.contains(pattern))]

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [1022]:
need_songs_df.SongID.unique().shape

(655,)

In [67]:
need_songs_df.drop_duplicates("SongID").loc[:, ["Song", "id", "Performer", "SongID"]]

,Song,id,Performer,SongID
0,i'm your angel,0QWFiyhOqFu9RP8qzP6z5L,r. kelly & celine dion,i'm your angelr. kelly & celine dion
2,lately,NaN,divine,latelydivine
4,because of you,NaN,98 degrees,because of you98 degrees
20,ghetto cowboy,6xihjuCX132pyKm5dNFe3T,mo thugs family featuring bone thugs-n-harmony,ghetto cowboymo thugs family featuring bone th...
36,one week,NaN,barenaked ladies,one weekbarenaked ladies
...,...,...,...,...
110152,saint-tropez,NaN,post malone,saint-tropezpost malone
110505,stuck in a dream,NaN,lil mosey x gunna,stuck in a dreamlil mosey x gunna
110733,drunk dialing...lodt,NaN,summer walker,drunk dialing...lodtsummer walker
110748,leave em alone,NaN,"layton greene, lil baby, city girls & pnb rock","leave em alonelayton greene, lil baby, city gi..."


In [68]:
# Call Spotify API for Track Information
    # Check If Songs Has an track id

        # If Not Will have to call the same API to Use Query to Find Track

    # Use Id to Get Audio Feature :: /audio-features endpoint, Genre Information, Popularity are received through the /tracks endpoint






In [69]:
master_song_list = pd.read_csv("../data/raw/archive/data.csv")

In [70]:
master_song_list.loc[1, "artists"]

'["Screamin\' Jay Hawkins"]'

In [71]:
def cleanNames(name):
    regex_names = re.compile(r"[\[\]\'\'\"\"]+")
    groups = regex_names.findall(name)
    for g in groups:
        name = name.replace(g, "")
    return name.lower()

In [72]:
master_song_list.name = master_song_list.name.apply(str.lower)

In [73]:
master_song_list.artists = master_song_list.artists.apply(cleanNames)

In [74]:
need_songs_df.drop_duplicates("SongID").loc[:, ["Song", "id", "Performer", "SongID"]][lambda x: x.id.isnull()].merge(master_song_list.loc[:, ["name", "artists" ,"id"]], left_on=["Song", "Performer"], right_on=["name", "artists"])

,Song,id_x,Performer,SongID,name,artists,id_y
0,one week,NaN,barenaked ladies,one weekbarenaked ladies,one week,barenaked ladies,29hBRadFZf9QTGRHZmxm65
1,the power of good-bye,NaN,madonna,the power of good-byemadonna,the power of good-bye,madonna,01VFDkHBNJcCNUjzD3flWg
2,angel in disguise,NaN,brandy,angel in disguisebrandy,angel in disguise,brandy,6PQTUESsPMNcDAg2LEmxh7
3,little good-byes,NaN,shedaisy,little good-byesshedaisy,little good-byes,shedaisy,5ZX3IrB9tPxOoBZdXILNpy
4,24/7,NaN,kevon edmonds,24/7kevon edmonds,24/7,kevon edmonds,4rZB2G955dQMcjlb7e3VNB
...,...,...,...,...,...,...,...
74,rudolph the red-nosed reindeer,NaN,gene autry,rudolph the red-nosed reindeergene autry,rudolph the red-nosed reindeer,gene autry,29dzQKn6AUNUCu1dE4KfqZ
75,rudolph the red-nosed reindeer,NaN,gene autry,rudolph the red-nosed reindeergene autry,rudolph the red-nosed reindeer,gene autry,1FERlwAufBFVhYLyF8nB9S
76,rudolph the red-nosed reindeer,NaN,gene autry,rudolph the red-nosed reindeergene autry,rudolph the red-nosed reindeer,gene autry,1BUtlqWX1INerbv3s99Uda
77,f.n,NaN,lil tjay,f.nlil tjay,f.n,lil tjay,0RLLvYBopDwooCyvnTwgYu


In [75]:
available_ids = need_songs_df.drop_duplicates("SongID").loc[:, ["Song", "id", "Performer", "SongID"]][lambda x: x.id.isnull()].merge(master_song_list.loc[:, ["name", "artists" ,"id"]], left_on=["Song", "Performer"], right_on=["name", "artists"]).SongID.unique()

In [76]:
available_ids = set(available_ids)

In [77]:
need_songs_df[~(need_songs_df.SongID.isin(available_ids))].drop_duplicates(["Song", "Performer"]).loc[:, ["Song", "Performer", "SongID"]]

,Song,Performer,SongID
0,i'm your angel,r. kelly & celine dion,i'm your angelr. kelly & celine dion
2,lately,divine,latelydivine
4,because of you,98 degrees,because of you98 degrees
20,ghetto cowboy,mo thugs family featuring bone thugs-n-harmony,ghetto cowboymo thugs family featuring bone th...
69,when a woman's fed up,r. kelly,when a woman's fed upr. kelly
...,...,...,...
109975,it's nice to have a friend,taylor swift,it's nice to have a friendtaylor swift
110505,stuck in a dream,lil mosey x gunna,stuck in a dreamlil mosey x gunna
110733,drunk dialing...lodt,summer walker,drunk dialing...lodtsummer walker
110748,leave em alone,"layton greene, lil baby, city girls & pnb rock","leave em alonelayton greene, lil baby, city gi..."


In [861]:
response = requests.post(r'https://accounts.spotify.com/api/token', {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials",
})

In [862]:
# Recieved Token @ 12:05 PM
response = response.json()

In [863]:
token = response["access_token"]

In [78]:
songs_to_search = need_songs_df[~(need_songs_df.SongID.isin(available_ids))].drop_duplicates(["Song", "Performer"]).loc[:, ["Song", "Performer", "SongID", "WeekID"]]

In [79]:
songs_to_search = songs_to_search.assign(year = lambda x: x.WeekID.apply(lambda x: x.year))

In [256]:
song_regex = re.compile(r"&+|feat\.|featuring|\s+and|,+|\sx\s+|with")

In [700]:
headers = {"Authorization": f"Bearer {token}"}
new_ids = pd.DataFrame(columns=["SongID", "Song", "Performer", "popularity", "explicit", "id"])
i = 0

In [289]:
for s in songs_to_search.values:
    artist = ""
    if song_regex.findall(s[1]) != []:
        artist = s[1].split(song_regex.findall(s[1])[0])[0].replace("'", "").strip()
    else:
        artist = s[1].replace("'", "").strip()
    song = s[0].replace("'", "").strip()
    _url = f"https://api.spotify.com/v1/search?q=artist:{artist} track:{song}&type=artist,track&market=ES&limit=10&offset=0"
    _response_s = requests.get(_url, headers=headers)
    _response_s = _response_s.json()
    try: 
        if _response_s["tracks"]["items"] != []:
            for _item in _response_s["tracks"]["items"]:
                new_ids.loc[i, "Performer"] = _item["artists"][0]["name"]
                new_ids.loc[i, "popularity"] = _item["popularity"]
                new_ids.loc[i, "explicit"] = _item["explicit"]
                new_ids.loc[i, "id"] = _item["id"]
                new_ids.loc[i, "Song"] = _item["name"]
                new_ids.loc[i, "SongID"] = s[2]
                i += 1
        else:
            new_ids.loc[i, "SongID"] = s[2]
            print(f"issue with {s[2]}")
            i += 1
    except KeyError:
        new_ids.loc[i, "SongID"] = s[2]
        print(f"issue with {s[2]}")
        i += 1
    print(f"finished with {s[2]}")
    time.sleep(2)

finished with i'm your angelr. kelly & celine dion
finished with latelydivine
finished with because of you98 degrees
finished with ghetto cowboymo thugs family featuring bone thugs-n-harmony
finished with when a woman's fed upr. kelly
finished with home aloner. kelly featuring keith murray
finished with can't get enoughwillie max featuring raphael saadiq
finished with talk show shhh!shae jones
finished with ex-factorlauryn hill
finished with big big worldemilia
finished with watch for the hookcool breeze featuring outkast, goodie mob & witchdoctor
finished with stay the samejoey mcintyre
issue with if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez
finished with if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez
issue with it ain't my fault 1 & 2silkk the shocker featuring mystikal
finished with it ain't my fault 1 & 2silkk the shocker featuring mystikal
issue with girlfriend/boyfriendblackstreet with janet
finished with girlfriend/boyfrien

In [290]:
new_ids.shape

(1741, 6)

In [292]:
new_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1741 entries, 0 to 1740
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SongID      1741 non-null   object
 1   Song        1475 non-null   object
 2   Performer   1475 non-null   object
 3   popularity  1475 non-null   object
 4   explicit    1475 non-null   object
 5   id          1475 non-null   object
dtypes: object(6)
memory usage: 175.2+ KB


In [293]:
new_ids.to_csv("../data/interim/api_scrapes.csv", index=False)

In [82]:
new_ids = pd.read_csv("../data/interim/api_scrapes.csv")

In [85]:
# Obtained 329 Song IDs from API of the 595 Missing; This Missing 266 Need Further Cleaning
new_ids.drop_duplicates("SongID", keep="first").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 595 entries, 0 to 1740
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SongID      595 non-null    object 
 1   Song        329 non-null    object 
 2   Performer   329 non-null    object 
 3   popularity  329 non-null    float64
 4   explicit    329 non-null    object 
 5   id          329 non-null    object 
dtypes: float64(1), object(5)
memory usage: 32.5+ KB


In [91]:
bad_search_map = {}
bad_songs = new_ids.drop_duplicates("SongID")[lambda x: x.id.isnull()].SongID.values

In [120]:
list(bad_search_map.values())[0]

'if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez'

In [95]:
for s in bad_songs:
    bad_search_map[s] = s

In [109]:
for l in song_regex.findall(list(bad_search_map.values())[1]):
    vc = list(bad_search_map.values())[1].split(l)[0]
    print(vc)

it ain't my fault 1 
it ain't my fault 1 & 2silkk the shocker 


In [165]:
punc_regex = r"[.,!?\"\'\"\'&0-9\.:\\\/\*]+"

In [124]:
re.sub(punc_regex, "",'if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez')

'if you could read my mindstars on  ultra nate amber jocelyn enriquez'

In [131]:
faulty_ids_df = new_ids.drop_duplicates("SongID")[lambda x: x.id.isnull()].dropna(axis=1).merge(songs_to_search, on="SongID")

In [257]:
faulty_ids_df = faulty_ids_df.assign(Song2 = lambda x: x.Song.apply(lambda x: re.sub(punc_regex, "", x).strip() if "n't" or "'s" or "'ll" in x else re.sub(punc_regex, " ", x).strip()))

In [258]:
faulty_ids_df = faulty_ids_df.assign(Performer2 = lambda x: x.Performer.apply(lambda x: re.sub(punc_regex, "", re.split(song_regex, x)[0]).strip() if song_regex.findall(x) != [] else  re.sub(punc_regex, "", x).strip()))

In [377]:
faulty_dict = faulty_ids_df.loc[:, ["Song2", "Performer2"]].to_dict("index")

In [378]:
faulty_dict[0]["Performer2"] = "stars on 54"
faulty_dict[0]["Song2"] = "if you could read my mind"
faulty_dict[2]["Song2"] = "girlfriend boyfriend"
faulty_dict[5]["Performer2"] = "missy elliott"
faulty_dict[7]["Song2"] = "jigga my"
faulty_dict[9]["Performer2"] = "missy elliott"
faulty_dict[11]["Performer2"] = "garth brooks"
faulty_dict[14]["Performer2"] = "missy elliott"
faulty_dict[15]["Performer2"] = "the eastsidaz"
faulty_dict[19]["Performer2"] = "the eastsidaz"
faulty_dict[21]["Performer2"] = "bow wow"
faulty_dict[24]["Performer2"] = "shade sheist"
faulty_dict[26]["Song2"] = "project bitch"
faulty_dict[26]["Performer2"] = "big tymers"
faulty_dict[27]["Performer2"] = "mr c"
faulty_dict[27]["Song2"] = "cha cha slide"
faulty_dict[28]["Performer2"] = "ricky martin"
faulty_dict[29]["Performer2"] = "aaron lewis"
faulty_dict[34]["Performer2"] = "missy elliott"
faulty_dict[36]["Performer2"] = "bow wow"
faulty_dict[37]["Performer2"] = "diddy"
faulty_dict[40]["Performer2"] = "missy elliott"
faulty_dict[41]["Performer2"] = "diddy"
faulty_dict[42]["Performer2"] = "bow wow"
faulty_dict[46]["Performer2"] = "missy elliott"
faulty_dict[48]["Song2"] = "never too far"
faulty_dict[49]["Performer2"] = "bow wow"
faulty_dict[50]["Song2"] = "the knoc"
faulty_dict[50]["Performer2"] = "knocturnal"
faulty_dict[51]["Song2"] = "take you home with me"
faulty_dict[52]["Performer2"] = "cee lo"
faulty_dict[53]["Performer2"] = "bow wow"
faulty_dict[56]["Song2"] = "sugar high"
faulty_dict[56]["Performer2"] = "jade anderson"
faulty_dict[57]["Song2"] = "move bitch"
faulty_dict[58]["Performer2"] = "ja rule"
faulty_dict[61]["Performer2"] = "missy elliott"
faulty_dict[64]["Performer2"] = "missy elliott"
faulty_dict[65]["Performer2"] = "american idol"
faulty_dict[66]["Performer2"] = "toby keith"
faulty_dict[67]["Song2"] = "pussycat"
faulty_dict[67]["Performer2"] = "missy elliott"
faulty_dict[69]["Song2"] = "love @ 1st sight"
faulty_dict[70]["Song2"] = "cop that shit"
faulty_dict[71]["Performer2"] = "toby keith"
faulty_dict[74]["Performer2"] = "2pac"
faulty_dict[75]["Performer2"] = "lost prophets"
faulty_dict[78]["Song2"] = "shake that shit"
faulty_dict[82]["Performer2"] = "boosie badazz"
faulty_dict[84]["Song2"] = "numb / encore"
faulty_dict[84]["Performer2"] = "linkin park"
faulty_dict[86]["Song2"] = "cry baby / piece of my heart"
faulty_dict[87]["Song2"] = "bitches aint shit"
faulty_dict[90]["Performer2"] = "jeezy"
faulty_dict[91]["Performer2"] = "jeezy"
faulty_dict[93]["Performer2"] = "jeezy"
faulty_dict[94]["Performer2"] = "troy"
faulty_dict[95]["Performer2"] = "troy"
faulty_dict[96]["Performer2"] = "troy"
faulty_dict[97]["Performer2"] = "sharpay evans"
faulty_dict[98]["Performer2"] = "sharpay evans"
faulty_dict[99]["Performer2"] = "vanessa hudgens"
faulty_dict[102]["Performer2"] = "daughtry"
faulty_dict[103]["Performer2"] = "jeezy"
faulty_dict[104]["Performer2"] = "boosie badazz"
faulty_dict[105]["Performer2"] = "jeezy"
faulty_dict[107]["Performer2"] = "the chicks"
faulty_dict[110]["Performer2"] = "boosie badazz"
faulty_dict[112]["Performer2"] = "r kelly"
faulty_dict[114]["Performer2"] = "bon jovi"
faulty_dict[118]["Performer2"] = "the zombies"
faulty_dict[119]["Performer2"] = "reba mcentire"
faulty_dict[120]["Performer2"] = "justin timberlake"
faulty_dict[122]["Performer2"] = "troy"
faulty_dict[123]["Performer2"] = "troy"
faulty_dict[124]["Performer2"] = "ryan"
faulty_dict[125]["Performer2"] = "troy"
faulty_dict[126]["Performer2"] = "sharpay evans"
faulty_dict[127]["Performer2"] = "keyshia cole"
faulty_dict[128]["Performer2"] = "c-side"
faulty_dict[130]["Performer2"] = "hillsong worship"
faulty_dict[132]["Song2"] = "love in this club pt ii"
faulty_dict[135]["Performer2"] = "u2"
faulty_dict[136]["Performer2"] = "collective soul"
faulty_dict[137]["Performer2"] = "john lennon"
faulty_dict[138]["Performer2"] = "aerosmith"
faulty_dict[139]["Performer2"] = "michael jackson"
faulty_dict[140]["Performer2"] = "george michael"
faulty_dict[142]["Performer2"] = "mariah carey"
faulty_dict[143]["Performer2"] = "lionel richie"
faulty_dict[144]["Performer2"] = "andrew lloyd webber"
faulty_dict[145]["Performer2"] = "aretha franklin"
faulty_dict[146]["Performer2"] = "celine dion"
faulty_dict[147]["Performer2"] = "yung joc"
faulty_dict[149]["Song2"] = "somethin special"
faulty_dict[150]["Performer2"] = "brad paisley"
faulty_dict[151]["Performer2"] ="vanessa hudgens"
faulty_dict[152]["Performer2"] ="jeezy"
faulty_dict[154]["Song2"] ="jizz in my pants"
faulty_dict[155]["Song2"] ="fuck you"
faulty_dict[157]["Song2"] ="3 A.M."
faulty_dict[158]["Performer2"] ="bill withers"
faulty_dict[159]["Performer2"] ="sam cooke"
faulty_dict[160]["Performer2"] = "one republic"
faulty_dict[161]["Performer2"] = "steve kazee"
faulty_dict[162]["Performer2"] = "keyshia cole"
faulty_dict[164]["Performer2"] = "jmkaraoke"
faulty_dict[171]["Song2"] = "fuck today"
faulty_dict[173]["Performer2"] = "the beatles"
faulty_dict[174]["Performer2"] = "three 6 mafia"
faulty_dict[176]["Performer2"] = "b the star"
faulty_dict[178]["Performer2"] = "jeezy"
faulty_dict[179]["Performer2"] = "jeff buckley"
faulty_dict[180]["Performer2"] = "steve kazee"
faulty_dict[181]["Performer2"] = "simon and garfunkel"
faulty_dict[185]["Performer2"] = "rock mafia"
faulty_dict[186]["Performer2"] = "michael jackson"
faulty_dict[192]["Performer2"] = "jeezy"
faulty_dict[193]["Performer2"] = "brad paisley"
faulty_dict[195]["Performer2"] = "machine gun kelly"
faulty_dict[196]["Performer2"] = "jeezy"
faulty_dict[197]["Song2"] = "cold"
faulty_dict[198]["Performer2"] = "james brown"
faulty_dict[202]["Performer2"] = "jeezy"
faulty_dict[203]["Song2"] = "rich as fuck"
faulty_dict[204]["Performer2"] = "ti"
faulty_dict[207]["Performer2"] = "fred stobaugh"
faulty_dict[208]["Performer2"] = "work bitch"
faulty_dict[216]["Song2"] = "she came ii give it ii u"
faulty_dict[219]["Performer2"] = "lilly wood"
faulty_dict[222]["Performer2"] = "maroon 5"
faulty_dict[226]["Song2"] = "fly shit only"
faulty_dict[227]["Performer2"] = "saygrace"
faulty_dict[232]["Song2"] = "my shit"
faulty_dict[233]["Performer2"] = "george michael"
faulty_dict[236]["Performer2"] = "dove cameron"
faulty_dict[237]["Song2"] = "fuck that check up"
faulty_dict[238]["Song2"] = "fuck love"
faulty_dict[239]["Performer2"] = "migos"
faulty_dict[240]["Song2"] = "my choppa hate"
faulty_dict[240]["Performer2"] = "21 savage"
faulty_dict[242]["Performer2"] = "ben platt"
faulty_dict[243]["Performer2"] = "rae sremmurd"
faulty_dict[249]["Performer2"] = "iggy azalea"
faulty_dict[252]["Song2"] = "dope n"
faulty_dict[253]["Song2"] = "same yung n"
faulty_dict[257]["Song2"] = "how bout now"
faulty_dict[259]["Performer2"] = "diplo"
faulty_dict[263]["Song2"] = "drunk dialing lodt"
faulty_dict[264]["Performer2"] = "lil baby"

In [379]:
faulty_ids_df_2  =pd.merge(faulty_ids_df, pd.DataFrame.from_dict(faulty_dict, orient="index"), left_index=True, right_index=True)

In [380]:
second_api_try_array = faulty_ids_df_2.drop(["Song2_x", "Performer2_x"], axis=1).loc[:, ["SongID", "Song2_y", "Performer2_y"]].values

In [381]:
for s in second_api_try_array:
    song = s[1]
    _id = s[0]
    artist = s[2] 
    _url = f"https://api.spotify.com/v1/search?q=artist:{artist} track:{song}&type=artist,track&market=ES&limit=10&offset=0"
    _response_s = requests.get(_url, headers=headers)
    _response_s = _response_s.json()
    try: 
        if _response_s["tracks"]["items"] != []:
            for _item in _response_s["tracks"]["items"]:
                new_ids.loc[i, "Performer"] = _item["artists"][0]["name"]
                new_ids.loc[i, "popularity"] = _item["popularity"]
                new_ids.loc[i, "explicit"] = _item["explicit"]
                new_ids.loc[i, "id"] = _item["id"]
                new_ids.loc[i, "Song"] = _item["name"]
                new_ids.loc[i, "SongID"] = _id
                i += 1
        else:
            new_ids.loc[i, "SongID"] = _id
            print(f"issue with {_id}")
            i += 1
    except KeyError:
        new_ids.loc[i, "SongID"] = _id
        print(f"issue with {_id}")
        i += 1
    print(f"finished with {_id}")

issue with if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez
finished with if you could read my mindstars on 54: ultra nate, amber, jocelyn enriquez
finished with it ain't my fault 1 & 2silkk the shocker featuring mystikal
finished with girlfriend/boyfriendblackstreet with janet
issue with one more trydivine
finished with one more trydivine
issue with can't get enoughpatty loveless
finished with can't get enoughpatty loveless
finished with she's a bitchmissy "misdemeanor" elliott
issue with hello l.o.v.e.john michael montgomery
finished with hello l.o.v.e.john michael montgomery
issue with jigga my njay-z
finished with jigga my njay-z
issue with let me knowcam'ron
finished with let me knowcam'ron
finished with all n my grillmissy "misdemeanor" elliott featuring big boi & nicole
issue with southern gulerykah badu featuring rahzel
finished with southern gulerykah badu featuring rahzel
finished with lost in yougarth brooks as chris gaines
issue with satisfy youpuff

In [384]:
new_ids.drop_duplicates("SongID").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 864
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SongID      266 non-null    object
 1   Song        140 non-null    object
 2   Performer   140 non-null    object
 3   popularity  140 non-null    object
 4   explicit    140 non-null    object
 5   id          140 non-null    object
dtypes: object(6)
memory usage: 14.5+ KB


In [409]:
remaining_id_dictionary = new_ids[new_ids.id.isnull()].drop_duplicates().to_dict(orient="records")

In [410]:
remaining_id_dictionary[0]["id"] = "7lQXHsKgBOdXQ51GSNYIf5"

In [411]:
remaining_id_dictionary[1]["id"]="3lMaQLliTd1wlzc3lQitPI"

In [412]:
remaining_id_dictionary[2]["id"] = "1XDihfFjzdKt3bgFIZcPsm"

In [413]:
remaining_id_dictionary[3]["id"] = "1GoVQ3eadHrcjpJuAzAxjA"

In [415]:
remaining_id_dictionary[4]["id"] = "0Db1a7sL9MOjXgO5Z7zRM4"

In [417]:
remaining_id_dictionary[5]["id"] = "1LzXEMMiR7vJZJv25VgKSB"

In [419]:
remaining_id_dictionary[6]["id"] = "2pskAvfiKJ0ulfNLIG8eBC"

In [421]:
remaining_id_dictionary[7]["id"] = "00FROhC5g4iJdax5US8jRr"

In [423]:
remaining_id_dictionary[8]["id"] = "0LywQAlPnXKeMDh8NNSYrs"

In [425]:
remaining_id_dictionary[9]["id"]= "1zIXpzuCC7FM4kECt2eEzu"

In [428]:
remaining_id_dictionary[10]["id"] = "7Gfzoer3tsu6AVTce0TwSM"

In [430]:
remaining_id_dictionary[11]["id"] = "5WUwOfEjm6azpR1O8PKlb3"

In [432]:
remaining_id_dictionary[12]["id"] = "6prKHhjQS7vc1YKLI77cQJ"

In [434]:
remaining_id_dictionary[13]["id"] = "2ozJmnjqCMJavEKoHqiou0"

In [436]:
remaining_id_dictionary[14]["id"] = "3qB0dFgHhKH6cSMLB3tBqQ"

In [438]:
remaining_id_dictionary[15]["id"] = "0h9mUYPhZd2aQK2JmrHPD2"

In [440]:
remaining_id_dictionary[16]["id"] = "49QCYHks5U6wHjVtIzLdeK"

In [443]:
remaining_id_dictionary[17]["id"] = "4qBrgITlDYLJ6onVx1dHVp"

In [445]:
remaining_id_dictionary[18]["id"] = "0pXqZFHEasl7G4GTyy4Vg9"

In [448]:
remaining_id_dictionary[19]["id"] =  "3rgAkK0NWIP78QDi5SFSCz"

In [452]:
remaining_id_dictionary[20]["id"] = "49XSbzWYyB20DTlBtQiZFw"

In [453]:
remaining_id_dictionary[21]["id"] = "6P3xtpHYY87MGHjrQ36i6X"

In [455]:
remaining_id_dictionary[22]["id"] = "7gUk6jBAXio5xONTkoYDCq"

In [457]:
remaining_id_dictionary[23]["id"] = "3lZfgXw5PttrcmWSqs6fBk"

In [459]:
remaining_id_dictionary[24]["id"] = "6Tydun5H6T2F3ZpE6aMWk6"

In [461]:
remaining_id_dictionary[25]["id"] = "4gdQ8QShlC3toP9Mkc8DwB"

In [463]:
remaining_id_dictionary[26]["id"] = "1PdTuh5X9eUsRvxOzJLuWy"

In [465]:
remaining_id_dictionary[27]["id"] = "1Q9b6CeMcDuO0uq5OJCrqu"

In [467]:
remaining_id_dictionary[28]["id"] = "713DjChfL2RflBcl5cavQT"

In [469]:
remaining_id_dictionary[29]["id"] = "0Uhnzk5zI46IRlQ04LNOtc"

In [471]:
remaining_id_dictionary[30]["id"] = "5xErd5QhOvTKFU7l8dffWC"

In [473]:
remaining_id_dictionary[31]["id"] = "2dCD8qq7yHZPZduA8FJazs"

In [475]:
remaining_id_dictionary[32]["id"] = "623lYBOJ7bLwh1bu3yeN5V"

In [478]:
remaining_id_dictionary[34]["id"] = "14k9qyaFUPF7fm5o0fRwZD"

In [480]:
remaining_id_dictionary[35]["id"] = "6uzesxG4dlBg3faZmzL6Qs"

In [482]:
remaining_id_dictionary[36]["id"] = "2zPEvSxzmCYrpgNn1ZhW4U"

In [484]:
remaining_id_dictionary[37]["id"] = "7Ijn5Ti7fVapkcuVopZfUn"

In [486]:
remaining_id_dictionary[38]["id"] = "5BSIpWY8xzjZSXmeWL8OYL"

In [488]:
remaining_id_dictionary[39]["id"] = "61W54VjGZwy7W36iu9Bhwq"

In [490]:
remaining_id_dictionary[40]["id"] = "2OkyM5knz8Q0HOnIK85vwr"

In [493]:
remaining_id_dictionary[42]["id"] = "6qoH3IoWU0NAoEEKevlZ9B"

In [495]:
remaining_id_dictionary[43]["id"] = "5sNESr6pQfIhL3krM8CtZn"

In [497]:
remaining_id_dictionary[44]["id"] = "5mEwhCsqadBZIE7SSrv1gC"

In [499]:
remaining_id_dictionary[45]["id"] = "0uktXWVUCLrWnl8ZTuWP9I"

In [501]:
remaining_id_dictionary[46]["id"] = "2vk14Ko9Jxr2Ef5PTxqmbp"

In [503]:
remaining_id_dictionary[47]["id"] = "0ICuq5hzkW5rheJZxnjFnS"

In [505]:
remaining_id_dictionary[48]["id"] = "40RcMOyjymXFKBxlMuQJnA"

In [507]:
remaining_id_dictionary[49]["id"] = "4gqjxYiEorFk0y9b92uFdE"

In [509]:
remaining_id_dictionary[50]["id"] = "0USctUIMP2DEp0oeBSyqJf"

In [511]:
remaining_id_dictionary[51]["id"] = "4fyruYUFg0HPlbdv3o0f6M"

In [513]:
remaining_id_dictionary[52]["id"] = "63MeDGLOFgYCvIv7kgA6mf"

In [515]:
remaining_id_dictionary[53]["id"] = "5Qe6G7ndKJEs0s6tMshaJm"

In [518]:
remaining_id_dictionary[55]["id"] = "1ddRRUstEVlCTPj2Wd5zjv"

In [521]:
remaining_id_dictionary[56]["id"] = "60wrTW60WiE0qyUjd8dIiP"

In [524]:
remaining_id_dictionary[57]["id"] = "5WU62y0mntinlwmUfu8YZD"

In [527]:
remaining_id_dictionary[59]["id"] = "2O3uBNEgne3rOu77pgbrYg"

In [529]:
remaining_id_dictionary[60]["id"] = "23ZjWmNcsSAjfGtGvjHdvj"

In [531]:
remaining_id_dictionary[61]["id"] = "7bWonRM5QK6qA0dwYISntA"

In [533]:
remaining_id_dictionary[62]["id"] = "34BtE2Fv3gnmP5NPar4FRH"

In [535]:
remaining_id_dictionary[63]["id"] = "1dZxI6OuzwJ283y3gOU1Kq"

In [539]:
remaining_id_dictionary[66]["id"] = "00RYiT3a2lNQ1wTKBQXWSw"

In [541]:
remaining_id_dictionary[67]["id"] = "0TwCdMeFjEltHiNjtii8Jk"

In [543]:
remaining_id_dictionary[68]["id"] = "7dZOwXFupdzK8Zi4iTJlXJ"

In [545]:
remaining_id_dictionary[69]["id"] = "1rkzASjTsDga7grN4g4aSx"

In [547]:
remaining_id_dictionary[70]["id"] = "0AI0VW1BPoOel5Gtyo11wO"

In [549]:
remaining_id_dictionary[71]["id"] = "6bGm59m6SQC16Xg9pWvGDh"

In [551]:
remaining_id_dictionary[72]["id"] = "3lLWP1a6wFYPvIwCNGBoOR"

In [553]:
remaining_id_dictionary[73]["id"] = "2nQmqmd3Qm6X2wBirpYclv"

In [557]:
remaining_id_dictionary[76]["id"] = "0Jcnr8HUGQdaOiSkqpdEKX"

In [559]:
remaining_id_dictionary[77]["id"] = "41gIW3UNXhTGdIXSy7w2au"

In [561]:
remaining_id_dictionary[78]["id"] = "3IvdD05hXRGba93ev8iieR"

In [563]:
remaining_id_dictionary[79]["id"] = "4L0nc0tHfDjr3HlAm1EFeu"

In [567]:
remaining_id_dictionary[82]["id"] = "7CAbonTfvCZz1JSoceVD4h"

In [570]:
remaining_id_dictionary[83]["id"] = "4qEoqyPbLYnLOii6mKlIjI"

In [572]:
remaining_id_dictionary[84]["id"] = "7eAWmk6IDva2iA0q2Wx9Lv"

In [575]:
remaining_id_dictionary[86]["id"] = "3mj46XKJ2aEm1z7ZvbwyLi"

In [577]:
remaining_id_dictionary[87]["id"] = "0kcxKrUvHMOYIEqjJgnQHZ"

In [579]:
remaining_id_dictionary[88]["id"] = "2JJ5h2I1S0UR4tyD0U1I7K"

In [581]:
remaining_id_dictionary[89]["id"] = "0A2lJqfi04TKDTJYoKBxa9"

In [583]:
remaining_id_dictionary[90]["id"] = "2QDx0OJsKHrrjkHbfyP3pv"

In [585]:
remaining_id_dictionary[91]["id"] = "2bvxIrwSsQiaHOePGrjnJ9"

In [587]:
remaining_id_dictionary[92]["id"] = "39mAAkuRCowaZ3J9lYjqnP"

In [589]:
remaining_id_dictionary[93]["id"] = "3KliPMvk1EvFZu9cvkj8p1"

In [591]:
remaining_id_dictionary[94]["id"] = "1imbxk7YGNkBVdAySWAKJ1"

In [593]:
remaining_id_dictionary[95]["id"] = "5wUaDFJN4O61Azp3zmUpyw"

In [595]:
remaining_id_dictionary[96]["id"] = "2Vevs2eAQNNb7NTpKj5kqA"

In [597]:
remaining_id_dictionary[97]["id"] = "30S7ayrPfoQ2zBHlJmavTn"

In [599]:
remaining_id_dictionary[98]["id"] = "0QDfwssgneF3kVgBo13lRr"

In [601]:
remaining_id_dictionary[99]["id"] = "0UpdZpqyGKwz3GRU1DvpGM"

In [603]:
remaining_id_dictionary[100]["id"] = "3xgK660fsZH7ZDcOMfIdfB"

In [605]:
remaining_id_dictionary[101]["id"] = "7LA65XN3XINyDpn454otUO"

In [607]:
remaining_id_dictionary[102]["id"] = "5sjlao6k5YWtekBIRjgMbx"

In [609]:
remaining_id_dictionary[103]["id"] = "7LvPr4Mchf8YC23O6PdWWY"

In [611]:
remaining_id_dictionary[104]["id"] = "5uVji14OWpj9HMKLXdhjAv"

In [613]:
remaining_id_dictionary[105]["id"] = "5SUKnxEQaFFeudmI4qYLd9"

In [615]:
remaining_id_dictionary[106]["id"] = "31mfhAKOC4q1qSUl8YqiIh"

In [617]:
remaining_id_dictionary[107]["id"] = "4hgrcrkcaHtsdKU35RZyn6"

In [619]:
remaining_id_dictionary[108]["id"] = "1LWgoxKUgZIeodlLY7n3vA"

In [621]:
remaining_id_dictionary[109]["id"] = "3sS03NvxbrVgFNZgvnR26k"

In [623]:
remaining_id_dictionary[110]["id"] = "4vfeVGbKyoHrYly5Sj0GRu"

In [625]:
remaining_id_dictionary[111]["id"] = "35ap6ekPdkGkiUkWfWhZB4"

In [627]:
remaining_id_dictionary[112]["id"] = "6sd2O1KmiyDWKb6x3hZSR9"

In [629]:
remaining_id_dictionary[113]["id"] = "3hWUIMNBwLNug1O8o2PPRW"

In [631]:
remaining_id_dictionary[114]["id"] = "3wdfLVUhmX64lIyWIIEW8s"

In [633]:
remaining_id_dictionary[115]["id"] = "6tZ3b7ik1QDXFAZlXib0YZ"

In [635]:
remaining_id_dictionary[116]["id"] = "6G8kHiVZ1jW7vHMPVRNZU0"

In [637]:
remaining_id_dictionary[117]["id"] = "1dUHF4RyMmMTveJ0Rby6Xm"

In [639]:
remaining_id_dictionary[118]["id"] = "6cblRiEGDRNZgowcm951R3"

In [641]:
remaining_id_dictionary[119]["id"] = "41a7dZcq30Ss5kPMayWRV0"

In [643]:
remaining_id_dictionary[120]["id"] = "3LHYmz86DxGInsRp3wiiW5"

In [645]:
remaining_id_dictionary[121]["id"] = "4lJNen4SMTIJMahALc3DcB"

In [648]:
remaining_id_dictionary[122]["id"] = "5IjRKZlpKVBBRn0COL7pik"

In [650]:
remaining_id_dictionary[123]["id"] = "0B3FovCVaGKS5w1FTidEUP"

In [652]:
remaining_id_dictionary[124]["id"] = "1SmiQ65iSAbPto6gPFlBYm"

In [654]:
remaining_id_dictionary[125]["id"] = "1bIEvOOqf2V3QBrFiClE3Y"

,SongID,id
0,if you could read my mindstars on 54: ultra na...,7lQXHsKgBOdXQ51GSNYIf5
1,one more trydivine,3lMaQLliTd1wlzc3lQitPI
2,can't get enoughpatty loveless,1XDihfFjzdKt3bgFIZcPsm
3,hello l.o.v.e.john michael montgomery,1GoVQ3eadHrcjpJuAzAxjA
4,jigga my njay-z,0Db1a7sL9MOjXgO5Z7zRM4
...,...,...
121,"thundercloudslabrinth, sia & diplo present... lsd",4lJNen4SMTIJMahALc3DcB
122,stop snitchingyg,5IjRKZlpKVBBRn0COL7pik
123,"brown skin girlbeyonce, saint jhn & wizkid fea...",0B3FovCVaGKS5w1FTidEUP
124,it's nice to have a friendtaylor swift,1SmiQ65iSAbPto6gPFlBYm


In [660]:
fully_missing_ids = new_ids.merge(pd.DataFrame(remaining_id_dictionary).loc[:, ["SongID", "id"]], how="left", on="SongID", suffixes=("", "_"))

In [664]:
fully_missing_ids.id_ = fully_missing_ids.id_.fillna(fully_missing_ids.id)

In [673]:
fully_missing_ids.popularity = fully_missing_ids.popularity.astype("float32")

In [675]:
fully_missing_ids[lambda x: ~(x.popularity.isnull())].groupby("SongID", group_keys=False).apply(lambda x: x.loc[x.popularity.idxmax()])

,SongID,Song,Performer,popularity,explicit,id,id_
SongID,,,,,,,
3ameminem,3ameminem,3 a.m.,Eminem,52.0,True,7Djpvy4lZJNI8rTOVLf1H7,7Djpvy4lZJNI8rTOVLf1H7
a change is gonna comeadam lambert,a change is gonna comeadam lambert,A Change Is Gonna Come,Sam Cooke,68.0,False,0KOE1hat4SIer491XKk4Pa,0KOE1hat4SIer491XKk4Pa
ain't no sunshinekris allen,ain't no sunshinekris allen,Ain't No Sunshine,Bill Withers,78.0,False,1k1Bqnv2R0uJXQN4u6LKYt,1k1Bqnv2R0uJXQN4u6LKYt
all datmoneybagg yo x megan thee stallon,all datmoneybagg yo x megan thee stallon,All Dat (with Megan Thee Stallion),Moneybagg Yo,60.0,True,2eukAOMjUM4pxnOdIsUia9,2eukAOMjUM4pxnOdIsUia9
"all n my grillmissy ""misdemeanor"" elliott featuring big boi & nicole","all n my grillmissy ""misdemeanor"" elliott feat...",All N My Grill (feat. Big Boi of Outkast & Nic...,Missy Elliott,51.0,True,4vO3U8AKIjwBEBCTbEFbD9,4vO3U8AKIjwBEBCTbEFbD9
...,...,...,...,...,...,...,...
you all datbaha men with imani coppola,you all datbaha men with imani coppola,You All Dat,Baha Men,28.0,False,4KkzkMg3wMqezfHYNPlVy8,4KkzkMg3wMqezfHYNPlVy8
you are the music in mezac efron & vanessa anne hudgens,you are the music in mezac efron & vanessa ann...,You Are the Music in Me,Troy,60.0,False,6TRp2628QKH3kY6KrCnjqp,6TRp2628QKH3kY6KrCnjqp
you don't own megrace featuring g-eazy,you don't own megrace featuring g-eazy,You Don't Own Me (feat. G-Eazy),SAYGRACE,70.0,False,6KI1ZpZWYAJLvmVhCJz65G,6KI1ZpZWYAJLvmVhCJz65G


In [680]:
last_api_track_calls = fully_missing_ids[lambda x: x.popularity.isnull()].drop_duplicates().drop(["Song", "Performer", "popularity", "explicit", "id"], axis=1)

In [681]:
last_api_track_calls

,SongID,id_
0,if you could read my mindstars on 54: ultra na...,7lQXHsKgBOdXQ51GSNYIf5
15,one more trydivine,3lMaQLliTd1wlzc3lQitPI
16,can't get enoughpatty loveless,1XDihfFjzdKt3bgFIZcPsm
21,hello l.o.v.e.john michael montgomery,1GoVQ3eadHrcjpJuAzAxjA
22,jigga my njay-z,0Db1a7sL9MOjXgO5Z7zRM4
...,...,...
817,"thundercloudslabrinth, sia & diplo present... lsd",4lJNen4SMTIJMahALc3DcB
826,stop snitchingyg,5IjRKZlpKVBBRn0COL7pik
836,"brown skin girlbeyonce, saint jhn & wizkid fea...",0B3FovCVaGKS5w1FTidEUP
849,it's nice to have a friendtaylor swift,1SmiQ65iSAbPto6gPFlBYm


In [719]:
calls = last_api_track_calls.dropna().reset_index(drop=True)
_temp_df = pd.DataFrame(columns=["id", "Performer", "popularity", "explicit", "Song"])
i = 0
for s in range(0, len(calls.values), 25):
    id_string = ",".join([x[1] for x in calls.values[s:s+25]])
    _url = f"https://api.spotify.com/v1/tracks?market=ES&ids={id_string}"
    _response_s = requests.get(_url, headers=headers)
    _response_s = _response_s.json()
    for _item in _response_s["tracks"]:
        if _item != None:
            _temp_df.loc[i, "Performer"] = _item["artists"][0]["name"]
            _temp_df.loc[i, "popularity"] = _item["popularity"]
            _temp_df.loc[i, "explicit"] = _item["explicit"]
            _temp_df.loc[i, "Song"] = _item["name"]
            _temp_df.loc[i, "id"] = _item["id"]
        i += 1
    print(f"finished with {s+25}")

finished with 25
finished with 50
finished with 75
finished with 100
finished with 125


In [723]:
last_api_track_calls.merge(_temp_df, "left", left_on="id_", right_on="id").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SongID      126 non-null    object
 1   id_         115 non-null    object
 2   id          105 non-null    object
 3   Performer   105 non-null    object
 4   popularity  105 non-null    object
 5   explicit    105 non-null    object
 6   Song        105 non-null    object
dtypes: object(7)
memory usage: 7.9+ KB


In [751]:
last_api_track_calls = last_api_track_calls.merge(_temp_df, "left", left_on="id_", right_on="id")

In [725]:
new_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 874 entries, 0 to 873
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SongID      874 non-null    object
 1   Song        747 non-null    object
 2   Performer   747 non-null    object
 3   popularity  747 non-null    object
 4   explicit    747 non-null    object
 5   id          747 non-null    object
dtypes: object(6)
memory usage: 87.8+ KB


In [757]:
last_api_track_calls

,SongID,id_,id,Performer,popularity,explicit,Song
0,if you could read my mindstars on 54: ultra na...,7lQXHsKgBOdXQ51GSNYIf5,NaN,NaN,NaN,NaN,NaN
1,one more trydivine,3lMaQLliTd1wlzc3lQitPI,3lMaQLliTd1wlzc3lQitPI,Done Again,0,False,One More Try
2,can't get enoughpatty loveless,1XDihfFjzdKt3bgFIZcPsm,1XDihfFjzdKt3bgFIZcPsm,BFM Hits,0,False,Can't Get Enough
3,hello l.o.v.e.john michael montgomery,1GoVQ3eadHrcjpJuAzAxjA,1GoVQ3eadHrcjpJuAzAxjA,John Michael Montgomery,4,False,Hello L-O-V-E
4,jigga my njay-z,0Db1a7sL9MOjXgO5Z7zRM4,0Db1a7sL9MOjXgO5Z7zRM4,Ruff Ryders,45,True,Jigga My Nigga
...,...,...,...,...,...,...,...
121,"thundercloudslabrinth, sia & diplo present... lsd",4lJNen4SMTIJMahALc3DcB,4lJNen4SMTIJMahALc3DcB,Sia,73,False,"Thunderclouds (feat. Sia, Diplo, and Labrinth)"
122,stop snitchingyg,5IjRKZlpKVBBRn0COL7pik,5IjRKZlpKVBBRn0COL7pik,YG,61,True,Stop Snitchin
123,"brown skin girlbeyonce, saint jhn & wizkid fea...",0B3FovCVaGKS5w1FTidEUP,0B3FovCVaGKS5w1FTidEUP,Blue Ivy,65,False,BROWN SKIN GIRL
124,it's nice to have a friendtaylor swift,1SmiQ65iSAbPto6gPFlBYm,1SmiQ65iSAbPto6gPFlBYm,Taylor Swift,65,False,It’s Nice To Have A Friend


In [766]:
missing_song_ids_1 = fully_missing_ids[lambda x: ~(x.popularity.isnull())].groupby("SongID", group_keys=False).apply(lambda x: x.loc[x.popularity.idxmax()]).reset_index(drop=True)

In [767]:
missing_song_ids_1.columns

Index(['SongID', 'Song', 'Performer', 'popularity', 'explicit', 'id', 'id_'], dtype='object')

In [769]:
last_api_track_calls = last_api_track_calls.loc[:, ["SongID", "Song", "Performer", "popularity", "explicit", "id", "id_"]]

,SongID,Song,Performer,popularity,explicit,id,id_
0,3ameminem,3 a.m.,Eminem,52,True,7Djpvy4lZJNI8rTOVLf1H7,7Djpvy4lZJNI8rTOVLf1H7
1,a change is gonna comeadam lambert,A Change Is Gonna Come,Sam Cooke,68,False,0KOE1hat4SIer491XKk4Pa,0KOE1hat4SIer491XKk4Pa
2,ain't no sunshinekris allen,Ain't No Sunshine,Bill Withers,78,False,1k1Bqnv2R0uJXQN4u6LKYt,1k1Bqnv2R0uJXQN4u6LKYt
3,all datmoneybagg yo x megan thee stallon,All Dat (with Megan Thee Stallion),Moneybagg Yo,60,True,2eukAOMjUM4pxnOdIsUia9,2eukAOMjUM4pxnOdIsUia9
4,"all n my grillmissy ""misdemeanor"" elliott feat...",All N My Grill (feat. Big Boi of Outkast & Nic...,Missy Elliott,51,True,4vO3U8AKIjwBEBCTbEFbD9,4vO3U8AKIjwBEBCTbEFbD9
...,...,...,...,...,...,...,...
261,"thundercloudslabrinth, sia & diplo present... lsd","Thunderclouds (feat. Sia, Diplo, and Labrinth)",Sia,73,False,4lJNen4SMTIJMahALc3DcB,4lJNen4SMTIJMahALc3DcB
262,stop snitchingyg,Stop Snitchin,YG,61,True,5IjRKZlpKVBBRn0COL7pik,5IjRKZlpKVBBRn0COL7pik
263,"brown skin girlbeyonce, saint jhn & wizkid fea...",BROWN SKIN GIRL,Blue Ivy,65,False,0B3FovCVaGKS5w1FTidEUP,0B3FovCVaGKS5w1FTidEUP
264,it's nice to have a friendtaylor swift,It’s Nice To Have A Friend,Taylor Swift,65,False,1SmiQ65iSAbPto6gPFlBYm,1SmiQ65iSAbPto6gPFlBYm


In [774]:
original_scraped_ids = pd.read_csv("../data/interim/api_scrapes.csv")

In [777]:
original_scraped_ids = original_scraped_ids.dropna()[lambda x: ~(x.popularity.isnull())].groupby("SongID", group_keys=False).apply(lambda x: x.loc[x.popularity.idxmax()]).reset_index(drop=True)

In [779]:
original_scraped_ids = original_scraped_ids.assign(id_ = lambda x: x.id)

In [841]:
attributes_left_to_get = pd.concat([missing_song_ids_1, last_api_track_calls, original_scraped_ids], axis=0, ignore_index=True).drop("id", axis=1)

In [790]:
found_m_songs = need_songs_df.drop_duplicates("SongID").loc[:, ["Song", "id", "Performer", "SongID"]][lambda x: x.id.isnull()].merge(master_song_list, left_on=["Song", "Performer"], right_on=["name", "artists"])

In [792]:
found_m_songs = found_m_songs.groupby("SongID", group_keys=False).apply(lambda x: x.loc[x.popularity.idxmax()]).reset_index(drop=True)

In [795]:
found_m_songs.columns

Index(['Song', 'id_x', 'Performer', 'SongID', 'acousticness', 'artists',
       'danceability', 'duration_ms', 'energy', 'explicit', 'id_y',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'name',
       'popularity', 'release_date', 'speechiness', 'tempo', 'valence',
       'year'],
      dtype='object')

In [ ]:
_temp_df2 = pd.DataFrame(columns=["id", "popularity", "id_y"])
headers = {"Authorization": f"Bearer {token}"}
i = 0
id_string = ",".join([x for x in found_m_songs.id_y.values[:30]])
_url = f"https://api.spotify.com/v1/tracks?market=ES&ids={id_string}"
_response_s = requests.get(_url, headers=headers)
_response_s = _response_s.json()
for _item in _response_s["tracks"]:
    if _item != None:
        _temp_df2.loc[i, "popularity"] = _item["popularity"]
        _temp_df2.loc[i, "id"] = _item["id"]
        _temp_df2.loc[i, "id_y"] = found_m_songs.id_y.values[i]
    i += 1

In [831]:
i = 30
id_string = ",".join([x for x in found_m_songs.id_y.values[30:]])
_url = f"https://api.spotify.com/v1/tracks?market=ES&ids={id_string}"
_response_s = requests.get(_url, headers=headers)
_response_s = _response_s.json()
for _item in _response_s["tracks"]:
    if _item != None:
        _temp_df2.loc[i, "popularity"] = _item["popularity"]
        _temp_df2.loc[i, "id"] = _item["id"]
        _temp_df2.loc[i, "id_y"] = found_m_songs.id_y.values[i]
    i += 1

In [836]:
found_m_songs = found_m_songs.merge(_temp_df2, "left", on=["id_y"], suffixes=("_old", ""))

In [843]:
found_m_songs.shape[0] + attributes_left_to_get.shape[0]

655

In [870]:
codes = attributes_left_to_get.dropna().id_.values

In [872]:
headers = {"Authorization": f"Bearer {token}"}
__temp_df = pd.DataFrame(columns=["id_", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature", "duration"])
i = 0
for g in range(0, len(codes), 25):
    id_string = ",".join([x for x in codes[g:g+25]])
    res = requests.get(f"https://api.spotify.com/v1/audio-features?ids={id_string}", headers=headers)
    res = res.json()
    for _item in res["audio_features"]:
        if _item != None:
            __temp_df.loc[i, "id_"] = _item["id"]
            __temp_df.loc[i, "danceability"] = _item["danceability"]
            __temp_df.loc[i, "energy"] = _item["energy"]
            __temp_df.loc[i, "key"] = _item["key"]
            __temp_df.loc[i, "loudness"] = _item["loudness"]
            __temp_df.loc[i, "mode"] = _item["mode"]
            __temp_df.loc[i, "speechiness"] = _item["speechiness"]
            __temp_df.loc[i, "acousticness"] = _item["acousticness"]
            __temp_df.loc[i, "instrumentalness"] = _item["instrumentalness"]
            __temp_df.loc[i, "liveness"] = _item["liveness"]
            __temp_df.loc[i, "valence"] = _item["valence"]
            __temp_df.loc[i, "tempo"] = _item["tempo"]
            __temp_df.loc[i, "time_signature"] = _item["time_signature"]
            __temp_df.loc[i, "duration"] = _item["duration_ms"]
        i += 1
    print(f"done with {g+25}")

done with 25
done with 50
done with 75
done with 100
done with 125
done with 150
done with 175
done with 200
done with 225
done with 250
done with 275
done with 300
done with 325
done with 350
done with 375
done with 400
done with 425
done with 450
done with 475
done with 500
done with 525
done with 550
done with 575


In [899]:
missing_songs_with_attr = attributes_left_to_get.merge(__temp_df, "left", on="id_").drop_duplicates()

In [896]:
found_m_songs = found_m_songs.loc[:, ['SongID', 'Song', 'Performer', 'popularity', 'explicit', 'id_y',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].rename(columns={"id_y": "id_", "duration_ms": "duration"}).assign(time_signature = lambda x: np.nan)

In [897]:
found_m_songs = found_m_songs.loc[:, ['SongID', 'Song', 'Performer', 'popularity', 'explicit', 'id_',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'duration']]

In [919]:
songs_to_add_to_full = pd.concat([missing_songs_with_attr, found_m_songs], axis=0, ignore_index=True) 

In [904]:
complete_info_df.shape

(104837, 31)

In [924]:
complete_info_df.columns

Index(['Song', 'Performer', 'Week Position', 'WeekID',
       'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
       'SongID', 'Week_No', 'Performer_y', 'Song_y', 'genre', 'id', 'url',
       'album', 'explicit', 'duration', 'popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'timesignature'],
      dtype='object')

In [949]:
pulled_songs = need_songs_df.loc[:, ["Song", "Performer", "Week Position", "WeekID", "Previous Week Position", "Peak Position", "Weeks on Chart", "Instance", "SongID", "Week_No", "genre", "url", "album"]].merge(songs_to_add_to_full, "inner", left_on=["SongID"], right_on=["SongID"]).rename(columns={"time_signature":"timesignature"})

In [951]:
pulled_songs = pulled_songs.rename(columns={"Song_x": "Song", "Performer_x": "Performer", "id_": "id"})

In [953]:
pulled_songs = pulled_songs.loc[: ,['Song', 'Performer', 'Week Position', 'WeekID',
'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
'SongID', 'Week_No', 'Performer_y', 'Song_y', 'genre', 'id', 'url',
'album', 'explicit', 'duration', 'popularity', 'danceability', 'energy',
'key', 'loudness', 'mode', 'speechiness', 'acousticness',
'instrumentalness', 'liveness', 'valence', 'tempo', 'timesignature']]

In [957]:
full_df_final = pd.concat([complete_info_df, pulled_songs], axis=0).sort_values(["WeekID", "Week Position"], ignore_index=True)

In [959]:
full_df.shape == full_df_final.shape

True

In [968]:
full_df_final.loc[:, ["Song", "Performer", "Week Position", "WeekID", "Previous Week Position", "Peak Position",
"Weeks on Chart", "Instance", "SongID", "Week_No", "Performer_y", "Song_y", "id"]].to_csv("../src/data/chart_data.csv", index=False, float_format="%.2f")

In [971]:
full_df_final.iloc[:, 12:].to_csv("../src/data/attribute_data.csv", index=False, float_format="%.4f")

In [6]:
full_df = pd.read_csv("../src/data/chart_data.csv")

In [7]:
attrs = pd.read_csv("../src/data/attribute_data.csv")

In [58]:
attrs = attrs.rename(columns={"id": "id2"})

In [59]:
full_df_final = pd.concat([full_df, attrs], axis=1)

In [72]:
straggalers = full_df_final[lambda x: x.id != x.id2].drop_duplicates("SongID").reset_index(drop=True).loc[:, ["Song", "Performer", "SongID", "id", "id2", "liveness"]].to_dict(orient="index")

In [74]:
len(straggalers)

107

In [79]:
straggalers[0]["id"] = "4mljNnZ2eaXKSbEAOzLlLE"

In [81]:
straggalers[1]["id"] = "1TQ6a2NEA8LmKfgf0yeBvT"

In [84]:
straggalers[2]["id"] = "0p5Bv16XNo5B7AdeKD0H0F"

In [86]:
straggalers[3]["id"] = "2BNV1D5X4ioefyOE0uDFOL"

In [88]:
straggalers[4]["id"] = "5pinOKTQyPCPpgjIOKVFY3"

In [90]:
straggalers[5]["id"] = "3y5pef5Z472KoYxaSdig9I"

In [92]:
straggalers[6]["id"] = "3wMUvT6eIw2L5cZFG1yH9j"

In [94]:
straggalers[7]["id"] = "1OvjOVieEsYqGiK1T3mUv9"

In [101]:
straggalers[8]["id"] = "65wuvhPFGi9VLiKyjL4oEg"

In [104]:
straggalers[10]["id"] = "2hKdd3qO7cWr2Jo0Bcs0MA"

In [106]:
straggalers[11]["id"] = "514AneEN1C63hmnVNC4dLJ"

In [108]:
straggalers[12]["id"] = "52LJ3hyknOijCrE5gCD0rE"

In [110]:
straggalers[13]["id"] = "1LTAztvz5NpyAwbGlVZTDr"

In [112]:
straggalers[14]["id"] = "4SyadrABZJIjeND1HPJS31"

In [114]:
straggalers[15]["id"] = "5PLExXqIuPID6n5Dh261QN"

In [116]:
straggalers[16]["id"] = "0jdkdbnRwNVHJQ6zL84pQs"

In [118]:
straggalers[17]["id"] = "0OSAJEA0ekX6pr2nuiPzX7"

In [120]:
straggalers[18]["id"] = "7lVNTXkI3cHFvcXiI8damb"

In [122]:
straggalers[19]["id"] = "4ipZsAA3YuqCDSXiPoEGIv"

In [124]:
straggalers[20]["id"] = "6Wt5QBzHp9vdSLAFlvdnDQ"

In [126]:
straggalers[21]["id"] = "7aXXjA800sXYKpM8D9pCVT"

In [128]:
straggalers[22]["id"] = "69cakNr5j4v5pyumMgarCY"

In [131]:
straggalers[24]["id"] = "14ymPtb1n06KzSm0J9bmMZ"

In [133]:
straggalers[25]["id"] = "3JdXx7GlzYAgw1ELOOu1KO"

In [135]:
straggalers[26]["id"] = "1ZlVEstPFrKs3alKIejN6I"

In [137]:
straggalers[27]["id"] = "1k5nyIskiZq4mbTElwO3PO"

In [139]:
straggalers[28]["id"] = "1p5kC161fWMA2QlwV3Vi4W"

In [141]:
straggalers[29]["id"] = "0q7z99dIbw37rabJtGBZt5"

In [143]:
straggalers[30]["id"] = "35Y18Dv2pAiJ7v0Zou9gxI"

In [146]:
straggalers[32]["id"] = "4UC4H4vX3bJtcgtKR0ZCFJ"

In [148]:
straggalers[33]["id"] = "3eRLKD1Lgc2OrYGxRxX62s"

In [150]:
straggalers[34]["id"] = "5dTHtzHFPyi8TlTtzoz1J9"

In [153]:
straggalers[35]["id"] = "4i2HYTGM7yHty0qVhD3lqD"

In [155]:
straggalers[36]["id"] = "3eY3wZJPHwVtD4TM518Ov6"

In [157]:
straggalers[37]["id"] = "7I60v3Ctgf1eMefMc69zAo"

In [159]:
straggalers[38]["id"] = "4OikxHAgM14El0wQIyyhP8"

In [161]:
straggalers[39]["id"] = "4hYI8nymxxb6VjmIBi5Agx"

In [163]:
straggalers[40]["id"] = "6w1DBbA8akxHPiBQ6nAyJq"

In [166]:
straggalers[42]["id"] = "0EKBV6GybPtALXUgWqWrym"

In [168]:
straggalers[43]["id"] = "0JfAMd3xTqm7ZYhBmQjYzt"

In [171]:
straggalers[45]["id"] = "5uGGmsI5G6i3OvgjRziSI0"

In [173]:
straggalers[46]["id"] = "4kmGRW1BgoKendL6V7cz29"

In [175]:
straggalers[47]["id"] = "66TRwr5uJwPt15mfFkzhbi"

In [177]:
straggalers[48]["id"] = "6A9nyMBdNUzVcFHZUEZ71F"

In [179]:
straggalers[49]["id"] = "7mZwaEEaHSeTYC2hTAjUki"

In [181]:
straggalers[50]["id"] = "27K0tchvM5KszaSJEX7fj5"

In [183]:
straggalers[51]["id"] = "7CMIy0uwXyeBv2MvaEUJM1"

In [185]:
straggalers[52]["id"] = "3ksZVeqlLDmEPG71vXQdT6"

In [189]:
straggalers[55]["id"] = "0wVluBsVAVzBKrqspuCcwR"

In [191]:
straggalers[56]["id"] = "4avrnJ1t946he63pkD4syn"

In [193]:
straggalers[57]["id"] = "7Kpqjspw4Y7HrvItIRcBiW"

In [195]:
straggalers[58]["id"] = "5kMsjfhYMZ1L9Btuu0AIRx"

In [197]:
straggalers[59]["id"] = "2igwFfvr1OAGX9SKDCPBwO"

In [199]:
straggalers[60]["id"] = "35srNkJVToAzYiZzron1FY"

In [201]:
straggalers[61]["id"] = "0HPD5WQqrq7wPWR7P7Dw1i"

In [203]:
straggalers[62]["id"] = "7lWUJpax919G4JdaFEVmCy"

In [205]:
straggalers[63]["id"] = "4ljDnzzqwnRIynr1g55um4"

In [211]:
straggalers[67]["id"] = "4JOP8ELK6AaeySe7sKe996"

In [213]:
straggalers[68]["id"] = "6BWrIaoodtt7EX6kToWZw0"

In [215]:
straggalers[69]["id"] = "4AIazttPmHpd7p7pwJw692"

In [217]:
straggalers[70]["id"] = "35QGdv6aFbIaLM0Ozr3rHt"

In [219]:
straggalers[71]["id"] = "0R2yQJOFsKqS0SsgPdWqtP"

In [222]:
straggalers[72]["id"] = "3avYqdwHKEq8beXbeWCKqJ"

In [224]:
straggalers[73]["id"] = "4ycLiPVzE5KamivXrAzGFG"

In [226]:
straggalers[74]["id"] = "1zPDgF3K6QXcTopDB3SVg9"

In [229]:
straggalers[76]["id"] = "3JTMWdhcJPiegDSe7SvZS3"

In [231]:
straggalers[77]["id"] = "6Uj8ru7TNLDIelbuQ8eBd6"

In [233]:
straggalers[78]["id"] = "1YcOjtjiwwiwlrzmeueQ45"

In [236]:
straggalers[80]["id"] = "57xW8NlNhWLdW78w25R3tb"

In [238]:
straggalers[81]["id"] = "7b5vGhtkuXfVSRxrP5OiQC"

In [241]:
straggalers[83]["id"] = "0m1KYWlT6LhFRBDVq9UNx4"

In [243]:
straggalers[84]["id"] = "7EQGXaVSyEDsCWKmUcfpLk"

In [245]:
straggalers[85]["id"] = "4X5f3vT8MRuXF68pfjNte5"

In [247]:
straggalers[86]["id"] = "4bO6IzKXMWhmncfIO9GBq2"

In [249]:
straggalers[87]["id"] = "6mH3qVIeOsnQIAho5eWwhH"

In [251]:
straggalers[88]["id"] = "07o54snAilKYHsW8bGFe9x"

In [253]:
straggalers[89]["id"] = "0UUqS0pVFpsFRbNjDv7Enf"

In [255]:
straggalers[90]["id"] = "4iiJ57RTsBw4hLZi3sCXBS"

In [260]:
straggalers[91]["id"] = "5h2nC1QYVpK0AJra70Ja3p"

In [262]:
straggalers[92]["id"] = "1SGt65i9AnXYdDQt1AtDRH"

In [265]:
straggalers[93]["id"] = "3osQ5lSSqZb8HWRozdbVPw"

In [267]:
straggalers[94]["id"] = "5qRj5I58zuHkVilcqVWN3P"

In [269]:
straggalers[95]["id"] = "55n9yjI6qqXh5F2mYvUc2y"

In [271]:
straggalers[96]["id"] = "6KBYefIoo7KydImq1uUQlL"

In [273]:
straggalers[97]["id"] = "2yLa0QULdQr0qAIvVwN6B5"

In [275]:
straggalers[98]["id"] = "7ER7vcy7u2ajg5IV45WBFw"

In [277]:
straggalers[99]["id"] = "3swc6WTsr7rl9DqQKQA55C"

In [279]:
straggalers[100]["id"] = "2ijef6ni2amuunRoKTlgww"

In [281]:
straggalers[101]["id"] = "3KkXRkHbMCARz0aVfEt68P"

In [283]:
straggalers[102]["id"] = "2EGtBS7Ws3ajiSkJLviFJ4"

In [285]:
straggalers[103]["id"] = "1lOe9qE0vR9zwWQAOk6CoO"

In [287]:
straggalers[104]["id"] = "6zegtH6XXd2PDPLvy1Y0n2"

In [289]:
straggalers[105]["id"] = "2wrJq5XKLnmhRXHIAf9xBa"

In [291]:
straggalers[106]["id"] = "6cy3ki60hLwimwIje7tALf"

In [295]:
straggalers_df = pd.DataFrame.from_dict(straggalers, "index")

,Song,Performer,SongID,id,id2,liveness
0,(god must have spent) a little more time on you,'n sync,(god must have spent) a little more time on yo...,4mljNnZ2eaXKSbEAOzLlLE,NaN,NaN
1,everybody's free (to wear sunscreen),baz luhrmann,everybody's free (to wear sunscreen)baz luhrmann,1TQ6a2NEA8LmKfgf0yeBvT,NaN,NaN
2,i do (cherish you),98 degrees,i do (cherish you)98 degrees,0p5Bv16XNo5B7AdeKD0H0F,NaN,NaN
3,the christmas song (chestnuts roasting on an o...,christina aguilera,the christmas song (chestnuts roasting on an o...,2BNV1D5X4ioefyOE0uDFOL,NaN,NaN
4,a country boy can survive (y2k version),chad brock with hank williams jr. & george jones,a country boy can survive (y2k version)chad br...,5pinOKTQyPCPpgjIOKVFY3,NaN,NaN
...,...,...,...,...,...,...
102,chopstix,schoolboy q + travis scott,chopstixschoolboy q + travis scott,2EGtBS7Ws3ajiSkJLviFJ4,NaN,NaN
103,ran$om,lil tecca,ran$omlil tecca,1lOe9qE0vR9zwWQAOk6CoO,NaN,NaN
104,don't call me angel (charlie's angels),"ariana grande, miley cyrus & lana del rey",don't call me angel (charlie's angels)ariana g...,6zegtH6XXd2PDPLvy1Y0n2,NaN,NaN
105,"10,000 hours",dan + shay & justin bieber,"10,000 hoursdan + shay & justin bieber",2wrJq5XKLnmhRXHIAf9xBa,NaN,NaN


In [309]:
headers = {"Authorization": f"Bearer {token}"}
__temp_df = pd.DataFrame(columns=["id", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature", "duration"])
i = 0

In [314]:
codes = straggalers_df[~(straggalers_df.id.isnull())].id.values
for g in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[g:g+50]])
    res = requests.get(f"https://api.spotify.com/v1/audio-features?ids={id_string}", headers=headers)
    res = res.json()
    for _item in res["audio_features"]:
        if _item != None:
            __temp_df.loc[i, "id"] = _item["id"]
            __temp_df.loc[i, "danceability"] = _item["danceability"]
            __temp_df.loc[i, "energy"] = _item["energy"]
            __temp_df.loc[i, "key"] = _item["key"]
            __temp_df.loc[i, "loudness"] = _item["loudness"]
            __temp_df.loc[i, "mode"] = _item["mode"]
            __temp_df.loc[i, "speechiness"] = _item["speechiness"]
            __temp_df.loc[i, "acousticness"] = _item["acousticness"]
            __temp_df.loc[i, "instrumentalness"] = _item["instrumentalness"]
            __temp_df.loc[i, "liveness"] = _item["liveness"]
            __temp_df.loc[i, "valence"] = _item["valence"]
            __temp_df.loc[i, "tempo"] = _item["tempo"]
            __temp_df.loc[i, "time_signature"] = _item["time_signature"]
            __temp_df.loc[i, "duration"] = _item["duration_ms"]
        i += 1
    print(f"done with {g+50}")

done with 50
done with 100


In [332]:
attrs_of_no_ids = straggalers_df.merge(__temp_df, how="left", on="id").drop(["id2", "liveness_x"], axis=1).rename(columns={"liveness_y": "liveness", "time_signature": "timesignature"})

In [333]:
attrs_of_no_ids.columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration'],
      dtype='object')

In [362]:
repeat_ids_set = set(full_df_final.loc[:, ["Song", "Performer", "Song_y", "Performer_y", "SongID", "id", 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration']].drop_duplicates().groupby("id", group_keys=False).count().sort_values("Song")[lambda x: x.Song > 1].index.values)

In [432]:
repeat_ids_df = full_df_final[lambda x: x.id.isin(repeat_ids_set)].drop_duplicates(["SongID"]).loc[:, ["Song", "Performer", "Song_y","SongID", "id", "WeekID"]].sort_values("id").reset_index(drop=True)

In [438]:
repeat_ids_df.iloc[1, 4] = "7ySUcLPVX7KudhnmNcgY2D"
repeat_ids_df.iloc[2, 4] = "5rmcjZTrE9JR1YrbNZDFNW"
repeat_ids_df.iloc[5, 4] = "0nC9dSFzv5rzeJx6i2NoMD"
repeat_ids_df.iloc[7, 4]  = "3mV41uxrSBgMI1Ws6O2gXx"
repeat_ids_df.iloc[9, 4] = "0Q45S96loRZtRPZYFaNqB0"
repeat_ids_df.iloc[10, 4] =  "4iLPK8QnOvqm0Azbsk2GDn"
repeat_ids_df.iloc[12, 4] = "0yIywEquxORnu1dR0WYsdN"
repeat_ids_df.iloc[14, 4] = "1MGWgLxUXDodD8Tw8TFppN"
repeat_ids_df.iloc[16, 4] = "3kqLybBT5LRSB9QCoR3ojK"
repeat_ids_df.iloc[17, 4] = "2DmBuFsXx1Pd5FHrMF8PD8"
repeat_ids_df.iloc[22, 4] = "4lNrlPtaAKIOZVAdnVpnzR"
repeat_ids_df.iloc[24, 4] = "6xRWoYwfwIKnT8bQGzKbxR"
repeat_ids_df.iloc[25, 4] = "6GNifiuBPrKFpwNBYnooFm"
repeat_ids_df.iloc[33, 4] = "6KUE0HSSudAp9AOycaF6SH"
repeat_ids_df.iloc[35, 4] = "4szcG1nYM1k4r7cu1sa5o0"
repeat_ids_df.iloc[37, 4] = "1huvTbEYtgltjQRXzrNKGi"
repeat_ids_df.iloc[39, 4] = "4RphIJB5s1N2TTrQFFMxb4"
repeat_ids_df.iloc[46, 4] = "5QxbGa4K41SPmY8zeyRs67"
repeat_ids_df.iloc[51, 4] = "1Ej96GIBCTvgH7tNX1r3qr"
repeat_ids_df.iloc[53, 4] = "2YzmfPLqUx5CJOaw5ThsBV"
repeat_ids_df.iloc[54, 4] = "0a4HnQAiD08Yg8z30yfPhI"
repeat_ids_df.iloc[57, 4] = "6fIvjDuusQK59brMgHV9qb"
repeat_ids_df.iloc[59, 4] = "3n3b5WNEohHP7Ou4ndBo6b"
repeat_ids_df.iloc[63, 4] = "0sTyPK3qbH0u7dFJsEHQYm"
repeat_ids_df.iloc[64, 4] = "15bzUwwD1AXJ3F5oxzl1sY"
repeat_ids_df.iloc[67, 4] = "0W5oXFrxZNBTIS1eMW9Ofz"
repeat_ids_df.iloc[68, 4] = "5r9YRhSv8rLodAFRjKmBqb"
repeat_ids_df.iloc[70, 4] = "1LL2jGP1hehUlAccRCxmV4"
repeat_ids_df.iloc[73, 4] = "4CH66Rxcjcj3VBHwmIBj4T"
repeat_ids_df.iloc[78, 4] = "5A1w94uzc1kO1Zhe8WWxC3"
repeat_ids_df.iloc[79, 4] = "6cZPfzVtkicrA3CyjqYKhu"
repeat_ids_df.iloc[81, 4] = "3744qov0yH77cUOwKS2PLW"
repeat_ids_df.iloc[83, 4] = "5MPPttjfGap2C6j6eKcO6J"
repeat_ids_df.iloc[85, 4] = "6dWNtdXVsT4t5oqN4rpN2U"
repeat_ids_df.iloc[87, 4] = "52Hq1nvCDDZ17dhgGWLnCI"
repeat_ids_df.iloc[89, 4] = "0l0CvurVUrr2w3Jj1hOVFc"
repeat_ids_df.iloc[92, 4] = "61frbeZKl33hLAeyWGc5Wn"
repeat_ids_df.iloc[93, 4] = "5eKWXeGMImIg8CLENs6OnB"
repeat_ids_df.iloc[96, 4] = "4bUfMNNmiCmPXVyWzIfKeZ"

In [439]:
repeat_ids_df

,Song,Performer,Song_y,SongID,id,WeekID
0,don't stop the music,rihanna,don't stop the music,don't stop the musicrihanna,0ByMNEPAPpOR5H69DVrTNy,2007-12-08
1,s&m,rihanna featuring britney spears,Don't Stop The Music,s&mrihanna featuring britney spears,7ySUcLPVX7KudhnmNcgY2D,2010-12-04
2,she's mine pt. 2,j. cole,she's mine pt. 2,she's mine pt. 2j. cole,5rmcjZTrE9JR1YrbNZDFNW,2016-12-31
3,she's mine pt.1,j. cole,She's Mine Pt. 1,she's mine pt.1j. cole,0HtOJj7Kl74s1Ngf3MWeif,2016-12-31
4,bad romance,glee cast,bad romance,bad romanceglee cast,0LpE5tfNe15QLvQL4YDi7T,2010-06-05
...,...,...,...,...,...,...
92,baby,glee cast,baby,babyglee cast,61frbeZKl33hLAeyWGc5Wn,2011-03-05
93,somewhere,glee cast,somewhere,somewhereglee cast,5eKWXeGMImIg8CLENs6OnB,2011-10-15
94,somewhere only we know,glee cast,somewhere only we know,somewhere only we knowglee cast,7qekEQyGFRim7qNhxzkaLo,2011-05-14
95,#1,nelly,#1,#1nelly,7rSj5gEdN5KgHwbnEVAZ4b,2001-10-20


In [441]:
headers = {"Authorization": f"Bearer {token}"}
__temp_df = pd.DataFrame(columns=["id", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature", "duration"])
i = 0
codes = repeat_ids_df.id.values
for g in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[g:g+50]])
    res = requests.get(f"https://api.spotify.com/v1/audio-features?ids={id_string}", headers=headers)
    res = res.json()
    for _item in res["audio_features"]:
        if _item != None:
            __temp_df.loc[i, "id"] = _item["id"]
            __temp_df.loc[i, "danceability"] = _item["danceability"]
            __temp_df.loc[i, "energy"] = _item["energy"]
            __temp_df.loc[i, "key"] = _item["key"]
            __temp_df.loc[i, "loudness"] = _item["loudness"]
            __temp_df.loc[i, "mode"] = _item["mode"]
            __temp_df.loc[i, "speechiness"] = _item["speechiness"]
            __temp_df.loc[i, "acousticness"] = _item["acousticness"]
            __temp_df.loc[i, "instrumentalness"] = _item["instrumentalness"]
            __temp_df.loc[i, "liveness"] = _item["liveness"]
            __temp_df.loc[i, "valence"] = _item["valence"]
            __temp_df.loc[i, "tempo"] = _item["tempo"]
            __temp_df.loc[i, "time_signature"] = _item["time_signature"]
            __temp_df.loc[i, "duration"] = _item["duration_ms"]
        i += 1
    print(f"done with {g+50}")
bad_ids =repeat_ids_df.merge(__temp_df, how="left", on="id")

done with 50
done with 100


In [446]:
bad_ids.drop(["Song_y", "WeekID"], axis=1, inplace=True)

In [452]:
bad_ids = bad_ids.rename(columns={"time_signature": "timesignature"})

In [454]:
bad_ids.columns == attrs_of_no_ids.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [459]:
single_ids = set(full_df_final.loc[:, ["Song", "Performer", "Song_y", "Performer_y", "SongID", "id", 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration']].drop_duplicates().groupby("id", group_keys=False).count().sort_values("Song")[lambda x: x.Song == 1].index.values)

In [466]:
single_id_df = full_df_final[full_df_final.id.isin(single_ids)].loc[:, ["Song", "Performer", "SongID", "id", 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration']].drop_duplicates(["SongID"]).reset_index(drop=True)

In [476]:
attribute_dataframe = pd.concat([single_id_df, bad_ids, attrs_of_no_ids], axis=0, ignore_index=True).drop_duplicates()

In [490]:
__temp_df3 = pd.DataFrame(columns=["popularity", "id"])
_artist_df = pd.DataFrame(columns=["artist", "followers", "genres", "artists_id", "popularity", "id_fk"])
i = 0
j = 0
codes = attribute_dataframe.id.dropna().values
for n in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[n:n+50]])
    res = requests.get(f"https://api.spotify.com/v1/tracks?market=US&ids={id_string}", headers=headers)
    res = res.json()
    for _item in res["tracks"]:
        if _item != None:
            __temp_df3.loc[i, "popularity"] = _item["popularity"]
            __temp_df3.loc[i, "id"] = _item["id"]
            for artist in _item["artists"]:
                _artist_df.loc[j, "artist"] = artist["name"]
                _artist_df.loc[j, "artist_id"] = artist["id"]
                _artist_df.loc[j, "id_fk"] = _item["id"]
                j += 1
        i += 1
    print(f"Done with {n+50}......")
    time.sleep(2)

Done with 50......
Done with 100......
Done with 150......
Done with 200......
Done with 250......
Done with 300......
Done with 350......
Done with 400......
Done with 450......
Done with 500......
Done with 550......
Done with 600......
Done with 650......
Done with 700......
Done with 750......
Done with 800......
Done with 850......
Done with 900......
Done with 950......
Done with 1000......
Done with 1050......
Done with 1100......
Done with 1150......
Done with 1200......
Done with 1250......
Done with 1300......
Done with 1350......
Done with 1400......
Done with 1450......
Done with 1500......
Done with 1550......
Done with 1600......
Done with 1650......
Done with 1700......
Done with 1750......
Done with 1800......
Done with 1850......
Done with 1900......
Done with 1950......
Done with 2000......
Done with 2050......
Done with 2100......
Done with 2150......
Done with 2200......
Done with 2250......
Done with 2300......
Done with 2350......
Done with 2400......
Done with 24

In [498]:
headers = {"Authorization": f"Bearer {token}"}
_artist_temp_df = pd.DataFrame(columns=["artist", "followers", "genres", "artist_id", "popularity"])
i = 0
codes = _artist_df.artist_id.dropna().values
for n in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[n:n+50]])
    res = requests.get(f"https://api.spotify.com/v1/artists?ids={id_string}", headers=headers)
    res = res.json()
    for _art in res["artists"]:
        if _art != None:
            _artist_temp_df.loc[i, "followers"] = _art["followers"]["total"]
            _artist_temp_df.loc[i, "genre"] = str(_art["genres"])
            _artist_temp_df.loc[i, "artist_id"] = _art["id"]
            _artist_temp_df.loc[i, "popularity"] =_art["popularity"]
        i += 1
    print(f"Done with {n+50}......")
    time.sleep(2)

Done with 50......
Done with 100......
Done with 150......
Done with 200......
Done with 250......
Done with 300......
Done with 350......
Done with 400......
Done with 450......
Done with 500......
Done with 550......
Done with 600......
Done with 650......
Done with 700......
Done with 750......
Done with 800......
Done with 850......
Done with 900......
Done with 950......
Done with 1000......
Done with 1050......
Done with 1100......
Done with 1150......
Done with 1200......
Done with 1250......
Done with 1300......
Done with 1350......
Done with 1400......
Done with 1450......
Done with 1500......
Done with 1550......
Done with 1600......
Done with 1650......
Done with 1700......
Done with 1750......
Done with 1800......
Done with 1850......
Done with 1900......
Done with 1950......
Done with 2000......
Done with 2050......
Done with 2100......
Done with 2150......
Done with 2200......
Done with 2250......
Done with 2300......
Done with 2350......
Done with 2400......
Done with 24

In [519]:
_artist_df.drop(["followers", "genres", "artists_id", "popularity"], axis=1).merge(_artist_temp_df, "inner",on="artist_id").drop(["artist_y", "genres"], axis=1).rename(columns={"artist_x": "artist"}).drop_duplicates()\
    .to_sql("artists", con="sqlite:///../src/data/music.db")

In [518]:
attribute_dataframe.merge(__temp_df3, "left", on="id").to_sql("attributes", con="sqlite:///../src/data/music.db")

In [523]:
full_df.to_sql("billboard", con="sqlite:///../src/data/music.db")

In [524]:
# Use Genre, Artist_id and Track Ids as the seeds to find similar songs that we can use as our sample to compares these songs to.

In [304]:
engine = sql.create_engine("sqlite:///../src/data/music.db")
conn = engine.connect()

In [8]:
artists  = pd.read_sql("SELECT * FROM artists", con=conn)
attributes = pd.read_sql("SELECT * FROM attributes", con=conn)
billboard = pd.read_sql("SELECT * FROM billboard", con=conn)
conn.close()

In [9]:
# Now Needs to Get a New Sample of Like-Songs to Compare to My Billboard Songs.
    # This will allow me to use Spotify's recommendation engine to gather a large sample of like songs to compare to those that charted.
    # From the recommendations route of the api, I can gather (artist_id, artist_name, album_id, album_name, release_date, song_id, popularity, explicit, duration_ms)
    # Getting this related songs linking them to the charted songs and then acquiring these song's attributes will allow me to start my analysis on charting vs. non-charting songs
    # These recommended songs will give me a better basis for comparing my own billboard data by allowing Spotify's algorithms to find these songs instead of relying on a random selection of
    # songs of previously acquired attributes for songs based on a previous person's analysis.
    # Steps Now:
        # Gather a Large Sample of Songs for each Song with Billboard Song's ID as the song_seed.
        # Acquire this sample's song attributes  

In [15]:
# artists.columns = [artist.name, song.id, artist.id, followers, artist.popularity]
artists.columns

Index(['index', 'artist', 'id_fk', 'artist_id', 'followers', 'popularity',
       'genre'],
      dtype='object')

In [32]:
# attributes.columns = [track.name, artist.name, SongID {billboard join key}, followers, artist.popularity]
attributes.id.dropna()

0       0QWFiyhOqFu9RP8qzP6z5L
1       51QxenFmXlJXUN9mpvxlaL
2       5pgkdQi8HMQHYrJCB4GLsA
3       3MjUtNVVq3C8Fn0MP3zhXa
4       25tmiXojR7whvZDz3LpMb2
                 ...          
8316    2EGtBS7Ws3ajiSkJLviFJ4
8317    1lOe9qE0vR9zwWQAOk6CoO
8318    6zegtH6XXd2PDPLvy1Y0n2
8319    2wrJq5XKLnmhRXHIAf9xBa
8320    6cy3ki60hLwimwIje7tALf
Name: id, Length: 8308, dtype: object

In [64]:
def get_bill_attr(df):
    _temp_df = pd.DataFrame(columns=["explicit", "popularity", "id", "name", "artist"])
    ids = df.id.dropna().values
    i = 0 
    for n in range(0, len(ids), 50):
        query_string = ",".join([x for x in ids[n:n+50]])
        _url = f"https://api.spotify.com/v1/tracks?market=US&ids={query_string}"
        response = requests.get(_url, headers=headers).json()
        for item in response["tracks"]:
            if item != None:
                _temp_df.loc[i, "explicit"] = item["explicit"]
                _temp_df.loc[i, "id"] = item["id"]
                _temp_df.loc[i, "popularity"] = item["popularity"]
                _temp_df.loc[i, "name"] = item["name"]
                _temp_df.loc[i, "artist"] = item["artists"][0]["name"]
            i += 1
        print(f"Done with {n+50}")
        time.sleep(1)
    #ret_df = attributes.merge(_temp_df, "left", on="id")
    return _temp_df

In [38]:
attributes_new = get_bill_attr(attributes)

Done with 50
Done with 100
Done with 150
Done with 200
Done with 250
Done with 300
Done with 350
Done with 400
Done with 450
Done with 500
Done with 550
Done with 600
Done with 650
Done with 700
Done with 750
Done with 800
Done with 850
Done with 900
Done with 950
Done with 1000
Done with 1050
Done with 1100
Done with 1150
Done with 1200
Done with 1250
Done with 1300
Done with 1350
Done with 1400
Done with 1450
Done with 1500
Done with 1550
Done with 1600
Done with 1650
Done with 1700
Done with 1750
Done with 1800
Done with 1850
Done with 1900
Done with 1950
Done with 2000
Done with 2050
Done with 2100
Done with 2150
Done with 2200
Done with 2250
Done with 2300
Done with 2350
Done with 2400
Done with 2450
Done with 2500
Done with 2550
Done with 2600
Done with 2650
Done with 2700
Done with 2750
Done with 2800
Done with 2850
Done with 2900
Done with 2950
Done with 3000
Done with 3050
Done with 3100
Done with 3150
Done with 3200
Done with 3250
Done with 3300
Done with 3350
Done with 3400


In [43]:
attributes_new.explicit = attributes_new.explicit.astype("float32")

In [59]:
multiple_songs_df = attributes_new[(attributes_new.popularity_y.isna()) & ~(attributes_new.id.isnull())]

In [65]:
multiple_ids = get_bill_attr(multiple_songs_df)

Done with 50
Done with 100
Done with 150
Done with 200
Done with 250
Done with 300
Done with 350
Done with 400
Done with 450
Done with 500


In [67]:
multiple_ids.head()

,explicit,popularity,id,name,artist
0,False,55,615auKcis6sGDrHk6UTeog,Hands,Jewel
1,False,63,3PqAnW2bB5TQnMoBivyFOb,Are You That Somebody,Aaliyah
2,True,54,6vUPPEN2V2lZ78b6DyEIjZ,What It's Like,Everlast
3,False,60,2QSUyofqpGDCo026OPiTBQ,1999 - 2019 Remaster,Prince
4,False,55,2feekrJqyB9lBEBgC54IQn,I Still Believe,Mariah Carey


In [70]:
multiple_ids.shape

(470, 5)

In [71]:
multiple_ids.artist = multiple_ids.artist.apply(lambda x: x.lower())
multiple_ids.name = multiple_ids.name.apply(lambda x: x.lower())
multiple_ids = multiple_ids.rename(columns={"name": "Song", "artist": "Performer"})

In [95]:
outer_songs = multiple_songs_df.loc[:, ["id", "Song", "Performer", "SongID"]].merge(multiple_ids, "outer", on="Song").sort_values(["Song"])[lambda x: (x.id_y.isna()) | (x.id_x.isna())].reset_index(drop=True)

In [130]:
right_ids = outer_songs[lambda x: x.id_x.isna()]
left_ids = outer_songs[lambda x: x.id_y.isna()]
# equivalents = {0: 1, 2:3, 4:5, 6:7, 10:11, 12:13, 14:15, 16:17, 18:19, 20:21, 22:23, 24:25, 26:27, 28:2} # 8, 9

In [140]:
left_ids = left_ids.dropna(axis=1)
right_ids = right_ids.dropna(axis=1)

In [183]:
def cleanSong(song):
    regex_ext = re.compile(r"&+|feat\.|featuring|\s+and|,+|\sx\s+|with|\s-\s\w+|\(.+\)")
    regex_punc = r"[^\w\s]"
    string = re.sub(regex_ext, "", song)
    string = re.sub(regex_punc, "", string)
    return string.lower().strip()

In [175]:
left_ids = left_ids.assign(Song2 = lambda x: x.Song.apply(cleanSong))
right_ids = right_ids.assign(Song2 = lambda x: x.Song.apply(cleanSong))

In [180]:
last_remaining_songs = left_ids.merge(right_ids, "outer", on="Song2")[lambda x: (x.popularity.isna()) | (x.Song_x.isna())]

In [182]:
last_remaining_songs.head()

,id_x,Song_x,Performer_x,SongID,Song2,Song_y,explicit,popularity,id_y,Performer_y
2,2H7PHVdQ3mXqEHXcvclTB0,1999,prince,1999prince,1999,NaN,NaN,NaN,NaN,NaN
4,65Q2mv3UlVi9eO70OpsmSe,a holly jolly christmas,burl ives,a holly jolly christmasburl ives,a holly jolly christmas,NaN,NaN,NaN,NaN,NaN
5,1ADciGUxOaUbBFd0rpYx7d,a-yo,lady gaga,a-yolady gaga,ayo,NaN,NaN,NaN,NaN,NaN
14,2YegxR5As7BeQuVp2U6pek,be without you,mary j. blige,be without youmary j. blige,be out you,NaN,NaN,NaN,NaN,NaN
17,3NBuhfOPdYOquXXMt8dwJW,beware of the boys (mundian to bach ke),panjabi mc featuring jay-z,beware of the boys (mundian to bach ke)panjabi...,beware of the boys,NaN,NaN,NaN,NaN,NaN


In [184]:
lrs_dict = {
    2:131,
    4:158,
    5:159,
    14:132,
    17:150,
    20:133,
    22:135,
    24:136,
    34:137,
    35:138,
    37:139,
    39:142,
    40:140,
    43:141,
    48:143,
    51:144,
    58:145,
    61:146,
    64:147,
    69:134,
    73:148,
    74:149,
    82:151,
    85:152,
    94:153,
    104:154,
    122:155,
    123:156,
    128:157
}

In [185]:
for k,v in lrs_dict.items():
    last_remaining_songs.loc[k, "Song_y"] = last_remaining_songs.loc[v, "Song_y"]
    last_remaining_songs.loc[k, "popularity"] = last_remaining_songs.loc[v, "popularity"]
    last_remaining_songs.loc[k, "explicit"] = last_remaining_songs.loc[v, "explicit"]
    last_remaining_songs.loc[k, "id_y"] = last_remaining_songs.loc[v, "id_y"]
    last_remaining_songs.loc[k, "Performer_y"] = last_remaining_songs.loc[v, "Performer_y"]

In [191]:
lrs_df = last_remaining_songs.loc[:128, :]

In [195]:
lrs_df.shape

(30, 10)

In [220]:
lrs = pd.concat([left_ids.merge(right_ids, "inner", on="Song2"), lrs_df], axis=0, ignore_index=True)

In [229]:
_to_attr = multiple_songs_df.iloc[:, :-3].merge(multiple_ids, "inner", on=["Song"])
_to_attr2 = multiple_songs_df.iloc[:, :-3].merge(lrs.loc[:, ["id_x", "id_y", "explicit", "popularity", "Performer_y"]], "inner", left_on="id", right_on="id_x")

In [242]:
_to_attr = _to_attr.rename(columns={"Performer_x": "Performer", "id_x": "id"}).drop("Performer_y", axis=1).loc[:, ['index', 'Song', 'Performer', 'SongID', 'id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'timesignature',
       'duration', 'id_y', 'explicit', 'popularity']]

In [243]:
_to_attr2 = _to_attr2.drop(["Performer_y", "id_x"], axis=1)

In [244]:
_to_attr.columns == _to_attr2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [251]:
_to_attr_full = pd.concat([_to_attr, _to_attr2], axis=0, ignore_index=True)
_to_attr_full.explicit = _to_attr_full.explicit.astype("float")

In [252]:
# contains 473 Songs That Previously did not have explicit or popularity
_to_attr_full

,index,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,...,acousticness,instrumentalness,liveness,valence,tempo,timesignature,duration,id_y,explicit,popularity
0,14,hands,jewel,handsjewel,12iERgT55LwVLtGni5WuKB,0.644,0.495,5.0,-10.175,0.0,...,0.42300,0.00128,0.0820,0.321,127.221,4.0,235000.0,615auKcis6sGDrHk6UTeog,0.0,55
1,73,what it's like,everlast,what it's likeeverlast,0nbuX7OZkEY5agMkpr98wS,0.678,0.556,0.0,-7.423,1.0,...,0.09040,0.0334,0.0866,0.405,85.158,4.0,303133.0,6vUPPEN2V2lZ78b6DyEIjZ,1.0,54
2,122,i still believe,mariah carey,i still believemariah carey,1bUrdVhrxVzTuROudSTYVO,0.604,0.530,7.0,-7.597,1.0,...,0.21700,0.0,0.1250,0.176,117.941,4.0,234933.0,2feekrJqyB9lBEBgC54IQn,0.0,55
3,139,stay the same,joey mcintyre,stay the samejoey mcintyre,0P5rgEbdd5mk1afZcfLhwe,0.630,0.487,2.0,-7.906,1.0,...,0.02150,0.0,0.1220,0.346,130.028,4.0,227760.0,34Tq4vFc9aiShjelMTEfdO,0.0,42
4,191,down so long,jewel,down so longjewel,5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9.0,-8.611,0.0,...,0.30600,0.00659,0.1120,0.622,78.501,4.0,255146.0,47f5nT8JIRG43AH0hYbQCJ,0.0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,8074,cuban links,rod wave & kevin gates,cuban linksrod wave & kevin gates,4ovta8hB5ymzzsR5EaCEdx,0.693,0.652,5.0,-8.766,1.0,...,0.25000,0.0,0.0294,0.770,87.986,4.0,192182.0,1upQoQBckBvpwN4lewptFd,1.0,62
469,8089,best on earth,russ & bia,best on earthruss & bia,2Xo3rLHjXfdb4avN68aQyi,0.568,0.764,5.0,-3.465,0.0,...,0.21700,0.0,0.0931,0.625,179.985,4.0,160720.0,0xKBkLcF71uUQeTA7FSu4A,1.0,71
470,8139,what doesn't kill you (stronger),glee cast,what doesn't kill you (stronger)glee cast,379NxsaOdaBZQsAwfsb4QW,0.618,0.752,0,-4.828,1,...,0.00863,0.000175,0.1270,0.679,115.957,4,221027,7iMyX6FTRrY4fO8dHQhAUs,0.0,53
471,8180,here comes goodbye,rascal flatts,here comes goodbyerascal flatts,5YSW99IUAIiAj243jF7PBO,0.431,0.584,2,-6.025,1,...,0.42000,0,0.1420,0.181,146.176,4,243787,4lhajjgcICdfhRkvWj9Tud,0.0,52


In [255]:
attributes_final = attributes_new.merge(_to_attr_full.loc[:, ["id", "id_y", "explicit", "popularity"]], "left", on="id")
attributes_final.id_y = attributes_final.id_y.fillna(attributes_final.id)
attributes_final.explicit_y = attributes_final.explicit_y.fillna(attributes_final.explicit_x) 
attributes_final.popularity = attributes_final.popularity.fillna(attributes_final.popularity_y) 

In [261]:
attributes_final = attributes_final.drop(["popularity_x", "explicit_x", "popularity_y"], axis=1).rename(columns={"explicit_y": "explicit"})

In [268]:
ids_no_attrs = attributes_final[lambda x: ~(x.id.isnull()) & (x.key.isnull())].id.values

In [284]:
ids_no_attrs_df = attributes_final[lambda x: ~(x.id.isnull()) & (x.key.isnull())].loc[:, ["index","Song", "Performer","SongID",	"id", "id_y", "explicit", "popularity"]]

In [288]:
ids_no_attrs_df.head()

,index,Song,Performer,SongID,id,id_y,explicit,popularity
36,36,hard knock life (ghetto anthem),jay-z,hard knock life (ghetto anthem)jay-z,5Tl0HJvynZtKdSUMKbFVVX,5Tl0HJvynZtKdSUMKbFVVX,1.0,67.0
146,146,if you could read my mind,"stars on 54: ultra nate, amber, jocelyn enriquez",if you could read my mindstars on 54: ultra na...,7lQXHsKgBOdXQ51GSNYIf5,7lQXHsKgBOdXQ51GSNYIf5,NaN,NaN
422,422,do it again (put ya hands up),jay-z featuring beanie sigel & amil,do it again (put ya hands up)jay-z featuring b...,4EIzsbsVOzfLwyIvfsRQ8H,4EIzsbsVOzfLwyIvfsRQ8H,1.0,45.0
661,661,i just wanna love u (give it 2 me),jay-z,i just wanna love u (give it 2 me)jay-z,0s46SltT4On0Z6kglK1I5c,0s46SltT4On0Z6kglK1I5c,1.0,60.0
857,857,she's all i got,jimmy cozier,she's all i gotjimmy cozier,49XSbzWYyB20DTlBtQiZFw,49XSbzWYyB20DTlBtQiZFw,0.0,36.0


In [270]:
query_string = ",".join([x for x in ids_no_attrs])
response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={query_string}", headers=headers).json()

In [277]:
_intermediate = pd.DataFrame()
i = 0
for j in response["audio_features"]:
    if j != None:
        for k, v in j.items():
            _intermediate.loc[i, k] = v
        i += 1

In [292]:
ids_no_attrs_ = ids_no_attrs_df.merge(_intermediate, "left", on="id").rename(columns={"time_signature": "timesignature", "duration_ms": "duration"}).loc[:, ['index', 'Song', 'Performer', 'SongID', 'id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'timesignature',
       'duration', 'id_y', 'explicit', 'popularity']]

In [413]:
attributes_done = pd.concat([attributes_final[lambda x: ~(~(x.id.isnull()) & (x.key.isnull()))], ids_no_attrs_], axis=0, ignore_index=True).sort_values("index")

In [414]:
attributes_done.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8436 entries, 0 to 8420
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             8436 non-null   int64  
 1   Song              8436 non-null   object 
 2   Performer         8436 non-null   object 
 3   SongID            8436 non-null   object 
 4   id                8423 non-null   object 
 5   danceability      8422 non-null   float64
 6   energy            8422 non-null   float64
 7   key               8422 non-null   object 
 8   loudness          8422 non-null   float64
 9   mode              8422 non-null   object 
 10  speechiness       8422 non-null   float64
 11  acousticness      8422 non-null   float64
 12  instrumentalness  8422 non-null   object 
 13  liveness          8422 non-null   float64
 14  valence           8422 non-null   float64
 15  tempo             8422 non-null   float64
 16  timesignature     8362 non-null   object 


In [415]:
attributes_done = attributes_done.drop("index", axis=1).drop_duplicates()

In [411]:
attributes_done = attributes_done.drop_duplicates(["id_y"])

In [397]:
attributes_done.to_sql("attributes", con=conn, if_exists="replace")

In [439]:
calls = attributes_done.id_y.dropna().values

In [440]:
url = f"https://api.spotify.com/v1/recommendations?seed_tracks={calls[0]}&market=US"

In [443]:
response = requests.get(url, headers=headers)

In [468]:
response = response.json()

In [496]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Boolean, String, Float
from sqlalchemy.orm import Session

In [532]:
Base = declarative_base()
engine = sql.create_engine("sqlite:///../src/data/samples.db")
conn = engine.connect()

In [533]:
class Samples(Base):
    __tablename__ = "samples"
    pk = Column(Integer, primary_key=True)
    id = Column(String)
    popularity = Column(Integer)
    explicit = Column(Boolean)
    Song = Column(String)
    Performer = Column(String)
    id_fk = Column(String)

class Artists(Base):
    __tablename__ = "artists"
    pk = Column(Integer, primary_key=True)
    artist_id = Column(String)
    Performer = Column(String)
    release_date = Column(String)
    id_fk = Column(String)

In [534]:
Base.metadata.create_all(engine)

In [558]:
i = 7336
for c in calls[7336:]:
    session = Session(bind=engine)
    url = f"https://api.spotify.com/v1/recommendations?seed_tracks={c}&market=US"
    _response = requests.get(url, headers=headers)
    if _response.status_code == 200:
        _response = _response.json()
    elif _response.status_code == 429:
        print(f"Recieved Call Block Sleeping for 30 Secs @ {dt.datetime.now()}")
        time.sleep(35)
        _response = requests.get(url, headers=headers)
    else:
        print(f"{i}")
        break

    if _response["tracks"] != []:
        samples = random.sample(range(len(_response["tracks"])), 5)
        for j in samples:
            track_obj = _response["tracks"][j]
            db_entry = Samples(id=track_obj["id"], popularity=track_obj["popularity"], explicit=track_obj["explicit"], Song=track_obj["name"], Performer=track_obj["artists"][0]["name"], id_fk=c)  
            session.add(db_entry)
            for a in track_obj["artists"]:
                db_entry_a = Artists(artist_id= a["id"], Performer=a["name"], release_date=track_obj["album"]["release_date"], id_fk=track_obj["id"])
                session.add(db_entry_a)
    
    session.commit()
    session.close()
    print(f"Done with {i}. {len(calls) - i} Left. Finished at {dt.datetime.now()}")
    i += 1
    time.sleep(1)

Done with 7336. 955 Left. Finished at 2021-11-29 09:00:09.341195
Done with 7337. 954 Left. Finished at 2021-11-29 09:00:10.609339
Done with 7338. 953 Left. Finished at 2021-11-29 09:00:11.875069
Done with 7339. 952 Left. Finished at 2021-11-29 09:00:13.113552
Done with 7340. 951 Left. Finished at 2021-11-29 09:00:14.351635
Done with 7341. 950 Left. Finished at 2021-11-29 09:00:15.577623
Done with 7342. 949 Left. Finished at 2021-11-29 09:00:16.811192
Done with 7343. 948 Left. Finished at 2021-11-29 09:00:18.063273
Done with 7344. 947 Left. Finished at 2021-11-29 09:00:19.347927
Done with 7345. 946 Left. Finished at 2021-11-29 09:00:20.585857
Done with 7346. 945 Left. Finished at 2021-11-29 09:00:21.820864
Done with 7347. 944 Left. Finished at 2021-11-29 09:00:23.120579
Done with 7348. 943 Left. Finished at 2021-11-29 09:00:24.446888
Done with 7349. 942 Left. Finished at 2021-11-29 09:00:25.717965
Done with 7350. 941 Left. Finished at 2021-11-29 09:00:26.998582
Done with 7351. 940 Left.

In [562]:
samples_temp = pd.read_sql("SELECT * FROM samples", con=conn)

In [563]:
artists_samples_temp = pd.read_sql("SELECT * FROM artists", con=conn)

In [569]:
set(attributes_done.columns).difference(set(samples_temp.columns))

{'SongID',
 'acousticness',
 'danceability',
 'duration',
 'energy',
 'id_y',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'speechiness',
 'tempo',
 'timesignature',
 'valence'}

In [570]:
set(artists.columns).difference(set(artists_samples_temp.columns))

{'artist', 'followers', 'genre', 'index', 'popularity'}

In [580]:
__temp_df = pd.DataFrame(columns=["id", "id_y","danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "timesignature", "duration"])
i = 0
codes = samples_temp.id.unique()
for g in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[g:g+50]])
    res = requests.get(f"https://api.spotify.com/v1/audio-features?ids={id_string}", headers=headers)
    res = res.json()
    for _item in res["audio_features"]:
        if _item != None:
            __temp_df.loc[i, "id"] = _item["id"]
            __temp_df.loc[i, "id_y"] = codes[i]
            __temp_df.loc[i, "danceability"] = _item["danceability"]
            __temp_df.loc[i, "energy"] = _item["energy"]
            __temp_df.loc[i, "key"] = _item["key"]
            __temp_df.loc[i, "loudness"] = _item["loudness"]
            __temp_df.loc[i, "mode"] = _item["mode"]
            __temp_df.loc[i, "speechiness"] = _item["speechiness"]
            __temp_df.loc[i, "acousticness"] = _item["acousticness"]
            __temp_df.loc[i, "instrumentalness"] = _item["instrumentalness"]
            __temp_df.loc[i, "liveness"] = _item["liveness"]
            __temp_df.loc[i, "valence"] = _item["valence"]
            __temp_df.loc[i, "tempo"] = _item["tempo"]
            __temp_df.loc[i, "timesignature"] = _item["time_signature"]
            __temp_df.loc[i, "duration"] = _item["duration_ms"]
        i += 1
    print(f"done with {g+50}. {len(codes)-(g+50)} LEFT.....Finished at {dt.datetime.now()}.")
    time.sleep(1)

done with 50. 18287 LEFT.....Finished at 2021-11-29 10:01:29.292492.
done with 100. 18237 LEFT.....Finished at 2021-11-29 10:01:30.585079.
done with 150. 18187 LEFT.....Finished at 2021-11-29 10:01:31.849677.
done with 200. 18137 LEFT.....Finished at 2021-11-29 10:01:33.110470.
done with 250. 18087 LEFT.....Finished at 2021-11-29 10:01:34.371944.
done with 300. 18037 LEFT.....Finished at 2021-11-29 10:01:35.627924.
done with 350. 17987 LEFT.....Finished at 2021-11-29 10:01:36.884239.
done with 400. 17937 LEFT.....Finished at 2021-11-29 10:01:38.150526.
done with 450. 17887 LEFT.....Finished at 2021-11-29 10:01:39.513835.
done with 500. 17837 LEFT.....Finished at 2021-11-29 10:01:40.759118.
done with 550. 17787 LEFT.....Finished at 2021-11-29 10:01:42.048418.
done with 600. 17737 LEFT.....Finished at 2021-11-29 10:01:43.336711.
done with 650. 17687 LEFT.....Finished at 2021-11-29 10:01:44.576992.
done with 700. 17637 LEFT.....Finished at 2021-11-29 10:01:45.836277.
done with 750. 17587 

In [593]:
samples_temp_df = samples_temp.merge(__temp_df, "inner", on="id")

In [598]:
samples_temp_df = samples_temp_df.assign(SongID = lambda x: x.Song.apply(str.lower) + x.Performer.apply(str.lower))

In [599]:
artists_samples_temp

,pk,artist_id,Performer,release_date,id_fk
0,1,2iojnBLj0qIMiKPvVhLnsH,Trey Songz,2012-08-17,25CA8QVJQrh5R05UUCaODM
1,2,0f5nVCcR06GX8Qikz0COtT,Omarion,2006-12-26,5qhlC4Rzals34zxovrPzKp
2,3,7G6hXrjGpi6I7waNl4wxAk,J. Holiday,2007-01-01,6TlRNJaezOdzdECnQeRuMM
3,4,5IUvXYeMtVmp8YJWf0eqvf,INOJ,1999,2N5CvukjQqiWxjzZLMqqBN
4,5,3sMYEBy0CZFxedcnm9i9hf,Frankie J,2003,05wbsMDvcaD56O4SnESOaJ
...,...,...,...,...,...
59633,59634,6JL8zeS1NmiOftqZTRgdTz,Meghan Trainor,2020-01-31,5HqRQwSAPAgJBQ3bhtpUTg
59634,59635,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,2020-01-31,5HqRQwSAPAgJBQ3bhtpUTg
59635,59636,7qG3b048QCHVRO5Pv1T5lw,Enrique Iglesias,2021-09-17,4RzuSWA1mIaKXXKG9d9Nwi
59636,59637,0TnOYISbd1XYRBk9myaseg,Pitbull,2021-09-17,4RzuSWA1mIaKXXKG9d9Nwi


In [607]:
__artist_temp_df = pd.DataFrame(columns=["artist", "followers", "genres", "artist_id", "popularity"])
i = 0
codes = artists_samples_temp.artist_id.unique()
for n in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[n:n+50]])
    res = requests.get(f"https://api.spotify.com/v1/artists?ids={id_string}", headers=headers)
    res = res.json()
    for _art in res["artists"]:
        if _art != None:
            __artist_temp_df.loc[i, "followers"] = _art["followers"]["total"]
            __artist_temp_df.loc[i, "genre"] = str(_art["genres"])
            __artist_temp_df.loc[i, "artist_id"] = _art["id"]
            __artist_temp_df.loc[i, "popularity"] =_art["popularity"]
        i += 1
    print(f"done with {n+50}. {len(codes)-(n+50)} LEFT.....Finished at {dt.datetime.now()}.")
    time.sleep(1)

done with 50. 6876 LEFT.....Finished at 2021-11-29 10:24:10.902860.
done with 100. 6826 LEFT.....Finished at 2021-11-29 10:24:12.123642.
done with 150. 6776 LEFT.....Finished at 2021-11-29 10:24:13.329982.
done with 200. 6726 LEFT.....Finished at 2021-11-29 10:24:14.549542.
done with 250. 6676 LEFT.....Finished at 2021-11-29 10:24:15.762818.
done with 300. 6626 LEFT.....Finished at 2021-11-29 10:24:16.972582.
done with 350. 6576 LEFT.....Finished at 2021-11-29 10:24:18.183739.
done with 400. 6526 LEFT.....Finished at 2021-11-29 10:24:19.434673.
done with 450. 6476 LEFT.....Finished at 2021-11-29 10:24:20.639767.
done with 500. 6426 LEFT.....Finished at 2021-11-29 10:24:21.861213.
done with 550. 6376 LEFT.....Finished at 2021-11-29 10:24:23.078489.
done with 600. 6326 LEFT.....Finished at 2021-11-29 10:24:24.293368.
done with 650. 6276 LEFT.....Finished at 2021-11-29 10:24:25.522172.
done with 700. 6226 LEFT.....Finished at 2021-11-29 10:24:26.735292.
done with 750. 6176 LEFT.....Finish

In [614]:
artists_temp_df = artists_samples_temp.merge(__artist_temp_df.drop(["artist", "genres"], axis=1), "inner", on="artist_id")

In [613]:
artists

,index,artist,id_fk,artist_id,followers,popularity,genre
0,0,R. Kelly,0QWFiyhOqFu9RP8qzP6z5L,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
1,36,R. Kelly,015A6ZcNAWSSuwY7iDcD1b,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
2,72,R. Kelly,4UDQEazTqhCICqssWtMZEO,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
3,108,R. Kelly,19XWdM9zSb4IIiFCXO1SlE,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
4,144,R. Kelly,7tNU9Ax7hzHA4nzo57aZb3,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
...,...,...,...,...,...,...,...
11452,337891,Markéta Irglová,3ksZVeqlLDmEPG71vXQdT6,4SklVMGMsWTq7cJd9MixUx,39237,51,"['chamber folk', 'czech singer-songwriter', 'm..."
11453,337892,Mr Hudson,35srNkJVToAzYiZzron1FY,1zV9UjTUevjp5VUddqIUUn,23130,58,[]
11454,337893,Anna Kendrick,6mH3qVIeOsnQIAho5eWwhH,6xfqnpe2HnLVUaYXs2F8YS,298865,65,"['hollywood', 'movie tunes']"
11455,337894,Nashville All Star Combo,4iiJ57RTsBw4hLZi3sCXBS,66J74CmVWbuqmYRGfQ5nJw,144,10,[]


In [616]:
artists_temp_df = artists_temp_df.rename(columns={"Performer": "artist", "pk": "index"}).loc[:, ["index", "artist", "id_fk", "artist_id", "followers", "popularity", "genre", "release_date"]]

In [621]:
__release_date = pd.DataFrame()
i = 0
codes = attributes_done.id_y.dropna().unique()
for n in range(0, len(codes), 50):
    id_string = ",".join([x for x in codes[n:n+50]])
    res = requests.get(f"https://api.spotify.com/v1/tracks?market=US&ids={id_string}", headers=headers)
    res = res.json()
    for _art in res["tracks"]:
        if _art != None:
            __release_date.loc[i, "release_date"] = _art["album"]["release_date"]
            __release_date.loc[i, "id_y"] = _art["id"]
        i += 1
    print(f"done with {n+50}. {len(codes)-(n+50)} LEFT.....Finished at {dt.datetime.now()}.")
    time.sleep(1)

done with 50. 8228 LEFT.....Finished at 2021-11-29 10:45:13.880871.
done with 100. 8178 LEFT.....Finished at 2021-11-29 10:45:15.110150.
done with 150. 8128 LEFT.....Finished at 2021-11-29 10:45:16.349578.
done with 200. 8078 LEFT.....Finished at 2021-11-29 10:45:17.603981.
done with 250. 8028 LEFT.....Finished at 2021-11-29 10:45:18.827826.
done with 300. 7978 LEFT.....Finished at 2021-11-29 10:45:20.090188.
done with 350. 7928 LEFT.....Finished at 2021-11-29 10:45:21.310205.
done with 400. 7878 LEFT.....Finished at 2021-11-29 10:45:22.546796.
done with 450. 7828 LEFT.....Finished at 2021-11-29 10:45:23.827413.
done with 500. 7778 LEFT.....Finished at 2021-11-29 10:45:25.055366.
done with 550. 7728 LEFT.....Finished at 2021-11-29 10:45:26.318762.
done with 600. 7678 LEFT.....Finished at 2021-11-29 10:45:27.588001.
done with 650. 7628 LEFT.....Finished at 2021-11-29 10:45:28.829318.
done with 700. 7578 LEFT.....Finished at 2021-11-29 10:45:30.089958.
done with 750. 7528 LEFT.....Finish

In [627]:
attributes_done = attributes_done.merge(__release_date, "left", on="id_y")

In [631]:
attributes_done.columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date'],
      dtype='object')

In [632]:
samples_temp_df[lambda x: x.id != x.id_y]

,pk,id,popularity,explicit,Song,Performer,id_fk,id_y,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,timesignature,duration,SongID


In [661]:
samples_with_release_date = samples_temp_df.merge(artists_temp_df.loc[:, ["id_fk", "release_date"]].rename(columns={"id_fk": "id"}), "left", on="id")

In [637]:
attributes_done.columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date'],
      dtype='object')

In [670]:
set(samples_with_release_date.columns).difference(set(attributes_done.columns))

{'id_fk'}

In [669]:
samples_with_release_date = samples_with_release_date.loc[:, ['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
'explicit', 'popularity', 'release_date', "id_fk"]]

In [ ]:
# We will have an intermediary table that with a one to many relationship to reduce or recommendations number. These are for tracks that were recommended for different popular tracks

In [705]:
# many to one relationship of billboard songs to their recommendations
samples_with_release_date.loc[:, ["id", "id_fk"]].drop_duplicates().rename(columns={"id_fk": "billboard_id", "id": "recommendation_id"})

,recommendation_id,billboard_id
0,25CA8QVJQrh5R05UUCaODM,0QWFiyhOqFu9RP8qzP6z5L
12,25CA8QVJQrh5R05UUCaODM,1Dx8rIZaXzZPvUQRF2j9hB
24,25CA8QVJQrh5R05UUCaODM,7Gfzoer3tsu6AVTce0TwSM
36,25CA8QVJQrh5R05UUCaODM,7mwLsFXNBWbtFzA032tUbx
48,25CA8QVJQrh5R05UUCaODM,2I1rr3kX0iMyorP5HOBRbL
...,...,...
287294,6jmZlyf9DxcIoRrjw02YXm,1lOe9qE0vR9zwWQAOk6CoO
287298,0qg0GJp7HSewRCVPP4fKGk,1lOe9qE0vR9zwWQAOk6CoO
287300,3LK6wDkhbTyg8GhJidU9oC,2wrJq5XKLnmhRXHIAf9xBa
287301,4guQztGKAETkvuTekNeqBe,2wrJq5XKLnmhRXHIAf9xBa


In [706]:
samples_final = samples_with_release_date.drop("id_fk", axis=1).drop_duplicates().reset_index(drop=True)

In [708]:
samples_final.head()

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,timesignature,duration,id_y,explicit,popularity,release_date
0,Heart Attack,Trey Songz,heart attacktrey songz,25CA8QVJQrh5R05UUCaODM,0.608,0.863,11,-4.87,1,0.131,...,0,0.228,0.476,150.029,4,233147,25CA8QVJQrh5R05UUCaODM,1,57,2012-08-17
1,Ice Box,Omarion,ice boxomarion,5qhlC4Rzals34zxovrPzKp,0.775,0.731,8,-5.446,1,0.134,...,0,0.129,0.821,131.105,4,256427,5qhlC4Rzals34zxovrPzKp,0,59,2006-12-26
2,Bed,J. Holiday,bedj. holiday,6TlRNJaezOdzdECnQeRuMM,0.684,0.606,5,-7.268,0,0.0504,...,0,0.058,0.723,127.901,4,275107,6TlRNJaezOdzdECnQeRuMM,0,66,2007-01-01
3,Time After Time,INOJ,time after timeinoj,2N5CvukjQqiWxjzZLMqqBN,0.776,0.563,0,-8.84,1,0.0324,...,0.00611,0.0681,0.395,131.728,4,255673,2N5CvukjQqiWxjzZLMqqBN,0,45,1999
4,How To Deal,Frankie J,how to dealfrankie j,05wbsMDvcaD56O4SnESOaJ,0.749,0.495,2,-8.456,1,0.0332,...,0,0.201,0.645,126.12,4,230507,05wbsMDvcaD56O4SnESOaJ,0,49,2003


In [710]:
chart_id_set = set(attributes_done.id_y.dropna().unique())

In [712]:
# remove recommendations for songs that charted
noncharting_samples = samples_final[lambda x: ~(x.id_y.isin(chart_id_set))]

In [715]:
noncharting_samples.columns == attributes_done.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [716]:
noncharting_samples = noncharting_samples.assign(chart=0)
attributes_done = attributes_done.assign(chart=1)

In [718]:
# table with all attributes from samples (chart=0) and from billboard songs (chart=1)
attributes_total = pd.concat([attributes_done, noncharting_samples], axis=0, ignore_index=True)

In [719]:
# many to one relating recommended songs (chart=0) to their billboard_seed (chart=0)  
recommendation_join = samples_with_release_date.loc[:, ["id", "id_fk"]].drop_duplicates().rename(columns={"id_fk": "billboard_id", "id": "recommendation_id"})

In [720]:
# Combine artists and artists_temp_df sans release date (now in tracks)
artists.head()

,index,artist,id_fk,artist_id,followers,popularity,genre
0,0,R. Kelly,0QWFiyhOqFu9RP8qzP6z5L,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
1,36,R. Kelly,015A6ZcNAWSSuwY7iDcD1b,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
2,72,R. Kelly,4UDQEazTqhCICqssWtMZEO,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
3,108,R. Kelly,19XWdM9zSb4IIiFCXO1SlE,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
4,144,R. Kelly,7tNU9Ax7hzHA4nzo57aZb3,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]


In [684]:
artists_temp_df = artists_temp_df.drop("release_date", axis=1)

In [721]:
artists_final_really = pd.concat([artists, artists_temp_df], axis=0, ignore_index=True)

In [722]:
# many to one relationship of billboard songs to their assocaited artist
artists_join = artists_final_really.loc[:, ["id_fk", "artist_id"]].drop_duplicates().rename(columns={"id_fk": "song_id"}).reset_index(drop=True)

In [724]:
# final artists with artists from billboard chart and these songs recommendations
artists_total = artists_final_really.drop(["id_fk", "index"], axis=1).drop_duplicates().reset_index(drop=True)

In [732]:
# Creating Backups for samples So I Don't Have to Make Intensive API calls again
samples_final.to_sql("samples", con=conn, if_exists="replace")
artists_temp_df.to_sql("artists", con=conn, if_exists="replace")
conn.close()

In [735]:
attributes_total.Song = attributes_total.Song.apply(str.lower)
attributes_total.Performer = attributes_total.Performer.apply(str.lower)

In [737]:
engine = sql.create_engine("sqlite:///../src/data/music.db")
conn = engine.connect()

In [740]:
attributes_total.to_sql("attributes", con=conn, if_exists="replace")
recommendation_join.to_sql("recommendation_join", con=conn)
artists_total.to_sql("artists", con=conn, if_exists="replace")
artists_join.to_sql("artists_join", con=conn)

In [741]:
conn.close()

In [756]:
charters_with_genre = attributes_done.merge(artists.loc[:, ["id_fk", "genre"]], "left", left_on="id_y", right_on="id_fk")

In [763]:
noncharters_with_genre = noncharting_samples.merge(artists_temp_df.loc[:, ["id_fk", "genre"]], "left", left_on="id_y", right_on="id_fk")

In [767]:
samples_with_genres = samples_final.merge(artists_temp_df.loc[:, ["id_fk", "genre"]], "left", left_on="id_y", right_on="id_fk")

In [768]:
engine = sql.create_engine("sqlite:///../src/data/samples.db")
conn = engine.connect()

In [771]:
samples_with_genres.to_sql("samples", con=conn, if_exists="replace")
conn.close()

In [1282]:
engine = sql.create_engine("sqlite:///../src/data/music.db")
conn = engine.connect()

In [772]:
charters_with_genre.columns == noncharters_with_genre.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [775]:
attributes_with_genre = pd.concat([charters_with_genre, noncharters_with_genre], axis=0, ignore_index=True)

In [793]:
charters_with_genre.id_y.unique().shape

(8279,)

In [794]:
noncharters_with_genre.id_y.unique().shape

(13921,)

In [777]:
attributes_with_genre.Song = attributes_with_genre.Song.apply(str.lower)
attributes_with_genre.Performer = attributes_with_genre.Performer.apply(str.lower) 

In [786]:
attributes_with_genre = attributes_with_genre.drop_duplicates(["SongID", "id_y"]).reset_index(drop=True)

In [789]:
attributes_with_genre.to_sql("attributes", con=conn, if_exists="replace")

In [781]:
conn.close()

In [1025]:
searched_songs = set(need_songs_df.SongID.unique()) 

,artist,song_id
0,R. Kelly,0QWFiyhOqFu9RP8qzP6z5L
1,R. Kelly,015A6ZcNAWSSuwY7iDcD1b
2,R. Kelly,4UDQEazTqhCICqssWtMZEO
3,R. Kelly,19XWdM9zSb4IIiFCXO1SlE
4,R. Kelly,7tNU9Ax7hzHA4nzo57aZb3
...,...,...
53671,Bantu,4Uy3kNxW2kB8AEoXljEcth
53672,Ari PenSmith,6Ic2MP2hzoBLe3r1VIle8J
53673,ATR Son Son,0qg0GJp7HSewRCVPP4fKGk
53674,Teddy Swims,4guQztGKAETkvuTekNeqBe


In [1034]:
attributes_with_genre[attributes_with_genre.SongID.isin(searched_songs)].merge(artists_total.merge(artists_join, "inner", on="artist_id").loc[:, ["artist", "song_id"]], "left", left_on="id_y", right_on="song_id").to_csv("../data/interim/searched.csv", index=False)

In [1044]:
bad_searched_ids = pd.read_csv("../data/interim/new_attr.csv")

In [1045]:
bad_ids_set = bad_searched_ids.song_id.unique()

In [1048]:
bad_searched_ids = bad_searched_ids.loc[:, ["Song", "Performer", "SongID", "id", "new_id"]]

In [1050]:
attributes_with_genre[attributes_with_genre.id_y.isin(bad_ids_set)].columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre'],
      dtype='object')

In [ ]:
# attributes = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "timesignature", "duration"]
# track_specific = ["explicit", "popularity"]
# artist_specific = ["genre"]
# album_specific = ["release_date"]


In [1055]:
ids = bad_searched_ids.new_id.dropna().unique()
id_string = ",".join([x for x in ids])
tracks_specific = requests.get(f"https://api.spotify.com/v1/tracks?market=US&ids={id_string}", headers=headers).json() # popularity, explicit, artist, release_date
tracks_features = requests.get(f"https://api.spotify.com/v1/audio-features?ids={id_string}", headers=headers).json() # feature

In [1086]:
bad_artists = pd.DataFrame()
bad_tracks = pd.DataFrame()
_temp_feat = pd.DataFrame()
j = 0
i = 0
s = 0
for t in tracks_specific["tracks"]:
    bad_tracks.loc[j, "release_date"] = t["album"]["release_date"]
    bad_tracks.loc[j, "id"] = t["id"]
    bad_tracks.loc[j, "popularity"] = t["popularity"]
    bad_tracks.loc[j, "explicit"] = t["explicit"]
    bad_tracks.loc[j, "Song"] = t["name"]
    bad_tracks.loc[j, "Performer"] = t["artists"][0]["name"]
    for _art in t["artists"]:
        bad_artists.loc[i, "artist"] = _art["name"]
        bad_artists.loc[i, "artist_id"] = _art["id"]
        bad_artists.loc[i, "id_fk"] = t["id"]
        i += 1
    j+= 1
    
for t in tracks_features["audio_features"]:
    _temp_feat.loc[s, "danceability"] = t["danceability"]
    _temp_feat.loc[s, "energy"] = t["energy"]
    _temp_feat.loc[s, "key"] = t["key"]
    _temp_feat.loc[s, "loudness"] = t["loudness"]
    _temp_feat.loc[s, "mode"] = t["mode"]
    _temp_feat.loc[s, "speechiness"] = t["speechiness"]
    _temp_feat.loc[s, "acousticness"] = t["acousticness"]
    _temp_feat.loc[s, "instrumentalness"] = t["instrumentalness"]
    _temp_feat.loc[s, "liveness"] = t["liveness"]
    _temp_feat.loc[s, "valence"] = t["valence"]
    _temp_feat.loc[s, "tempo"] = t["tempo"]
    _temp_feat.loc[s, "duration"] = t["duration_ms"]
    _temp_feat.loc[s, "timesignature"] = t["time_signature"]
    _temp_feat.loc[s, "id"] = t["id"]
    s += 1

In [1087]:
tracks_to_add = bad_tracks.merge(_temp_feat, "inner", on="id")

In [1074]:
ids = bad_artists.artist_id.unique()
id_string = ",".join([x for x in ids])
artists_specific = requests.get(f"https://api.spotify.com/v1/artists?ids={id_string}", headers=headers).json() 

In [1088]:
_art_ = pd.DataFrame()
i = 0
for _art in artists_specific["artists"]:
    _art_.loc[i, "followers"] = _art["followers"]["total"] # followers.total
    _art_.loc[i, "genre"] = str(_art["genres"]) # followers.genres
    _art_.loc[i, "popularity"] = _art["popularity"] # followers.popularity
    _art_.loc[i, "artist_id"] = _art["id"]
    i += 1

bad_artists = bad_artists.merge(_art_, "inner", on="artist_id")

In [1091]:
artists_total

,artist,artist_id,followers,popularity,genre
0,R. Kelly,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
1,Céline Dion,4S9EykWXhStSc15wEx8QFK,5445654,77,"['canadian pop', 'dance pop']"
2,Deborah Cox,601893mmW5hl1FBOykWZHG,589295,52,"['canadian pop', 'hip pop', 'neo soul', 'new j..."
3,Divine,71L2a0yZtQZ4yrITx73YOW,1003,13,['fijian pop']
4,Britney Spears,26dSoYclwsYLMAKD3tpOr4,10012615,82,"['dance pop', 'pop', 'post-teen pop']"
...,...,...,...,...,...
9428,Mason Ramsey,6xvpfMjWTougrRRtK7iikz,104332,48,"['contemporary country', 'country']"
9429,Rod Wave,45TgXXqMDdF8BkjA83OM7z,2539817,79,['florida rap']
9430,ATR Son Son,1nGRqM8VNcQOcFsOnhpInD,638,53,[]
9431,Teddy Swims,33qOK5uJ8AR2xuQQAhHump,299986,60,[]


In [1089]:
bad_artists

,artist,artist_id,id_fk,followers,genre,popularity
0,Ruff Ryders,2q0HDEYPYC2oZjXgYqAWMZ,0wQqyhLetDqk3waR7JSttl,2987.0,"['hardcore hip hop', 'hip pop', 'rap']",45.0
1,Eve,4d3yvTptO48nOYTPBcPFZC,0wQqyhLetDqk3waR7JSttl,971338.0,"['dance pop', 'gangster rap', 'hip hop', 'hip ...",69.0
2,Jadakiss,5pnbUBPifNnlusY8kTBivi,0wQqyhLetDqk3waR7JSttl,837537.0,"['battle rap', 'east coast hip hop', 'gangster...",70.0
3,LIL LO,17SUtl275ibg2bA3c31cVj,0BKlETtyw17Zz0d76Yfxs3,6784.0,['dmv rap'],38.0
4,Dubble Trubble,6RnUwB5FnD60Cfd2SlWnyi,6Tydun5H6T2F3ZpE6aMWk6,929.0,[],26.0
5,Charli Baltimore,6xzdpjZ5OURzMdpjCFW008,21iEEekDSvQjVARSQ4dpXY,52173.0,['hip pop'],37.0
6,Ja Rule,1J2VVASYAamtQ3Bt8wGgA6,21iEEekDSvQjVARSQ4dpXY,1326212.0,"['dance pop', 'east coast hip hop', 'gangster ...",71.0
7,Ashanti,5rkVyNGXEgeUqKkB5ccK83,21iEEekDSvQjVARSQ4dpXY,2252059.0,"['dance pop', 'hip hop', 'hip pop', 'r&b', 'ur...",72.0
8,Vita,3ot6fEzKvRrCnWy5BWlnxU,21iEEekDSvQjVARSQ4dpXY,740.0,[],54.0
9,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,4j08A1efdJcVINBHSmQKPS,6399967.0,"['emo', 'pop punk', 'rock']",79.0


In [1106]:
bad_tracks = bad_tracks.merge(_temp_feat, "inner", on="id")

In [1112]:
last_tracks = bad_tracks.merge(bad_artists.loc[:, ["id_fk", "genre"]], "left", left_on="id", right_on="id_fk").drop_duplicates(["Song", "id"], keep="first")

In [1100]:
a_w_g = attributes_with_genre[~(attributes_with_genre.id_y.isin(bad_ids_set))]

In [1105]:
a_join = artists_join[~(artists_join.song_id.isin(bad_ids_set))]

In [1113]:
artists_total

,artist,artist_id,followers,popularity,genre
0,R. Kelly,2mxe0TnaNL039ysAj51xPQ,2370657,69,[]
1,Céline Dion,4S9EykWXhStSc15wEx8QFK,5445654,77,"['canadian pop', 'dance pop']"
2,Deborah Cox,601893mmW5hl1FBOykWZHG,589295,52,"['canadian pop', 'hip pop', 'neo soul', 'new j..."
3,Divine,71L2a0yZtQZ4yrITx73YOW,1003,13,['fijian pop']
4,Britney Spears,26dSoYclwsYLMAKD3tpOr4,10012615,82,"['dance pop', 'pop', 'post-teen pop']"
...,...,...,...,...,...
9428,Mason Ramsey,6xvpfMjWTougrRRtK7iikz,104332,48,"['contemporary country', 'country']"
9429,Rod Wave,45TgXXqMDdF8BkjA83OM7z,2539817,79,['florida rap']
9430,ATR Son Son,1nGRqM8VNcQOcFsOnhpInD,638,53,[]
9431,Teddy Swims,33qOK5uJ8AR2xuQQAhHump,299986,60,[]


In [1115]:
conn.close()

In [1116]:
engine = sql.create_engine("sqlite:///../src/data/samples.db")
conn = engine.connect()

In [119]:
def returnEntries(ids):
    art_df = pd.DataFrame()
    song_df = pd.DataFrame()
    i = 0
    k = 0 
    for id in ids:
        url = f"https://api.spotify.com/v1/recommendations?seed_tracks={id}&market=US"
        _response = requests.get(url, headers=headers).json()
        track_obj = _response["tracks"]
        for j in range(5):
            song_df.loc[i, "popularity"] = track_obj[j]["popularity"]
            song_df.loc[i, "explicit"] = track_obj[j]["explicit"]
            song_df.loc[i, "release_date"] = track_obj[j]["album"]["release_date"]
            song_df.loc[i, "id"] = track_obj[j]["id"]
            song_df.loc[i, "Song"] = track_obj[j]["name"]
            song_df.loc[i, "Performer"] = track_obj[j]["artists"][0]["name"]
            for _art in track_obj[j]["artists"]:
                art_df.loc[k, "artist"] = _art["name"]
                art_df.loc[k, "artist_id"] = _art["id"]
                artist_specific = requests.get(f"https://api.spotify.com/v1/artists/{_art['id']}", headers=headers).json() 
                art_df.loc[k, "followers"] = artist_specific["followers"]["total"] # followers.total
                art_df.loc[k, "genre"] = str(artist_specific["genres"]) # followers.genres
                art_df.loc[k, "popularity"] = artist_specific["popularity"] # followers.popularity
                art_df.loc[k, "artist_id"] = artist_specific["id"]
                art_df.loc[k, "id_fk"] = track_obj[j]["id"]
                k += 1
            feature = requests.get(f"https://api.spotify.com/v1/audio-features/{track_obj[j]['id']}", headers=headers).json()
            song_df.loc[i, "danceability"] = feature["danceability"]
            song_df.loc[i, "energy"] = feature["energy"]
            song_df.loc[i, "key"] = feature["key"]
            song_df.loc[i, "loudness"] = feature["loudness"]
            song_df.loc[i, "mode"] = feature["mode"]
            song_df.loc[i, "speechiness"] = feature["speechiness"]
            song_df.loc[i, "acousticness"] = feature["acousticness"]
            song_df.loc[i, "instrumentalness"] = feature["instrumentalness"]
            song_df.loc[i, "liveness"] = feature["liveness"]
            song_df.loc[i, "valence"] = feature["valence"]
            song_df.loc[i, "tempo"] = feature["tempo"]
            song_df.loc[i, "timesignature"] = feature["time_signature"]
            song_df.loc[i, "duration"] = feature["duration_ms"]
            song_df.loc[i, "id_fk"] = id
            i += 1
        print(f"finished with {id}. {len(ids)} LEFT.")

    return [art_df, song_df]


In [1199]:
recomend_art_last, recommend_song_last = returnEntries(last_tracks.id.values)

finished with 0wQqyhLetDqk3waR7JSttl. 15 LEFT.
finished with 0BKlETtyw17Zz0d76Yfxs3. 15 LEFT.
finished with 6Tydun5H6T2F3ZpE6aMWk6. 15 LEFT.
finished with 21iEEekDSvQjVARSQ4dpXY. 15 LEFT.
finished with 4j08A1efdJcVINBHSmQKPS. 15 LEFT.
finished with 0USctUIMP2DEp0oeBSyqJf. 15 LEFT.
finished with 5AJrhrwz4oSZX2PwwV4qrN. 15 LEFT.
finished with 1dZxI6OuzwJ283y3gOU1Kq. 15 LEFT.
finished with 5dANgSy7v091dhiPnEXNrf. 15 LEFT.
finished with 3KysI5xBjESoRGwwdhp2Vl. 15 LEFT.
finished with 5pPi94jT2q88tkjDhBu6IV. 15 LEFT.
finished with 2jKrwn5GUuLUmO7dNUMc5c. 15 LEFT.
finished with 4IeUAegXVbETb7FpgNDnSM. 15 LEFT.
finished with 6Az2Ll2CPnWYQ2BgiHf4Di. 15 LEFT.
finished with 4PpuH4mxL0rD35mOWaLoKS. 15 LEFT.


In [1119]:
a_w_g.columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre'],
      dtype='object')

In [1201]:
recommendations = recommend_song_last.merge(recomend_art_last.loc[:, ["genre", "id_fk"]], "left", left_on="id", right_on="id_fk").drop_duplicates(["Song", "id"])

In [1202]:
r_j = recommendation_join[~(recommendation_join.billboard_id.isin(bad_ids_set))]

In [1203]:
recommendations = recommendations.assign(chart=0).assign(SongID = lambda x: x.Song.apply(str.lower)+x.Performer.apply(str.lower)).assign(id_y = lambda x: x.id)
last_tracks = last_tracks.assign(chart=1).assign(SongID = lambda x: x.Song.apply(str.lower)+x.Performer.apply(str.lower)).assign(id_y = lambda x: x.id)

In [1204]:
a_j = artists_join[~(artists_join.song_id.isin(bad_ids_set))]

In [1210]:
recommendations = recommendations.drop(["id_fk_y"], axis=1).rename(columns={"id_fk_x": "id_fk"})

In [1211]:
recommendations = recommendations.loc[:, ['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre']]

In [1212]:
last_tracks = last_tracks.loc[:, ['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre']]

In [1213]:
to_awg = pd.concat([recommendations, last_tracks], axis=0, ignore_index=True)

In [1214]:
to_awg.explicit =to_awg.explicit.astype("float")

In [ ]:
# r_j many recommendations to one id 
# a_j many artists to one song 

In [1236]:
recomend_art_last

,artist,artist_id,followers,genre,popularity,id_fk
0,Nicky Jam,1SupJlEpv7RS2tPNRaHViT,15622318.0,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",85.0,4VgYtXCVJ7IbWAZ5ryfvEQ
1,Daddy Yankee,4VMYDCV2IEDYJArk749S6m,25028043.0,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",88.0,4VgYtXCVJ7IbWAZ5ryfvEQ
2,Alesso,4AVFqumd2ogHFlRbKIjp1t,3718029.0,"['dance pop', 'edm', 'electro house', 'pop', '...",77.0,2JoIjuzRplpYGvvLpSW2on
3,TINI,7vXDAI8JwjW531ouMGbfcp,3423279.0,"['latin pop', 'pop argentino']",81.0,2JoIjuzRplpYGvvLpSW2on
4,Black Eyed Peas,1yxSLGMDHlW21z4YXirZDS,6238354.0,"['dance pop', 'pop', 'pop rap']",82.0,04YutdH0j1INpsva3gOipt
...,...,...,...,...,...,...
157,Don Diablo,1l2ekx5skC4gJH8djERwh1,1766111.0,"['dance pop', 'edm', 'electro house', 'future ...",72.0,0JAttbSYaro6UZKb3TiAlK
158,Brando,5uEeqYFuIChoWKy34jp8xE,15350.0,['dance pop'],67.0,0JAttbSYaro6UZKb3TiAlK
159,Sofía Reyes,0haZhu4fFKt0Ag94kZDiz2,2146185.0,"['dance pop', 'latin', 'latin pop', 'mexican p...",72.0,6JqpFboOH2bq9pzaYbkKBn
160,Rita Ora,5CCwRZC6euC8Odo6y9X8jr,7052686.0,"['australian pop', 'dance pop', 'edm', 'pop', ...",77.0,6JqpFboOH2bq9pzaYbkKBn


In [1243]:
bad_artists_last = bad_artists.loc[:, ['artist', 'artist_id', 'followers', 'popularity', 'genre']]
recommend_art_last = recomend_art_last.loc[:,  ['artist', 'artist_id', 'followers', 'popularity', 'genre']]

In [1244]:
to_artists_total = pd.concat([bad_artists_last, recommend_art_last], axis=0, ignore_index=True)

In [1220]:
to_rj = recommendations.loc[:, ["id", "id_fk"]].rename(columns={"id": "recommendation_id", "id_fk": "billboard_id"})

In [1276]:
to_aj = to_awg.loc[:, ["Performer", "id"]].merge(artists_perfect.loc[:, ["artist", "artist_id"]],"left", left_on="Performer", right_on="artist").loc[:, ["id", "artist_id"]].drop_duplicates().rename(columns={"id": "song_id"})

In [1279]:
artists_join_perfect = pd.concat([a_j, to_aj], axis=0, ignore_index=True)

In [1280]:
recommendations_join_perfect = pd.concat([r_j, to_rj], axis=0, ignore_index=True)

In [1247]:
artists_perfect = pd.concat([artists_total, to_artists_total], axis=0, ignore_index=True)

In [1281]:
attributes_perfect = pd.concat([a_w_g, to_awg], axis=0, ignore_index=True)

In [1283]:
attributes_perfect.to_sql("attributes", con=conn, if_exists="replace")
recommendations_join_perfect.to_sql("recommendation_join", con=conn, if_exists="replace")
artists_perfect.to_sql("artists", con=conn, if_exists="replace")
artists_join_perfect.to_sql("artists_join", con=conn, if_exists="replace")

In [1284]:
conn.close()

In [1289]:
billboard[lambda x: x.id.isin(bad_ids_set)]

,index,Song,Performer,Week Position,WeekID,Previous Week Position,Peak Position,Weeks on Chart,Instance,SongID,Week_No,Performer_y,Song_y,id
8089,8089,got it all,eve & jadakiss,89,2000-07-15,NaN,89.0,1.0,1.0,got it alleve & jadakiss,28,Even All Out,Got It Figured Out,6oL0bOaqBsyM5Q9Rnp7DaH
8188,8188,got it all,eve & jadakiss,88,2000-07-22,89.0,88.0,2.0,1.0,got it alleve & jadakiss,29,Even All Out,Got It Figured Out,6oL0bOaqBsyM5Q9Rnp7DaH
8288,8288,got it all,eve & jadakiss,88,2000-07-29,88.0,88.0,3.0,1.0,got it alleve & jadakiss,30,Even All Out,Got It Figured Out,6oL0bOaqBsyM5Q9Rnp7DaH
8391,8391,got it all,eve & jadakiss,91,2000-08-05,88.0,88.0,4.0,1.0,got it alleve & jadakiss,31,Even All Out,Got It Figured Out,6oL0bOaqBsyM5Q9Rnp7DaH
8495,8495,got it all,eve & jadakiss,95,2000-08-12,91.0,88.0,5.0,1.0,got it alleve & jadakiss,32,Even All Out,Got It Figured Out,6oL0bOaqBsyM5Q9Rnp7DaH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100313,100313,lemon,n*e*r*d & rihanna,56,2018-03-17,51.0,40.0,17.0,1.0,lemonnerd & rihanna,10,David Rose and His Orchestra,Serenade to a Lemonade,4nMb8pMRQ5d5pTDv0z4dqH
100418,100418,lemon,n*e*r*d & rihanna,61,2018-03-24,56.0,40.0,18.0,1.0,lemonnerd & rihanna,11,David Rose and His Orchestra,Serenade to a Lemonade,4nMb8pMRQ5d5pTDv0z4dqH
100497,100497,lemon,n*e*r*d & rihanna,40,2018-03-31,61.0,40.0,19.0,1.0,lemonnerd & rihanna,12,David Rose and His Orchestra,Serenade to a Lemonade,4nMb8pMRQ5d5pTDv0z4dqH
100593,100593,lemon,n*e*r*d & rihanna,36,2018-04-07,40.0,36.0,20.0,1.0,lemonnerd & rihanna,13,David Rose and His Orchestra,Serenade to a Lemonade,4nMb8pMRQ5d5pTDv0z4dqH


In [96]:
response = requests.post(r'https://accounts.spotify.com/api/token', {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials",
}).json()
token = response["access_token"]
headers =  {"Authorization": f"Bearer {token}"}
print(f"{dt.datetime.now()} Acquired. Fails At {dt.datetime.now() + dt.timedelta(0, 3600)}")

2021-12-06 14:12:48.116570 Acquired. Fails At 2021-12-06 15:12:48.116570


In [1512]:
def genrify(series):
    genres = {}
    array = series.values
    for a in array:
        if type(a) is not str:
            a = str(a)
        a = a.replace("[", "").replace("]", "")
        a = a.split(",")
        for genre in a:
            genre = genre.replace("'", "").replace("'", "").strip()
            if genre not in genres.keys():
                genres[genre] = 1
            else:
                genres[genre] += 1

    return genres

In [134]:
top_level_genres = ["pop", "rock", "country", "rap", "electronic", "latin", "r&b", "indie", "reggae","metal", "jazz", "soundtrack", "christian","other"]

In [1514]:
genres = genrify(attributes_perfect.genre)

In [1515]:
genre_dict = dict(sorted(genres.items(), key=lambda x: x[1], reverse=True))

In [1516]:
genres_df = pd.DataFrame.from_dict(genre_dict, orient="index", columns=["count"]).reset_index().rename(columns={"index": "subgenre"})

In [1517]:
genres_df.head(30)

,subgenre,count
0,dance pop,5661
1,pop,5411
2,rap,4656
3,pop rap,4351
4,hip hop,3966
5,trap,2716
6,country,2601
7,r&b,2550
8,urban contemporary,2476
9,post-teen pop,2366


In [136]:
# create a dict that will select the right top-level genre by the given sub genre
sub_genre_parser = {
    "hip hop": "rap",
    "urban": "r&b",
    "trap": "rap",
    "hip": "rap",
    "soul": "r&b",
    "electropop": "electronic",
    "edm": "electronic",
    "post-grunge": "rock",
    "grunge": "rock",
    "wave": "rock",
    "dance": "electronic",
    "alternative": "rock",
    "folk": "indie",
    "funk": "r&b",
    "house": "electronic",
    "boy band": "pop",
    "hollywood": "soundtrack",
    "singer-songwriter": "indie",
    "punk": "rock",
    "electro": "electronic",
    "girl group": "pop",
    "crunk": "rap",
    "lilith": "indie",
    "redneck": "country",
    "glee club": "soundtrack",
    "talent show": "soundtrack",
    "disco": "pop",
    "stomp and holler": "indie",
    "g funk": "rap",
    "mexican": "latin",
    "tronica": "electronic",
    "show tunes": "soundtrack",
    "bubblegum": "pop",
    "blues": "r&b",
    "rave": "electronica",
    "contemporary": "r&b",
    "motown": "r&b",
    "emo": "rock",
    "ccm": "christian",
    "gospel": "christian",
    "worship": "christian",
    "puerto rican": "latin",
    "idol": "soundtrack",
    "colombian": "latin",
    "dancehall": "reggae",
    "screamo": "metal",
    "big room": "electronic",
    "broadway": "soundtrack",
    "hyphy": "rap",
    "americana": "indie",
    "brostep": "electronic",
    "espanol": "latin",
    "afrofuturism": "r&b",
    "movie": "soundtrack",
    "chicano": "latin",
    "bachata": "latin",
    "soca": "reggae",
    "techno": "electronic",
    "complextro": "electronic",
    "big beat": "electronic",
    "salsa": "latin",
    "drill": "rap",
    "trance": "electronic",
    "grime": "rap",
    "spanish": "latin",
    "jam band": "indie",
    "post-hardcore": "rock",
    "bop": "rap",
    "trip hop": "electronic",
    "plugg": "rap",
    "turntablism": "rap",
    "disney": "soundtrack",
    "red dirt": "country",
    "chillwave": "electronic",
    "thrash": "metal",
    "deathcore": "metal",
    "downtempo": "electronic",
    "freestyle": "pop",
    "industrial": "metal",
    "moombahton": "electronic",
    "basshall": "electronic",
    "children": "soundtrack",
    "children's": "soundtrack",
    "vgm": "soundtrack",
    "bollywood": "soundtrack",
    "tejano": "latin",
    "comedy": "other",
    "comic": "other",
    "karaoke": "pop",
    "fake": "pop",
    "sertanejo": "latin",
    "theme": "soundtrack",
    "cartoon": "soundtrack",
    "ost": "soundtrack",
    "easy listening": "pop", # christmas music
    "uk garage": "electronic",
    "tropical": "latin",
    "neo mellow": "pop",
    "bounce": "rap",
    "video game": "soundtrack"
     
}

In [1526]:
re.findall(f"{'|'.join([x for x in top_level_genres+ list(sub_genre_parser.keys())])}", attributes_perfect.genre[4521])#.split(",")#[3].split("_")

['hip hop', 'rap', 'hip hop', 'pop', 'rap', 'rap', 'hip hop', 'trap']

In [132]:
def genreGuess(subgenre):
    reg_expression = "|".join([x for x in  top_level_genres+ list(sub_genre_parser.keys())])
    super_genre ={}
    subgenre = str(subgenre)
    subgenre =re.sub(r"[\[\]\'\'\"\"]", "", subgenre)
    if subgenre == "":
        return "missing"
    else:
        sub_list = re.findall(reg_expression, subgenre)
    for genre in sub_list:
        if genre in list(sub_genre_parser):
            genre =  sub_genre_parser[genre]
        if genre not in super_genre.keys():
            super_genre[genre] = 1
        else:
            super_genre[genre] += 1
    try:
        max_genre = max(super_genre.items(), key=operator.itemgetter(1))
        return max_genre[0] 
    except ValueError:
        return "empty"

In [1530]:
attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess)).value_counts("genre_super")#[lambda x: x.genre_super == "empty"]

genre_super
pop           5715
rap           5480
country       2887
rock          2389
r&b           1744
electronic    1109
soundtrack     700
missing        645
latin          564
indie          355
metal          281
christian      140
empty          125
reggae          81
other           41
jazz            27
dtype: int64

In [1476]:
attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess))[lambda x: (x.genre_super == "missing")]

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
0,i'm your angel,r. kelly & celine dion,i'm your angelr. kelly & celine dion,0QWFiyhOqFu9RP8qzP6z5L,0.62,0.504,0.0,-7.26,1.0,0.0301,...,4.0,332067.0,0QWFiyhOqFu9RP8qzP6z5L,0.0,52.0,1998-11-08,1,0QWFiyhOqFu9RP8qzP6z5L,[],missing
20,ghetto cowboy,mo thugs family featuring bone thugs-n-harmony,ghetto cowboymo thugs family featuring bone th...,6xihjuCX132pyKm5dNFe3T,0.693,0.637,0.0,-5.487,1.0,0.158,...,4.0,324493.0,6xihjuCX132pyKm5dNFe3T,0.0,9.0,2000-12-04,1,6xihjuCX132pyKm5dNFe3T,[],missing
68,when a woman's fed up,r. kelly,when a woman's fed upr. kelly,015A6ZcNAWSSuwY7iDcD1b,0.682,0.579,2.0,-6.526,0.0,0.101,...,4.0,278760.0,015A6ZcNAWSSuwY7iDcD1b,1.0,47.0,1998-11-08,1,015A6ZcNAWSSuwY7iDcD1b,[],missing
85,home alone,r. kelly featuring keith murray,home aloner. kelly featuring keith murray,4UDQEazTqhCICqssWtMZEO,0.887,0.643,4.0,-2.578,0.0,0.214,...,4.0,299560.0,4UDQEazTqhCICqssWtMZEO,1.0,45.0,1998-11-08,1,4UDQEazTqhCICqssWtMZEO,[],missing
86,hooch,everything,hoocheverything,0hRK7UKSKVS5a18mxYT6pO,0.69,0.803,4.0,-6.549,1.0,0.0354,...,4.0,219843.0,0hRK7UKSKVS5a18mxYT6pO,0.0,46.0,2006-04-21,1,0hRK7UKSKVS5a18mxYT6pO,[],missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22205,you're still the one,teddy swims,you're still the oneteddy swims,4guQztGKAETkvuTekNeqBe,0.546,0.44,1,-6.668,1,0.0273,...,4,196413,4guQztGKAETkvuTekNeqBe,0.0,59.0,2020-07-17,0,4guQztGKAETkvuTekNeqBe,[],missing
22270,Take You Home With Me a.k.a. Body,Dubble Trubble,take you home with me a.k.a. bodydubble trubble,6Tydun5H6T2F3ZpE6aMWk6,0.878,0.439,10,-13.791,0,0.146,...,4,184080,6Tydun5H6T2F3ZpE6aMWk6,1.0,6.0,2013-01-05,1,6Tydun5H6T2F3ZpE6aMWk6,[],missing
22273,In the Sun,Joseph Arthur,in the sunjoseph arthur,0USctUIMP2DEp0oeBSyqJf,0.514,0.631,10,-7.458,1,0.0341,...,4,336293,0USctUIMP2DEp0oeBSyqJf,0.0,43.0,2000,1,0USctUIMP2DEp0oeBSyqJf,[],missing
22278,Send It On - Karaoke Version - Originally by D...,JMKaraoke,send it on - karaoke version - originally by d...,5pPi94jT2q88tkjDhBu6IV,0.623,0.319,9,-14.138,1,0.0303,...,4,208849,5pPi94jT2q88tkjDhBu6IV,0.0,13.0,2018-01-12,1,5pPi94jT2q88tkjDhBu6IV,[],missing


In [1522]:
others_genres = genrify(attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess))[lambda x: x.genre_super == "empty"].genre)

In [1523]:
others_dict = dict(sorted(others_genres.items(), key=lambda x: x[1], reverse=True))

In [1524]:
others_dict

{'nan': 14,
 'escape room': 11,
 'a cappella': 10,
 'tribute': 8,
 'marching band': 7,
 'beats': 6,
 'new jack swing': 4,
 'classify': 4,
 'sad lo-fi': 4,
 'new age': 3,
 'adult standards': 3,
 'mellow gold': 3,
 'miami bass': 3,
 'otacore': 3,
 'opm': 3,
 'alt z': 3,
 'lo-fi chill': 3,
 'musical advocacy': 2,
 'futuristic swag': 2,
 'wrestling': 2,
 'british invasion': 2,
 'classic dubstep': 2,
 'panpipe': 2,
 'jersey club': 2,
 'video game music': 2,
 'speedrun': 2,
 'australian psych': 2,
 'neo-psychedelic': 2,
 'novelty': 2,
 'celtic': 1,
 'middle earth': 1,
 'balkan brass': 1,
 'lullaby': 1,
 'chillhop': 1,
 'choral': 1,
 'shatta': 1,
 'riddim': 1,
 'new age piano': 1,
 'go-go': 1,
 'hands up': 1,
 'bluegrass': 1,
 'progressive bluegrass': 1,
 'chanson': 1,
 'acoustic cover': 1,
 'west end': 1,
 'papuri': 1,
 'lds': 1,
 'francoton': 1,
 'workout product': 1,
 'military band': 1,
 'wind symphony': 1,
 'college marching band': 1,
 'pixie': 1,
 'deep flow': 1,
 'melancholia': 1,
 'br

In [1536]:
attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess))[lambda x: (x.genre_super.isin(["missing", "other", "empty"]))].shape

(811, 25)

In [1537]:
# there are 811 songs left that who only are classified by specialized genres (missing), have a specific class (other) or are altogether empty.
# Thanks to my collection of recommendations I will be able to use these specific set of like songs to be able to make an educated guess on the over-arching classifying genre of songs with no or ambiguous genres.

In [1548]:
attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess))[lambda x: (x.genre_super.isin(["missing", "other", "empty"]))].chart.value_counts()

0    538
1    273
Name: chart, dtype: int64

In [1559]:
missing_super_genre = attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess))[lambda x: (x.genre_super.isin(["missing", "other", "empty"]))]

In [1593]:
def interpolateGenre(dataframe):
    ids_and_new_genre = pd.DataFrame()
    i = 0
    # use chart to determine if a song is either a recommendation or a billboard chart
    charters = dataframe.query("chart == 1").id_y.values
    recommendations = dataframe.query("chart == 0").id_y.values    
    # then use id on recommendations_join_perfect to gather recommendations or billboard songs
    recoms_for_charters = recommendations_join_perfect[lambda x: x.billboard_id.isin(charters)]
    charters_for_recom = recommendations_join_perfect[lambda x: x.recommendation_id.isin(recommendations)]
    # return to attributes_perfect with these ids and get all the subgenres in genre
    for id in charters:
        like_songs = recoms_for_charters[lambda x: x.billboard_id == id].recommendation_id.values
        like_genres = attributes_perfect[lambda x: x.id_y.isin(like_songs)].genre.values
        genre = genreGuess("[" + " ".join([re.sub(r"[\[\]]", "", x) for x in like_genres]) + "]")
        ids_and_new_genre.loc[i, "id_y"] = id
        ids_and_new_genre.loc[i, "genre_super"] = genre
        i+=1

    for id in recommendations:
        like_songs = recoms_for_charters[lambda x: x.recommendation_id == id].billboard_id.values
        recoms = np.array([])
        for bil in like_songs:
            recoms = np.concatenate([recoms, (recommendations_join_perfect[lambda x: x.billboard_id == bil].recommendation_id.values)])
        like_songs = np.concatenate([like_songs, recoms]) 
        like_genres = attributes_perfect[lambda x: x.id_y.isin(like_songs)].genre.values
        genre = genreGuess("[" + " ".join([re.sub(r"[\[\]\'\']", "", x) for x in like_genres]) + "]")
        ids_and_new_genre.loc[i, "id_y"] = id
        ids_and_new_genre.loc[i, "genre_super"] = genre
        i+=1


    return ids_and_new_genre

In [1594]:
second_genre_guesses = interpolateGenre(missing_super_genre)

In [1692]:
still_missing = second_genre_guesses[lambda x: x.genre_super == "missing"].id_y.dropna().values

In [1626]:
# my last line of defense will be to use chosic to get the related artist genres.
# thankfully, i can input directly into the website the spotify id and get the return of related artists genres.
# reference: "https://www.chosic.com/music-genre-finder/?track="

In [1671]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [1690]:
def lastLine(ids):
    driver = webdriver.Chrome("../../../Python/scraping/chromedriver.exe")
    last_guess = pd.DataFrame()
    j = 0
    for id in ids:
        driver.get(f"https://www.chosic.com/music-genre-finder/?track={id}")
        time.sleep(5)
        related_subgenres = driver.find_elements(By.XPATH, "//div[@class='spotify-result']/div/a")
        if related_subgenres != []:
            genre_string = "[" + ",".join([x.text for x in related_subgenres]) + "]"
        else:
            genre_string = "[]"
        last_guess.loc[j, "genre_super"] = genreGuess(genre_string)
        last_guess.loc[j, "id_y"] = id
        j+=1
        time.sleep(1)

    driver.quit()    
    return last_guess

In [1694]:
last_guess = lastLine(still_missing)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [1697]:
last_guess.shape

(340, 2)

In [1696]:
last_guess.genre_super.value_counts()

empty         68
pop           63
missing       40
soundtrack    32
rap           28
electronic    18
r&b           18
rock          18
indie         16
other         14
country       14
christian      5
latin          4
jazz           1
metal          1
Name: genre_super, dtype: int64

In [1699]:
second_genre_guesses.shape

(811, 2)

In [1724]:
attributes_first = attributes_perfect.assign(genre_super = lambda x: x.genre.apply(genreGuess))

In [1725]:
attributes_second = attributes_first.merge(second_genre_guesses[lambda x: x.genre_super != "missing"], "left", on="id_y")

In [1726]:
attributes_second.genre_super_y = attributes_second.genre_super_y.fillna(attributes_second.genre_super_x)

In [1727]:
attributes_last = attributes_second.merge(last_guess, "left", on="id_y")

In [1728]:
attributes_last.genre_super = attributes_last.genre_super.fillna(attributes_last.genre_super_y)

In [1729]:
attributes_last = attributes_last.drop(["genre_super_x", "genre_super_y"], axis=1)

In [1730]:
attributes_last.genre_super.value_counts()

pop           5895
rap           5583
country       2930
rock          2444
r&b           1816
electronic    1133
soundtrack     750
latin          568
indie          375
metal          282
empty          193
christian      145
reggae          81
missing         40
jazz            28
other           20
Name: genre_super, dtype: int64

In [1731]:
attributes_last.shape

(22283, 25)

In [1732]:
attributes_perfect.shape

(22283, 24)

In [1735]:
attributes_last

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
0,i'm your angel,r. kelly & celine dion,i'm your angelr. kelly & celine dion,0QWFiyhOqFu9RP8qzP6z5L,0.62,0.504,0.0,-7.26,1.0,0.0301,...,4.0,332067.0,0QWFiyhOqFu9RP8qzP6z5L,0.0,52.0,1998-11-08,1,0QWFiyhOqFu9RP8qzP6z5L,[],r&b
1,nobody's supposed to be here,deborah cox,nobody's supposed to be heredeborah cox,51QxenFmXlJXUN9mpvxlaL,0.662,0.531,8.0,-5.3,1.0,0.0361,...,4.0,262333.0,51QxenFmXlJXUN9mpvxlaL,0.0,56.0,1998-09-17,1,51QxenFmXlJXUN9mpvxlaL,"['canadian pop', 'hip pop', 'neo soul', 'new j...",r&b
2,lately,divine,latelydivine,5pgkdQi8HMQHYrJCB4GLsA,0.582,0.665,2.0,-6.992,1.0,0.0517,...,4.0,218070.0,5pgkdQi8HMQHYrJCB4GLsA,0.0,13.0,1996-04-26,1,5pgkdQi8HMQHYrJCB4GLsA,['fijian pop'],pop
3,...baby one more time,britney spears,...baby one more timebritney spears,3MjUtNVVq3C8Fn0MP3zhXa,0.759,0.699,0.0,-5.745,0.0,0.0307,...,4.0,211066.0,3MjUtNVVq3C8Fn0MP3zhXa,0.0,79.0,1999-01-12,1,3MjUtNVVq3C8Fn0MP3zhXa,"['dance pop', 'pop', 'post-teen pop']",pop
4,because of you,98 degrees,because of you98 degrees,25tmiXojR7whvZDz3LpMb2,0.732,0.603,0.0,-7.624,1.0,0.0271,...,4.0,297893.0,25tmiXojR7whvZDz3LpMb2,0.0,50.0,1998-01-01,1,25tmiXojR7whvZDz3LpMb2,"['boy band', 'dance pop']",pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22278,Send It On - Karaoke Version - Originally by D...,JMKaraoke,send it on - karaoke version - originally by d...,5pPi94jT2q88tkjDhBu6IV,0.623,0.319,9,-14.138,1,0.0303,...,4,208849,5pPi94jT2q88tkjDhBu6IV,0.0,13.0,2018-01-12,1,5pPi94jT2q88tkjDhBu6IV,[],rap
22279,I Am Beautiful,Candice Glover,i am beautifulcandice glover,2jKrwn5GUuLUmO7dNUMc5c,0.628,0.558,4,-5.872,0,0.0343,...,4,177053,2jKrwn5GUuLUmO7dNUMc5c,0.0,28.0,2013-01-01,1,2jKrwn5GUuLUmO7dNUMc5c,[],rap
22280,I Luv This Shit,August Alsina,i luv this shitaugust alsina,4IeUAegXVbETb7FpgNDnSM,0.643,0.592,1,-6.684,0,0.277,...,4,267920,4IeUAegXVbETb7FpgNDnSM,1.0,63.0,2013-01-01,1,4IeUAegXVbETb7FpgNDnSM,"['dance pop', 'pop r&b', 'pop rap', 'r&b', 'tr...",r&b
22281,A-YO,Lady Gaga,a-yolady gaga,6Az2Ll2CPnWYQ2BgiHf4Di,0.666,0.889,7,-5.699,1,0.144,...,4,207720,6Az2Ll2CPnWYQ2BgiHf4Di,0.0,56.0,2016-10-21,1,6Az2Ll2CPnWYQ2BgiHf4Di,"['art pop', 'dance pop', 'pop']",pop


In [1739]:
billboard.info()#.merge(attributes_last.loc[:, ["SongID", "energy"]], "left", on="SongID").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111970 entries, 0 to 111969
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   111970 non-null  int64  
 1   Song                    111970 non-null  object 
 2   Performer               111970 non-null  object 
 3   Week Position           111970 non-null  int64  
 4   WeekID                  111970 non-null  object 
 5   Previous Week Position  101967 non-null  float64
 6   Peak Position           111970 non-null  float64
 7   Weeks on Chart          111970 non-null  float64
 8   Instance                111970 non-null  float64
 9   SongID                  111970 non-null  object 
 10  Week_No                 111970 non-null  int64  
 11  Performer_y             111804 non-null  object 
 12  Song_y                  111804 non-null  object 
 13  id                      110497 non-null  object 
dtypes: float64(4), int64

In [1755]:
billboard_songids = billboard.merge(attributes_last.loc[:, ["SongID", "energy"]], "left", on="SongID")[lambda x: x.energy.isnull()].SongID.unique()

In [1820]:
attributes_last[lambda x: x.Performer.str.startswith("lady gaga")].head()

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
3157,just dance,lady gaga featuring colby o'donis,just dancelady gaga featuring colby o'donis,6HkjpGIOHv4y0IBAxUbE5z,0.822,0.753,1.0,-4.499,0.0,0.0332,...,4.0,241933.0,2x7MyWybabEz6Y6wvHuwGE,0.0,72.0,2008-01-01,1,2x7MyWybabEz6Y6wvHuwGE,"['dance pop', 'pop rap', 'urban contemporary']",pop
3323,poker face,lady gaga,poker facelady gaga,0WfKDYeUAoLA3vdvLKKWMW,0.846,0.815,4.0,-4.512,1.0,0.0761,...,4.0,237200.0,5R8dQOPq8haW94K7mgERlO,0.0,74.0,2008-01-01,1,5R8dQOPq8haW94K7mgERlO,"['art pop', 'dance pop', 'pop']",pop
3392,lovegame,lady gaga,lovegamelady gaga,5c0Wgs68kjgViFCVDGLB2I,0.892,0.649,6.0,-5.644,0.0,0.0652,...,4.0,216333.0,0TcJ7QWpggdSg8t0fHThHm,0.0,64.0,2008-01-01,1,0TcJ7QWpggdSg8t0fHThHm,"['art pop', 'dance pop', 'pop']",pop
3580,paparazzi,lady gaga,paparazzilady gaga,16Me2jlMlTX4xsps2j71mC,0.755,0.688,8.0,-3.97,1.0,0.0535,...,4.0,208306.0,7Hqig8kp32q2Ire3ECQvWM,0.0,67.0,2008-01-01,1,7Hqig8kp32q2Ire3ECQvWM,"['art pop', 'dance pop', 'pop']",pop
3659,bad romance,lady gaga,bad romancelady gaga,0SiywuOBRcynK0uKGWdCnn,0.696,0.921,0.0,-3.755,1.0,0.0363,...,4.0,294573.0,0SiywuOBRcynK0uKGWdCnn,1.0,82.0,2009-11-05,1,0SiywuOBRcynK0uKGWdCnn,"['art pop', 'dance pop', 'pop']",pop


In [1802]:
attributes_last[lambda x: x.Song.str.startswith("Lemon")]#.SongID.values

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
22282,Lemon,N.E.R.D,lemonn.e.r.d,4PpuH4mxL0rD35mOWaLoKS,0.916,0.727,1,-6.556,1,0.0861,...,4,219707,4PpuH4mxL0rD35mOWaLoKS,1.0,62.0,2017-12-15,1,4PpuH4mxL0rD35mOWaLoKS,"['hip hop', 'virginia hip hop']",rap


In [1805]:
last_parser = {"got it alleve & jadakiss": "got it allruff ryders", #1
"take you home with me a.k.a. bodyr. kelly & jay-z": "take you home with me a.k.a. bodydubble trubble", #4
"down 4 uirv gotti presents the inc. featuring ja rule, ashanti, charli baltimore & vita": "down 4 ucharli baltimore", #5
"under pressurethe used and my chemical romance": "under pressuremy chemical romance", #8
"in the sunmichael stipe":"in the sunjoseph arthur", #9
"time of the seasonblake lewis":"time of the season - mono versionthe zombies",#10
"many rivers to crossannie lennox":"many rivers to crossjimmy cliff", #11
"put onyoung jeezy featuring kanye west":"put onjeezy",
"remember met.i. featuring mary j. blige":"remember me (feat. mary j. blige)t.i.",
"send it ondisney's friends for change":"send it on - karaoke version - originally by disney's friends for changejmkaraoke",
"stranded (haiti mon amour)jay-z, bono, the edge & rihanna":'stranded (haiti mon amour)jay-z, bono, the edge & rihanna',
"i luv this shtaugust alsina featuring trinidad james":"i luv this shitaugust alsina",
"lemonnerd & rihanna": "lemonn.e.r.d"}

In [1811]:
final_songids = pd.DataFrame.from_dict(last_parser, columns=["song_id"], orient="index")

In [1815]:
final_songids = final_songids.reset_index().rename(columns={"index": "SongID"})

In [1817]:
billboard_end = billboard.merge(final_songids, "left", on="SongID")

In [1818]:
billboard_end.song_id = billboard_end.song_id.fillna(billboard_end.SongID)

In [1821]:
billboard_end.merge(attributes_last.loc[:, ["SongID", "energy"]], "left", left_on="song_id", right_on="SongID").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117494 entries, 0 to 117493
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   117494 non-null  int64  
 1   Song                    117494 non-null  object 
 2   Performer               117494 non-null  object 
 3   Week Position           117494 non-null  int64  
 4   WeekID                  117494 non-null  object 
 5   Previous Week Position  107106 non-null  float64
 6   Peak Position           117494 non-null  float64
 7   Weeks on Chart          117494 non-null  float64
 8   Instance                117494 non-null  float64
 9   SongID_x                117494 non-null  object 
 10  Week_No                 117494 non-null  int64  
 11  Performer_y             117328 non-null  object 
 12  Song_y                  117328 non-null  object 
 13  id                      116021 non-null  object 
 14  song_id             

In [1824]:
billboard_end = billboard_end.drop(["index", "SongID"], axis=1).rename(columns={"song_id": "SongID"})

In [1825]:
billboard_end.columns

Index(['Song', 'Performer', 'Week Position', 'WeekID',
       'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
       'Week_No', 'Performer_y', 'Song_y', 'id', 'SongID'],
      dtype='object')

In [1827]:
billboard_end = billboard_end.loc[:, ['Song', 'Performer', 'SongID', 'Week Position', 'WeekID',
       'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
       'Week_No', 'Performer_y', 'Song_y', 'id']]

In [1828]:
billboard_end.shape

(111970, 13)

In [1833]:
attributes_last[lambda x: x.Song == "panda"]

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
6213,panda,desiigner,pandadesiigner,275a9yzwGB6ncAW4SxY7q3,0.72,0.748,10.0,-5.522,0.0,0.393,...,4.0,247866.0,275a9yzwGB6ncAW4SxY7q3,1.0,68.0,2016-02-26,1,275a9yzwGB6ncAW4SxY7q3,"['pop rap', 'rap', 'trap', 'viral trap']",rap
17002,panda,desiigner,pandadesiigner,5YEOzOojehCqxGQCcQiyR4,0.576,0.766,10,-4.943,0,0.449,...,4,246761,5YEOzOojehCqxGQCcQiyR4,1.0,73.0,2016-06-26,0,5YEOzOojehCqxGQCcQiyR4,"['pop rap', 'rap', 'trap', 'viral trap']",rap
17467,panda,desiigner,pandadesiigner,6r2jK1A6oFRPREZfxjc5d1,0.601,0.695,10,-5.811,0,0.395,...,3,226520,6r2jK1A6oFRPREZfxjc5d1,0.0,62.0,2016-02-26,0,6r2jK1A6oFRPREZfxjc5d1,"['pop rap', 'rap', 'trap', 'viral trap']",rap
17546,panda,kevin hart,pandakevin hart,5U51BJU0ncHHsLw9raErsS,0.868,0.69,8,-6.546,1,0.516,...,4,44300,5U51BJU0ncHHsLw9raErsS,0.0,45.0,2019-06-01,0,5U51BJU0ncHHsLw9raErsS,"['black comedy', 'comedy']",other


In [1834]:
billboard_end.merge(attributes_last.drop_duplicates("SongID").loc[:, ["SongID", "energy"]], "left",on="SongID").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111970 entries, 0 to 111969
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Song                    111970 non-null  object 
 1   Performer               111970 non-null  object 
 2   SongID                  111970 non-null  object 
 3   Week Position           111970 non-null  int64  
 4   WeekID                  111970 non-null  object 
 5   Previous Week Position  101967 non-null  float64
 6   Peak Position           111970 non-null  float64
 7   Weeks on Chart          111970 non-null  float64
 8   Instance                111970 non-null  float64
 9   Week_No                 111970 non-null  int64  
 10  Performer_y             111804 non-null  object 
 11  Song_y                  111804 non-null  object 
 12  id                      110497 non-null  object 
 13  energy                  111894 non-null  object 
dtypes: float64(4), int64

In [4]:
engine = sql.create_engine("sqlite:///../src/data/music.db")
conn = engine.connect()

In [1837]:
attributes_last.to_sql("attributes", con=conn, if_exists="replace", index=False)

In [1840]:
billboard_end.to_sql("billboard", con=conn, if_exists="replace", index=False)

In [1841]:
conn.close()

In [ ]:
# We will now recall the api in order to acquire popularity metrics for all songs at the same time (11/27/21).
# From these calls, we will also get the id and genre information from the artist object associated with each track so that we have all available seeds to us when we 
# call on the Spotify Recommendation algorithm to give us a good sample of similar songs that did not chart. 

In [6]:
engine.table_names()

['artists', 'artists_join', 'attributes', 'billboard', 'recommendation_join']

In [ ]:
# The Major Groupings of Our Data and Its Structure:
    # Year
        # Billboard chart data is based in time. The Top 100 Songs are recorded every week.
        # The noncharting songs have their associated release date approximating when they were released.
        # Will help analysis by: Allowing Us to See the Trends in Popular Music and Music in general throughout the time periods analyzed.
        # such as: change in Spotify attributes, genres, artists, popularity.

    # Artist:
        # Indicate the performer associated with the song.
        # will help analysis by: Identifying one-hit wonders(who are they? who lasted the longest on the chart? who is still being listened today?)
        # Identifying the artists who have been the most resilent through the massive changes in the music industry. Who has the most hits?
        # Identifying artists who's popularity, captured in the Hot 100, shows a discrepancy between the Spotify popularity metric and the billboard status.  
    
    # Genre:
        # There are two genre categories. Genre that holds the list of all their subgenres and genre_super that holds the song's high level genre.
        # will help analysis by: Using genre_super as a classifier of songs, see the differences in Spotify attributes amongst popular songs and their genre counterparts from the sample recommendations.
        # See the differences between only Hot 100 Songs. Compare these aggregations to past years and report the results. Were popular songs more danceable this year than last?
        # How did the distribution of #1 Songs compare to a typical years? Is one genre becoming more popular than the others? 
    
    # Songs (the base level of all our analysis)
        # its groups:
            # Songs that charted on the Billboard Hot 100 singles chart
            # A sample of like Songs based on Spotify's Recommendation engine
            # **represented by the chart categorical variable**
    # 
    # Billboard Data
        # will help analysis by: understanding grounding data through time.
        # need to create: popularity metric based on chart position, weeks on chart and reappearence on chart.
        # Our data grounding America's music taste in time. How has it evolved over the years? More Songs? More Artists? More Genres?
        # Creating a popularity corollary based that can be directly compared to Spotify's own popularity metric will help users understand past popularity and compare it to music consumption today.



## Charts for Year Summary

In [153]:
with engine.connect() as con:
    billboard = pd.read_sql("SELECT * FROM billboard", parse_dates={"WeekID": "%Y-%m-%d"}, con=con)
    attributes = pd.read_sql("SELECT * FROM attributes", con=con)
    artists = pd.read_sql("SELECT * FROM artists", con=con)
    artists_join = pd.read_sql("SELECT * FROM artists_join", con=con)
    recommendation_join = pd.read_sql("SELECT * FROM recommendation_join", con=con)

In [154]:
billboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111970 entries, 0 to 111969
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Song                    111970 non-null  object        
 1   Performer               111970 non-null  object        
 2   SongID                  111970 non-null  object        
 3   Week Position           111970 non-null  int64         
 4   WeekID                  111970 non-null  datetime64[ns]
 5   Previous Week Position  101967 non-null  float64       
 6   Peak Position           111970 non-null  float64       
 7   Weeks on Chart          111970 non-null  float64       
 8   Instance                111970 non-null  float64       
 9   Week_No                 111970 non-null  int64         
 10  Performer_y             111804 non-null  object        
 11  Song_y                  111804 non-null  object        
 12  id                      110497

In [155]:
flip_chart = billboard.merge(attributes.drop_duplicates("SongID").loc[:, ["SongID", "chart"]], "left", on="SongID")[lambda x: x.chart == 0.0].SongID.unique()

In [156]:
no_dup_attributes = attributes.drop_duplicates("SongID")

In [157]:
no_dup_attributes.loc[lambda x: x.SongID.isin(flip_chart), "chart"]=1

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [158]:
billboard.merge(no_dup_attributes.loc[:, ["SongID", "chart"]], "left", on="SongID")[lambda x: x.chart.isnull()]

,Song,Performer,SongID,Week Position,WeekID,Previous Week Position,Peak Position,Weeks on Chart,Instance,Week_No,Performer_y,Song_y,id,chart
8500,ta da,lil' mo,ta dalil' mo,100,2000-08-12,NaN,100.0,1.0,1.0,32,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
8599,ta da,lil' mo,ta dalil' mo,99,2000-08-19,100.0,99.0,2.0,1.0,33,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
8697,ta da,lil' mo,ta dalil' mo,97,2000-08-26,99.0,97.0,3.0,1.0,34,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
8797,ta da,lil' mo,ta dalil' mo,97,2000-09-02,97.0,97.0,4.0,1.0,35,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
8900,ta da,lil' mo,ta dalil' mo,100,2000-09-09,97.0,97.0,5.0,1.0,36,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
8995,ta da,lil' mo,ta dalil' mo,95,2000-09-16,100.0,95.0,6.0,1.0,37,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
9099,ta da,lil' mo,ta dalil' mo,99,2000-09-23,95.0,95.0,7.0,1.0,38,LIL LO,Damn Tana,0BKlETtyw17Zz0d76Yfxs3,NaN
56454,hell of a life,t.i.,hell of a lifet.i.,54,2009-10-24,NaN,54.0,1.0,1.0,42,The Last Ten Seconds of Life,Jealousy is Hell,0pj4GlfLXlCscZrYpQB6Do,NaN
56593,hell of a life,t.i.,hell of a lifet.i.,93,2009-10-31,54.0,54.0,2.0,1.0,43,The Last Ten Seconds of Life,Jealousy is Hell,0pj4GlfLXlCscZrYpQB6Do,NaN
58744,i'm back,t.i.,i'm backt.i.,44,2010-04-03,NaN,44.0,1.0,1.0,13,10cc,"I'm So Laid Back, I'm Laid Out",07biPM7Obf2vXxj6vOaOkL,NaN


In [97]:
ids = ["6rkAY9rk1NTFB94QxG3LJR", "2fSDr3jBjwJsHBWwvvezgj"]
id_string = ",".join([x for x in ids])
tracks_specific = requests.get(f"https://api.spotify.com/v1/tracks?market=US&ids={id_string}", headers=headers).json() # popularity, explicit, artist, release_date
tracks_features = requests.get(f"https://api.spotify.com/v1/audio-features?ids={id_string}", headers=headers).json() # feature

In [98]:
bad_artists = pd.DataFrame()
bad_tracks = pd.DataFrame()
_temp_feat = pd.DataFrame()
j = 0
i = 0
s = 0
for t in tracks_specific["tracks"]:
    bad_tracks.loc[j, "release_date"] = t["album"]["release_date"]
    bad_tracks.loc[j, "id"] = t["id"]
    bad_tracks.loc[j, "popularity"] = t["popularity"]
    bad_tracks.loc[j, "explicit"] = t["explicit"]
    bad_tracks.loc[j, "Song"] = t["name"]
    bad_tracks.loc[j, "Performer"] = t["artists"][0]["name"]
    for _art in t["artists"]:
        bad_artists.loc[i, "artist"] = _art["name"]
        bad_artists.loc[i, "artist_id"] = _art["id"]
        bad_artists.loc[i, "id_fk"] = t["id"]
        i += 1
    j+= 1
    
for t in tracks_features["audio_features"]:
    _temp_feat.loc[s, "danceability"] = t["danceability"]
    _temp_feat.loc[s, "energy"] = t["energy"]
    _temp_feat.loc[s, "key"] = t["key"]
    _temp_feat.loc[s, "loudness"] = t["loudness"]
    _temp_feat.loc[s, "mode"] = t["mode"]
    _temp_feat.loc[s, "speechiness"] = t["speechiness"]
    _temp_feat.loc[s, "acousticness"] = t["acousticness"]
    _temp_feat.loc[s, "instrumentalness"] = t["instrumentalness"]
    _temp_feat.loc[s, "liveness"] = t["liveness"]
    _temp_feat.loc[s, "valence"] = t["valence"]
    _temp_feat.loc[s, "tempo"] = t["tempo"]
    _temp_feat.loc[s, "duration"] = t["duration_ms"]
    _temp_feat.loc[s, "timesignature"] = t["time_signature"]
    _temp_feat.loc[s, "id"] = t["id"]
    s += 1

In [99]:
tracks_to_add = bad_tracks.merge(_temp_feat, "inner", on="id")

In [105]:
tracks_to_add.loc[0, "SongID"] = "you and ilady gaga"
tracks_to_add.loc[1, "SongID"] = "ta dalil' mo"

In [106]:
tracks_to_add.loc[0, "id_y"] = tracks_to_add.loc[0, "id"] 
tracks_to_add.loc[1, "id_y"] = tracks_to_add.loc[1, "id"] 

In [107]:
tracks_to_add.loc[0, "id_fk"] = tracks_to_add.loc[0, "id"] 
tracks_to_add.loc[1, "id_fk"] = tracks_to_add.loc[1, "id"] 

In [108]:
tracks_to_add.loc[0, "genre"] = "['pop']"
tracks_to_add.loc[1, "genre"] = "['r&b']"

In [109]:
tracks_to_add.loc[0, "genre_super"] = "pop"
tracks_to_add.loc[1, "genre_super"] = "r&b"

In [113]:
tracks_to_add.loc[0, "chart"] = 1
tracks_to_add.loc[1, "chart"] = 1

In [114]:
attributes.columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre',
       'genre_super'],
      dtype='object')

In [116]:
tracks_to_add = tracks_to_add.loc[:, ['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre',
       'genre_super']]

In [120]:
recomend_art_last, recommend_song_last = returnEntries(ids)

finished with 6rkAY9rk1NTFB94QxG3LJR. 2 LEFT.
finished with 2fSDr3jBjwJsHBWwvvezgj. 2 LEFT.


In [125]:
recommendations = recommend_song_last.merge(recomend_art_last.loc[:, ["genre", "id_fk"]], "left", left_on="id", right_on="id_fk")

In [130]:
recommendations = recommendations.drop("id_fk_y", axis=1).rename(columns={"id_fk_x": "id_fk"})

In [139]:
recommendations = recommendations.assign(genre_super = lambda x: x.genre.apply(genreGuess))

In [144]:
recommendations.loc[:, "explicit"] = recommendations.explicit.astype("float")

In [147]:
recommendations.loc[:, "chart"] = 0.0

In [148]:
recommendations = recommendations.assign(SongID = lambda x: x.Song.apply(str.lower) + x.Performer.apply(str.lower))

In [149]:
recommendations = recommendations.assign(id_y = lambda x: x.id)

In [180]:
recommendations.loc[12, "genre_super"] = "rap"

In [151]:
recommendations = recommendations.loc[:, ['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre',
       'genre_super']]

In [163]:
artists = artists.drop("index", axis=1)

In [162]:
recomend_art_last = recomend_art_last.loc[:, ['artist', 'artist_id', 'followers', 'popularity', 'genre']]

In [166]:
to_rj = recommendations.loc[:, ["id", "id_fk"]].rename(columns={"id": "recommendation_id", "id_fk": "billboard_id"})

In [167]:
to_aj = recommendations.loc[:, ["Performer", "id"]].merge(recomend_art_last.loc[:, ["artist", "artist_id"]],"left", left_on="Performer", right_on="artist").loc[:, ["id", "artist_id"]].drop_duplicates().rename(columns={"id": "song_id"})

In [171]:
recommendation_join = recommendation_join.drop("index", axis=1)

In [208]:
new_recommend = pd.concat([recommendation_join, to_rj], axis=0, ignore_index=True)

In [209]:
new_art_join = pd.concat([artists_join.drop("index", axis=1), pd.DataFrame.from_records([{"song_id": "2fSDr3jBjwJsHBWwvvezgj", "artist_id": "5wBDD4FNJvob5fAGkAIQ92"},
{"song_id": "6rkAY9rk1NTFB94QxG3LJR", "artist_id": "1HY2Jd0NmPuamShAr6KMms"}]),to_aj], axis=0, ignore_index=True)

In [210]:
new_attr = pd.concat([no_dup_attributes, recommendations,  tracks_to_add], axis=0, ignore_index=True)

In [193]:
artists.followers = artists.followers.astype("float")

In [211]:
new_arts = pd.concat([artists, recomend_art_last], axis=0, ignore_index=True)

In [215]:
billboard.columns

Index(['Song', 'Performer', 'SongID', 'Week Position', 'WeekID',
       'Previous Week Position', 'Peak Position', 'Weeks on Chart', 'Instance',
       'Week_No', 'Performer_y', 'Song_y', 'id'],
      dtype='object')

In [218]:
new_billboard = billboard.loc[:, ["WeekID", "Week Position", "Song", "Performer", 'Previous Week Position', 'Peak Position', 'Weeks on Chart', "SongID"]]

In [229]:
new_billboard["Previous Week Position"] = new_billboard["Previous Week Position"].fillna(value=0)

In [262]:
new_billboard = new_billboard.assign(bill_popularity = lambda x: (101-x["Week Position"]))

In [264]:
engine.table_names()

['artists', 'artists_join', 'attributes', 'billboard', 'recommendation_join']

In [270]:
with engine.connect() as con:   
    new_billboard.to_sql("billboard", con=con, index=False, if_exists="replace")
    new_attr.to_sql("attributes", con=con, index=False, if_exists="replace")
    new_arts.to_sql("artists", con=con, index=False, if_exists="replace")
    new_recommend.to_sql("recommendation_join", con=con, index=False, if_exists="replace")
    new_art_join.to_sql("artists_join", con=con, index=False, if_exists="replace")

In [ ]:
# The goal for each year's dashboard and the charts that comprise it is to provide the user with the requiste knowledge of American Popular Music as a whole given the data and allowing for comments on this data based on the
# metrics and classifying done for this project. In a sense, commenting on the data by referencing the averages of popular music as a whole and how this year differed from the averages and the preceding and future years.
# For year, I want:
    # Sunburst Diagram with structure of genre_super > subgenre > artist > songs
    # Multitrace Radar Charts of Spotify Attributes for Chart Songs (chart = 0) and Non-Chart Recommendations (chart=0)
        # Will need to allow for additional filter for segmenting chart into overarching genre_super.  
    # The rest of the charts will deal with distribution and properties of the hot 100 songs and their different gradations compared to the other to years analyzed.
    # Which and how many songs reached number 1? Was this in-line or diverge from a typical year?
    # Which song lasted the longest on the chart? (i.e., had the most longevity in the popular imagination for the year)
    # Explain the structure of the song via the Spotify attributes.
    # Which songs cannot be found on Spotify? Popular Songs that have been lost to history, have become unwritable to Spotify's servers? Ghosts of the Past. Locked Away.
    # Which songs who gathered the most popularity via the bill_popularity metric are the least popular in terms of Spotify's popularity metric.

In [3]:
engine = sql.create_engine("sqlite:///../src/data/music.db")

In [4]:
with engine.connect() as con:
    billboard = pd.read_sql("SELECT * FROM billboard", con=con)
    attributes = pd.read_sql("SELECT * FROM attributes", con=con)
    artists = pd.read_sql("SELECT * FROM artists", con=con)
    recommend_join = pd.read_sql("SELECT * FROM recommendation_join", con=con)
    artists_join = pd.read_sql("SELECT * FROM artists_join", con=con)

In [5]:
artists_join.head()

,song_id,artist_id
0,0QWFiyhOqFu9RP8qzP6z5L,2mxe0TnaNL039ysAj51xPQ
1,015A6ZcNAWSSuwY7iDcD1b,2mxe0TnaNL039ysAj51xPQ
2,4UDQEazTqhCICqssWtMZEO,2mxe0TnaNL039ysAj51xPQ
3,19XWdM9zSb4IIiFCXO1SlE,2mxe0TnaNL039ysAj51xPQ
4,7tNU9Ax7hzHA4nzo57aZb3,2mxe0TnaNL039ysAj51xPQ


In [6]:
billboard.merge(attributes.merge(artists_join, "left", left_on="id_y", right_on="song_id").loc[:, ['SongID', 'energy', 'id_y', 'artist_id']], 'left', on="SongID")\
    .drop_duplicates(subset=["id_y", 'artist_id']).groupby('id_y').count().value_counts('artist_id')

artist_id
1     5948
2     1758
3      405
4      114
5       30
6        8
7        7
8        2
22       1
16       1
13       1
10       1
0        1
dtype: int64

In [7]:
billboard.merge(attributes.merge(artists_join, "left", left_on="id_y", right_on="song_id").loc[:, ['SongID', 'energy', 'id_y', 'artist_id']], 'left', on="SongID")\
    .drop_duplicates(subset=["id_y", 'artist_id']).groupby('id_y').count().value_counts("artist_id", ascending=False)

artist_id
1     5948
2     1758
3      405
4      114
5       30
6        8
7        7
8        2
22       1
16       1
13       1
10       1
0        1
dtype: int64

In [8]:
billboard.merge(attributes.merge(artists_join, "left", left_on="id_y", right_on="song_id").loc[:, ['SongID', 'energy', 'id_y', 'artist_id']], 'left', on="SongID")\
    .drop_duplicates(subset=["id_y", 'artist_id']).groupby('id_y').count()[lambda x: x.artist_id == 5]

,WeekID,Week Position,Song,Performer,Previous Week Position,Peak Position,Weeks on Chart,SongID,bill_popularity,energy,artist_id
id_y,,,,,,,,,,,
12PNcnMsjsZ3eHm62t8hiy,5,5,5,5,5,5,5,5,5,5,5
1ZHYJ2Wwgxes4m8Ba88PeK,5,5,5,5,5,5,5,5,5,5,5
1bIEvOOqf2V3QBrFiClE3Y,5,5,5,5,5,5,5,5,5,5,5
1jYiIOC5d6soxkJP81fxq2,5,5,5,5,5,5,5,5,5,5,5
1zPDgF3K6QXcTopDB3SVg9,5,5,5,5,5,5,5,5,5,5,5
25fqrbEPDnqe9Bf8RLL7an,5,5,5,5,5,5,5,5,5,5,5
26XUAwGZqQEyy8Ubhtj4kq,5,5,5,5,5,5,5,5,5,5,5
2AYTGaclgKXACh13nJRVcV,5,5,5,5,5,5,5,5,5,5,5
2Ima72UXnfvR539afL3SAF,5,5,5,5,5,5,5,5,5,5,5


In [9]:
billboard[lambda x: x.Performer.str.contains("\s+and|\s+And|&|feat.|featuring|feat")].merge(attributes.merge(artists_join, "left", left_on="id_y", right_on="song_id").loc[:, ['SongID', 'energy', 'id_y', 'artist_id']], 'left', on="SongID")\
    .drop_duplicates(subset=["id_y", 'artist_id']).groupby('id_y').count()[lambda x: x.artist_id < 2]

,WeekID,Week Position,Song,Performer,Previous Week Position,Peak Position,Weeks on Chart,SongID,bill_popularity,energy,artist_id
id_y,,,,,,,,,,,
00FROhC5g4iJdax5US8jRr,1,1,1,1,1,1,1,1,1,1,1
00FU5uypELwgQuM9IbFC6G,1,1,1,1,1,1,1,1,1,1,1
030OCtLMrljNhp8OWHBWW3,1,1,1,1,1,1,1,1,1,1,1
06iIZcy6VspGTukqKkgLvw,1,1,1,1,1,1,1,1,1,1,1
0AI0VW1BPoOel5Gtyo11wO,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
7vOjTY220BqIttYDbwshED,1,1,1,1,1,1,1,1,1,1,1
7wfALtuf2PVDiTeWvfJWPh,1,1,1,1,1,1,1,1,1,1,1
7yNK27ZTpHew0c55VvIJgm,1,1,1,1,1,1,1,1,1,1,1


In [10]:
attributes[lambda x: x.id_y == "00FU5uypELwgQuM9IbFC6G"]

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
1065,give it to her,tanto metro & devonte,give it to hertanto metro & devonte,00FU5uypELwgQuM9IbFC6G,0.6,0.583,9.0,-6.617,0.0,0.281,...,5.0,209053.0,00FU5uypELwgQuM9IbFC6G,0.0,47.0,2005,1.0,00FU5uypELwgQuM9IbFC6G,"['dancehall', 'soca']",electronic


In [11]:
attributes.merge(artists_join, "left", left_on='id_y', right_on='song_id').merge(artists, "left", on="artist_id")[lambda x: x.id_y == "00FU5uypELwgQuM9IbFC6G"]

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,chart,id_fk,genre_x,genre_super,song_id,artist_id,artist,followers,popularity_y,genre_y
2552,give it to her,tanto metro & devonte,give it to hertanto metro & devonte,00FU5uypELwgQuM9IbFC6G,0.6,0.583,9.0,-6.617,0.0,0.281,...,1.0,00FU5uypELwgQuM9IbFC6G,"['dancehall', 'soca']",electronic,00FU5uypELwgQuM9IbFC6G,4mVpYIVAOE8sEdYuL8HDpd,Tanto Metro & Devonte,45956.0,44,"['dancehall', 'soca']"
2553,give it to her,tanto metro & devonte,give it to hertanto metro & devonte,00FU5uypELwgQuM9IbFC6G,0.6,0.583,9.0,-6.617,0.0,0.281,...,1.0,00FU5uypELwgQuM9IbFC6G,"['dancehall', 'soca']",electronic,00FU5uypELwgQuM9IbFC6G,4mVpYIVAOE8sEdYuL8HDpd,Tanto Metro & Devonte,46018.0,44,"['dancehall', 'soca']"


In [12]:
attributes

,Song,Performer,SongID,id,danceability,energy,key,loudness,mode,speechiness,...,timesignature,duration,id_y,explicit,popularity,release_date,chart,id_fk,genre,genre_super
0,i'm your angel,r. kelly & celine dion,i'm your angelr. kelly & celine dion,0QWFiyhOqFu9RP8qzP6z5L,0.62,0.504,0.0,-7.260,1.0,0.0301,...,4.0,332067.0,0QWFiyhOqFu9RP8qzP6z5L,0.0,52.0,1998-11-08,1.0,0QWFiyhOqFu9RP8qzP6z5L,[],r&b
1,nobody's supposed to be here,deborah cox,nobody's supposed to be heredeborah cox,51QxenFmXlJXUN9mpvxlaL,0.662,0.531,8.0,-5.300,1.0,0.0361,...,4.0,262333.0,51QxenFmXlJXUN9mpvxlaL,0.0,56.0,1998-09-17,1.0,51QxenFmXlJXUN9mpvxlaL,"['canadian pop', 'hip pop', 'neo soul', 'new j...",r&b
2,lately,divine,latelydivine,5pgkdQi8HMQHYrJCB4GLsA,0.582,0.665,2.0,-6.992,1.0,0.0517,...,4.0,218070.0,5pgkdQi8HMQHYrJCB4GLsA,0.0,13.0,1996-04-26,1.0,5pgkdQi8HMQHYrJCB4GLsA,['fijian pop'],pop
3,...baby one more time,britney spears,...baby one more timebritney spears,3MjUtNVVq3C8Fn0MP3zhXa,0.759,0.699,0.0,-5.745,0.0,0.0307,...,4.0,211066.0,3MjUtNVVq3C8Fn0MP3zhXa,0.0,79.0,1999-01-12,1.0,3MjUtNVVq3C8Fn0MP3zhXa,"['dance pop', 'pop', 'post-teen pop']",pop
4,because of you,98 degrees,because of you98 degrees,25tmiXojR7whvZDz3LpMb2,0.732,0.603,0.0,-7.624,1.0,0.0271,...,4.0,297893.0,25tmiXojR7whvZDz3LpMb2,0.0,50.0,1998-01-01,1.0,25tmiXojR7whvZDz3LpMb2,"['boy band', 'dance pop']",pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21916,Soon as I Get Home,Faith Evans,soon as i get homefaith evans,6SkGfPa77E4giShVbk9N6R,0.652,0.498,11.0,-6.200,1.0,0.0413,...,4.0,324280.0,6SkGfPa77E4giShVbk9N6R,0.0,58.0,1995,0.0,2fSDr3jBjwJsHBWwvvezgj,"['contemporary r&b', 'hip hop', 'hip pop', 'ne...",r&b
21917,Be Without You - Kendu Mix,Mary J. Blige,be without you - kendu mixmary j. blige,6Y3WvyUG9iE5bQYg38SPtQ,0.726,0.7,2.0,-5.881,0.0,0.0998,...,4.0,246333.0,6Y3WvyUG9iE5bQYg38SPtQ,1.0,68.0,2005-01-01,0.0,2fSDr3jBjwJsHBWwvvezgj,"['contemporary r&b', 'dance pop', 'hip pop', '...",r&b
21918,Swing My Way,K.P. & Envyi,swing my wayk.p. & envyi,1DwrNN815hUHka1Z7qtvcM,0.557,0.865,9.0,-6.075,0.0,0.139,...,4.0,249213.0,1DwrNN815hUHka1Z7qtvcM,0.0,48.0,1997-10-27,0.0,2fSDr3jBjwJsHBWwvvezgj,['atlanta bass'],rap
21919,Yoü And I,Lady Gaga,you and ilady gaga,6rkAY9rk1NTFB94QxG3LJR,0.518,0.709,2.0,-5.234,1.0,0.0476,...,4.0,307347.0,6rkAY9rk1NTFB94QxG3LJR,0,60.0,2011-01-01,1.0,6rkAY9rk1NTFB94QxG3LJR,['pop'],pop


In [13]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21921 entries, 0 to 21920
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Song              21921 non-null  object 
 1   Performer         21921 non-null  object 
 2   SongID            21921 non-null  object 
 3   id                21908 non-null  object 
 4   danceability      21907 non-null  object 
 5   energy            21907 non-null  object 
 6   key               21907 non-null  object 
 7   loudness          21907 non-null  float64
 8   mode              21907 non-null  object 
 9   speechiness       21907 non-null  object 
 10  acousticness      21907 non-null  object 
 11  instrumentalness  21907 non-null  object 
 12  liveness          21907 non-null  object 
 13  valence           21907 non-null  object 
 14  tempo             21907 non-null  object 
 15  timesignature     21847 non-null  object 
 16  duration          21907 non-null  object

In [14]:
# attribute types: categorical = ['key', 'mode', 'timesignature', 'explicit']
# interval with values 0 - 1 = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']
# interval with values -60 - 0 (db) = ['loudness']
# interval with values 0 - 100 = ['popularity]
# ratio with various ranges (differences matter) = 
# {'tempo': beats per minute, 'duration': ms duration of track}
spotify_attributes = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'explicit', 'popularity']

In [15]:
for s in spotify_attributes:
    attributes[s] = attributes[s].astype('float')

In [16]:
attributes.columns

Index(['Song', 'Performer', 'SongID', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'timesignature', 'duration', 'id_y',
       'explicit', 'popularity', 'release_date', 'chart', 'id_fk', 'genre',
       'genre_super'],
      dtype='object')

In [17]:
attributes.genre_super.unique()

array(['r&b', 'pop', 'country', 'indie', 'rock', 'rap', 'missing',
       'christian', 'metal', 'electronic', 'empty', 'latin', 'soundtrack',
       'jazz', 'reggae', 'other'], dtype=object)

In [18]:
# Radar Chart (chart=1 and release_date=specified year and chart=0)
polar_categories = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']
fig = go.Figure()
data = attributes.groupby('genre_super', as_index=False).mean().loc[:, ["genre_super"] + polar_categories]

for genre in data.genre_super.unique():
    df = data[lambda x: x.genre_super == genre]
    fig.add_trace(go.Scatterpolar(
        r=[df[col].values[0] for col in df.columns if col in polar_categories],
        theta = polar_categories,
        fill='toself',
        name=df["genre_super"].values[0]
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()


In [19]:
# Radar Chart for noncharters
polar_categories = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']
fig = go.Figure()
data = attributes.query("chart == 0").groupby('genre_super', as_index=False).mean().loc[:, ["genre_super"] + polar_categories]

for genre in data.genre_super.unique():
    df = data[lambda x: x.genre_super == genre]
    fig.add_trace(go.Scatterpolar(
        r=[df[col].values[0] for col in df.columns if col in polar_categories],
        theta = polar_categories,
        fill='toself',
        name=df["genre_super"].values[0]
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()

In [20]:
# Radar Chart for charters
polar_categories = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']
fig = go.Figure()
data = attributes.query("chart == 1").groupby('genre_super', as_index=False).mean().loc[:, ["genre_super"] + polar_categories]

for genre in data.genre_super.unique():
    df = data[lambda x: x.genre_super == genre]
    fig.add_trace(go.Scatterpolar(
        r=[df[col].values[0] for col in df.columns if col in polar_categories],
        theta = polar_categories,
        fill='toself',
        name=df["genre_super"].values[0]
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()

In [21]:
# Distribution Exploration VIA genre_super and chart variables # Distribution of the Spotify Attributes 
fig = px.histogram(attributes, x="energy")
fig.show()

In [24]:
fig = px.histogram(attributes[lambda x: x.chart == 0], x="energy")
fig.show()

In [25]:
fig = px.histogram(attributes[lambda x: x.chart == 1], x="energy")
fig.show()

In [26]:
fig = px.histogram(attributes[lambda x: x.chart == 1], x="popularity")
fig.show()

In [30]:
fig = px.histogram(attributes[lambda x: x.chart == 0], x="popularity")
fig.show()

In [31]:
fig = px.histogram(attributes, x="popularity", color="chart")
fig.show()

In [32]:
fig = px.histogram(attributes, x="tempo", color="chart")
fig.show()

In [22]:
# Heatmap of Attributes and Both Popularity Metrics
# group billboard songs by SongID and then sum their bill_popularity metrics and add the column to attributes
# find correlation between spotify popularity, this new popularity metric and spotify metrics

In [ ]:
# Classifying Charts: Attributes and Genre // Attributes and Chart

In [ ]:
# Show Case Important extremes: Most Popular Songs (Then and Now), Most Popular Artists (Then and Now), Un-available Songs (no attributes),
# Most Popular Genres and Subgenres. Songs from this year that are popular on Spotify today but were not back then.